In [ ]:
import_libraries = True

if import_libraries:

    import sys
    import os
    import os.path
    from pathlib import Path
    from rdflib import Graph, Namespace, RDF
    from collections import Counter
    import json

    import math
    import pandas as pd
    import numpy as np

    import open3d as o3d

    import laspy
    from sklearn.decomposition import PCA

    from scipy.spatial import ConvexHull
    from scipy.spatial.distance import euclidean

    # sys.path.append(r"C:\Users\oscar\anaconda3\envs\cvpr\Lib\site-packages")
    import geomapi
    print(dir(geomapi))  # List available modules inside geomapi

    import geomapi
    import geomapi.tools as tl
    from geomapi.nodes import PointCloudNode
    import geomapi.utils as ut
    from geomapi.utils import geometryutils as gmu
    from geomapi.nodes import PointCloudNode

    import topologicpy 
    from topologicpy.Topology import Topology
    from topologicpy.Vertex import Vertex
    from topologicpy.Edge import Edge
    from topologicpy.Wire import Wire
    from topologicpy.Face import Face
    from topologicpy.Shell import Shell
    from topologicpy.Cell import Cell
    from topologicpy.CellComplex import CellComplex

    from topologicpy.Vector import Vector
    from topologicpy.Plotly import Plotly

    import matplotlib.pyplot as plt
    from mpl_toolkits.mplot3d import Axes3D
    from matplotlib import colormaps
    import plotly.graph_objects as go

    import geopy
    from geopy.geocoders import Nominatim
    import math as m
    import pvlib
    import pandas as pd
    from datetime import datetime

    import context_KUL
    import utils_KUL as kul
    import utils_energy as en
        

A - GEOMETRIC MODEL

In [ ]:
folders = True

if folders:

    current_directory = Path(os.getcwd()) 
    print(f'Current directory: {current_directory}')

    data_base = Path.cwd().parents[0] / 'data'
    laz_files = list(data_base.glob('**/*.laz'))
    print(f'Found n. {len(laz_files)} in laz folder')

    input_folder_root = Path.cwd().parents[0] / 'outputs' / 'edts'

    print(f"Input root folder: {input_folder_root}")


    names = []

    for name_folder in input_folder_root.iterdir():
        if name_folder.is_dir():  # Ensure it's a directory
            names.append(name_folder.name)
            print(f"Found folder: {name_folder.name}")
            
            json_folder = name_folder / 'json'

            if json_folder.exists():

                json_files = list(json_folder.glob('*.json'))
                print(f"JSON files in '{name_folder.name}' folder:")
                
                for json_file in json_files:
                    filename = json_file.stem
                    extracted_name = filename.split('_walls')[0] if '_walls' in filename else filename
                    print(f"Original filename: {json_file.name}, Extracted name: {extracted_name}")
            else:
                print(f"No 'json' folder found in {name_folder.name}")
            

    print("\nAll files and folders in 'edts' root folder:")
    for file in input_folder_root.glob('*'):
        print(file)


In [ ]:
first_floor_folder = True

if first_floor_folder:

    from pathlib import Path
    import os

    current_directory = Path(os.getcwd()) 
    print(f'Current directory: {current_directory}')

    data_base = Path.cwd().parents[0] / 'data'
    laz_files = list(data_base.glob('**/*.laz'))
    print(f'Found n. {len(laz_files)}\n     in laz folder {data_base}')

    input_folder_root = Path.cwd().parents[0] / 'outputs' / 'edts'
    print(f"Input root folder: {input_folder_root}")

    target_folder = input_folder_root / 'first_floor'

    if target_folder.exists() and target_folder.is_dir():
        print(f"Working on folder: {target_folder.name}")

        json_folder = target_folder / 'json'

        if json_folder.exists():
            json_files = list(json_folder.glob('*.json'))
            print(f"JSON files in '{target_folder.name}' folder:")

            for json_file in json_files:
                filename = json_file.stem
                extracted_name = filename.split('_walls')[0] if '_walls' in filename else filename
                print(f"Original filename: {json_file.name}, Extracted name: {extracted_name}")
        else:
            print(f"No 'json' folder found in {target_folder.name}")

    else:
        print(f"Folder 'first_floor' does not exist in {input_folder_root}")


BUILDING ELEMENT

In [ ]:
extract_topo_info = True

if extract_topo_info:

    walls_data_dict = {}

    for name in names:
        
        input_json_walls_file = Path.cwd().parents[0] / 'outputs' / 'edts' / name / 'json' / f'{name}_walls.json'
        print(f"Checking path: {input_json_walls_file}")
        
        if input_json_walls_file.exists():
            print(f"Found file: {input_json_walls_file}")
            try:
                with input_json_walls_file.open() as file:
                    data = json.load(file)
                    print(f"Loaded data for {name}: {data}")

                extracted_data = []

                # Extract walls from the 'first_floor' key
                if "first_floor" in data:
                    walls = data["first_floor"]
                    if isinstance(walls, list):
                        for item in walls:
                            wall_data = {
                                "wall_index": item.get("wall_index"),
                                "center": item.get("center"),
                                "height": item.get("height"),
                                "thickness": item.get("thickness"),
                                "length": item.get("length"),
                                "rotation": item.get("rotation"),
                                "base_center_lines": item.get("base_center_lines"),
                                "top_center_lines": item.get("top_center_lines"),
                                "surface_vertices": item.get("surface_vertices"),
                                "wall_id": item.get("wall_id"),
                                "base_line_start": item.get("base_line", {}).get("start"),
                                "base_line_end": item.get("base_line", {}).get("end")
                            }
                            extracted_data.append(wall_data)
                    else:
                        print(f"Unexpected data structure for walls in {input_json_walls_file}: {walls}")
                else:
                    print(f"Key 'first_floor' not found in {input_json_walls_file}")

                walls_data_dict[name] = extracted_data
                print(f"Data extracted for {name}: {extracted_data}")

            except json.JSONDecodeError as e:
                print(f"Error decoding JSON in file {input_json_walls_file}: {e}")
        else:
            print(f"File not found: {input_json_walls_file}")

    if walls_data_dict:
        #print("Final data dictionary:", walls_data_dict)
        print("Final data dictionary found!")
    else:
        print("No data was added to the data dictionary.")

    import json
    from pathlib import Path

    surfaces_data_dict = {}

    for name in names:
        input_json_surfaces_file = Path.cwd().parents[0] / 'outputs' / 'edts' / name / 'json' / f'{name}_walls_centerline_surfaces.json'
        print(f"Checking path: {input_json_surfaces_file}")

        if input_json_surfaces_file.exists():   
            print(f"✅ Found file: {input_json_surfaces_file}")
            try:
                with input_json_surfaces_file.open() as file:
                    data = json.load(file)
                    print(f"📦 Loaded data for {name}")

                extracted_data = []

                # Expecting flat list of centerline face dictionaries
                if isinstance(data, list):
                    for item in data:
                        if item.get("type") == "centerline":
                            wall_data = {
                                "wall_id": item.get("wall_id"),
                                "vertices": item.get("vertices"),  # This is a list of 4 3D points
                            }
                            extracted_data.append(wall_data)
                else:
                    print(f"⚠️ Unexpected data structure in {input_json_surfaces_file}: {type(data)}")

                surfaces_data_dict[name] = extracted_data
                print(f"✅ Surfaces extracted for {name}, total: {len(extracted_data)}")

            except json.JSONDecodeError as e:
                print(f"❌ Error decoding JSON in file {input_json_surfaces_file}: {e}")
        else:
            print(f"❌ File not found: {input_json_surfaces_file}")



    openings_data_dict = {}

    for name in names:
        input_json_openings_file = Path.cwd().parents[0] / 'outputs' / 'edts' / name / 'json' / f'{name}_openings.json'
        print(f"🔍 Checking path: {input_json_openings_file}")

        if input_json_openings_file.exists():
            try:
                with input_json_openings_file.open() as file:
                    data = json.load(file)
                    print(f"✅ Loaded data for {name}")

                extracted_data = []

                if "openings" in data:
                    openings = data["openings"]

                    # ---- DOORS ----
                    if "doors" in openings and isinstance(openings["doors"], list):
                        for door in openings["doors"]:
                            door_data = {
                                "id": door.get("id"),
                                "center": door.get("center"),
                                "bbox_vertices": door.get("bbox_vertices"),
                                "height": door.get("height"),
                                "thickness": door.get("thickness"),
                                "length": door.get("length"),
                                "rotation": door.get("rotation"),
                                "type": "door"
                            }

                            bbox = door_data["bbox_vertices"]
                            center = door_data["center"]
                            length = door_data["length"]
                            thickness = door_data["thickness"]
                            height = door_data["height"]
                            rotation = door_data["rotation"]

                            if not bbox or len(bbox) < 8 or any(len(pt) != 3 for pt in bbox):
                                print(f"⚠️ Fixing malformed DOOR bbox for ID {door_data.get('id')}")

                                try:
                                    # if all(isinstance(x, (int, float)) for x in [length, thickness, height, rotation]) and isinstance(center, list) and len(center) == 3:
                                    if isinstance(center, (list, tuple)) and len(center) == 3 and all(isinstance(c, (int, float)) for c in center):

                                        cx, cy, cz = center
                                        half_l, half_t = length / 2, thickness / 2
                                        box = np.array([
                                            [-half_l, -half_t, 0], [half_l, -half_t, 0],
                                            [half_l, half_t, 0], [-half_l, half_t, 0],
                                            [-half_l, -half_t, height], [half_l, -half_t, height],
                                            [half_l, half_t, height], [-half_l, half_t, height],
                                        ])
                                        rot_matrix = np.array([
                                            [np.cos(rotation), -np.sin(rotation), 0],
                                            [np.sin(rotation),  np.cos(rotation), 0],
                                            [0,                 0,                1]
                                        ])
                                        rotated_box = (rot_matrix @ box.T).T
                                        translated_box = rotated_box + np.array([cx, cy, cz])
                                        door_data["bbox_vertices"] = translated_box.tolist()
                                    else:
                                        print(f"🚫 Skipping malformed door data: {door_data.get('id')}")
                                except Exception as e:
                                    print(f"❌ Failed to fix bbox for door {door_data.get('id')}: {e}")

                            extracted_data.append(door_data)

                    # ---- WINDOWS ----
                    if "windows" in openings and isinstance(openings["windows"], list):
                        for window in openings["windows"]:
                            window_data = {
                                "id": window.get("id"),
                                "center": window.get("center"),
                                "bbox_vertices": window.get("bbox_vertices"),
                                "height": window.get("height"),
                                "thickness": window.get("thickness"),
                                "length": window.get("length"),
                                "rotation": window.get("rotation"),
                                "type": "window"
                            }

                            bbox = window_data["bbox_vertices"]
                            center = window_data["center"]
                            length = window_data["length"]
                            thickness = window_data["thickness"]
                            height = window_data["height"]
                            rotation = window_data["rotation"]

                            if not bbox or len(bbox) < 8 or any(len(pt) != 3 for pt in bbox):
                                print(f"⚠️ Fixing malformed WINDOW bbox for ID {window_data.get('id')}")

                                try:
                                    if all(isinstance(x, (int, float)) for x in [length, thickness, height]) and isinstance(rotation, (int, float)):

                                        cx, cy, cz = center
                                        half_l, half_t = length / 2, thickness / 2
                                        box = np.array([
                                            [-half_l, -half_t, 0], [half_l, -half_t, 0],
                                            [half_l, half_t, 0], [-half_l, half_t, 0],
                                            [-half_l, -half_t, height], [half_l, -half_t, height],
                                            [half_l, half_t, height], [-half_l, half_t, height],
                                        ])
                                        rot_matrix = np.array([
                                            [np.cos(rotation), -np.sin(rotation), 0],
                                            [np.sin(rotation),  np.cos(rotation), 0],
                                            [0,                 0,                1]
                                        ])
                                        rotated_box = (rot_matrix @ box.T).T
                                        translated_box = rotated_box + np.array([cx, cy, cz])
                                        window_data["bbox_vertices"] = translated_box.tolist()
                                    else:
                                        print(f"🚫 Skipping malformed window data: {window_data.get('id')}")
                                except Exception as e:
                                    print(f"❌ Failed to fix bbox for window {window_data.get('id')}: {e}")

                            extracted_data.append(window_data)

                else:
                    print(f"⚠️ Key 'openings' not found in {input_json_openings_file}")

                openings_data_dict[name] = extracted_data
                print(f"📦 Openings data extracted for {name}: {len(extracted_data)} items")

            except Exception as e:
                print(f"❌ Error processing {input_json_openings_file}: {e}")
        else:
            print(f"❌ File not found: {input_json_openings_file}")
             

In [ ]:
walls_data_dict

In [ ]:
openings_data_dict = {'08_ShortOffice_01_F1_small_pred': [],
 '08_ShortOffice_01_F2_small_pred': [],
 'block_N_1': [],
 'first_floor': [{'id': 'door_0',
   'center': [-48.26234316138794, 21.60752863185751, 3.709],
   'bbox_vertices': [[-49.257737033967906, 19.993910486942273, 3.709],
    [-46.47165808461158, 22.230440692247974, 3.709],
    [-47.26694928880797, 23.221146776772745, 3.709],
    [-50.053028238164295, 20.984616571467043, 3.709],
    [-49.257737033967906, 19.993910486942273, 3.709]],
   'height': 2.1990000000000003,
   'thickness': 3.227236289830472,
   'length': 3.4313701535527117,
   'rotation': None,
   'type': 'door'},
  {'id': 'door_1',
   'center': [-57.556992225883576, 14.413211471997561, 3.786],
   'bbox_vertices': [[-57.948171397337205, 13.98855129508105, 3.786],
    [-57.1645453278227, 13.989722579297524, 3.786],
    [-57.165813054429954, 14.837871648914073, 3.786],
    [-57.94943912394446, 14.836700364697599, 3.786],
    [-57.948171397337205, 13.98855129508105, 3.786]],
   'height': 2.076,
   'thickness': 0.8493203538330221,
   'length': 0.6348937961217572,
   'rotation': None,
   'type': 'door'},
  {'id': 'door_2',
   'center': [-53.387000745711084, 18.494822418232303, 4.602],
   'bbox_vertices': [[-53.38973973480698, 18.37965848811881, 4.602],
    [-53.39630905557106, 18.60964222587891, 4.602],
    [-53.384261756615196, 18.609986348345796, 4.602],
    [-53.37769243585112, 18.3800026105857, 4.602],
    [-53.38973973480698, 18.37965848811881, 4.602]],
   'height': 1.0419999999999998,
   'thickness': 0.2303278602269856,
   'length': 0.8,
   'rotation': None,
   'type': 'door'},
  {'id': 'door_3',
   'center': [-52.87394161401309, 19.83433065646058, 3.694],
   'bbox_vertices': [[-53.353156408723216, 19.726924937141153, 3.694],
    [-52.392122484779314, 19.739287851809284, 3.694],
    [-52.394726819302974, 19.941736375780003, 3.694],
    [-53.355760743246876, 19.929373461111872, 3.694],
    [-53.353156408723216, 19.726924937141153, 3.694]],
   'height': 2.3529999999999998,
   'thickness': 0.21481143863885066,
   'length': 0.8136382584675615,
   'rotation': None,
   'type': 'door'},
  {'id': 'door_4',
   'center': [-45.214981576857845, 14.188057025107504, 3.715],
   'bbox_vertices': [[-45.92829337013968, 14.062558491159223, 3.715],
    [-44.49920953573739, 14.07744969439033, 3.715],
    [-44.501669783576006, 14.313555559055784, 3.715],
    [-45.93075361797829, 14.298664355824677, 3.715],
    [-45.92829337013968, 14.062558491159223, 3.715]],
   'height': 2.449,
   'thickness': 0.2509970678965612,
   'length': 1.2815440822408974,
   'rotation': None,
   'type': 'door'},
  {'id': 'door_5',
   'center': [-33.7741493702169, 22.26971339382088, 3.771],
   'bbox_vertices': [[-34.03640057447143, 21.89629069014392, 3.771],
    [-33.318091842747535, 22.254490339170303, 3.771],
    [-33.511898165962364, 22.643136097497838, 3.771],
    [-34.23020689768626, 22.284936448471456, 3.771],
    [-34.03640057447143, 21.89629069014392, 3.771]],
   'height': 2.0700000000000003,
   'thickness': 0.7468454073539164,
   'length': 0.7621150549387238,
   'rotation': None,
   'type': 'door'},
  {'id': 'door_6',
   'center': [-32.36510746903901, 20.589994650185808, 3.081],
   'bbox_vertices': [[-32.75010254551739, 20.17896610902573, 3.081],
    [-31.891725615512417, 20.284912216551056, 3.081],
    [-31.980112392560628, 21.001023191345887, 3.081],
    [-32.83848932256561, 20.895077083820564, 3.081],
    [-32.75010254551739, 20.17896610902573, 3.081]],
   'height': 2.7420000000000004,
   'thickness': 0.822057082320157,
   'length': 0.79676370705319,
   'rotation': None,
   'type': 'door'},
  {'id': 'door_7',
   'center': [-36.166244231741686, 18.37081254568707, 3.727],
   'bbox_vertices': [[-36.5613844549713, 17.576288075298, 3.727],
    [-35.56406870586513, 17.719053720586647, 3.727],
    [-35.77110400851207, 19.16533701607614, 3.727],
    [-36.76841975761823, 19.022571370787492, 3.727],
    [-36.5613844549713, 17.576288075298, 3.727]],
   'height': 2.922,
   'thickness': 1.5890489407781416,
   'length': 1.0543510517531005,
   'rotation': None,
   'type': 'door'},
  {'id': 'door_8',
   'center': [-52.76115807150357, 14.108100099834473, 3.715],
   'bbox_vertices': [[-53.82098573643327, 14.002588712055324, 3.715],
    [-51.7080711161465, 13.948804159025498, 3.715],
    [-51.70133040657386, 14.213611487613623, 3.715],
    [-53.81424502686063, 14.26739604064345, 3.715],
    [-53.82098573643327, 14.002588712055324, 3.715]],
   'height': 2.4620000000000006,
   'thickness': 0.3185918816179516,
   'length': 1.9696553298594126,
   'rotation': None,
   'type': 'door'},
  {'id': 'door_9',
   'center': [-40.389068989380746, 22.227946683259795, 3.723],
   'bbox_vertices': [[-41.44891589205341, 21.920436654821746, 3.723],
    [-39.296036878231355, 22.075898405758466, 3.723],
    [-39.32922208670808, 22.535456711697847, 3.723],
    [-41.48210110053014, 22.379994960761127, 3.723],
    [-41.44891589205341, 21.920436654821746, 3.723]],
   'height': 2.1640000000000006,
   'thickness': 0.6150200568761015,
   'length': 2.036064222298782,
   'rotation': None,
   'type': 'door'},
  {'id': 'door_10',
   'center': [-49.57288763592143, 17.724170972802163, 3.786],
   'bbox_vertices': [[-49.790751601281386, 16.51869758233216, 3.786],
    [-49.95526771887104, 18.88796488698846, 3.786],
    [-49.35502367056148, 18.929644363272168, 3.786],
    [-49.19050755297182, 16.560377058615867, 3.786],
    [-49.790751601281386, 16.51869758233216, 3.786]],
   'height': 2.0100000000000002,
   'thickness': 2.4109467809400087,
   'length': 0.6147601658992187,
   'rotation': None,
   'type': 'door'},
  {'id': 'door_11',
   'center': [-37.0699717260175, 14.695214764773944, 3.728],
   'bbox_vertices': [[-37.56628915313489, 14.395993127167076, 3.728],
    [-36.54365229294253, 14.452619564408812, 3.728],
    [-36.57365429890012, 14.994436402380812, 3.728],
    [-37.59629115909247, 14.937809965139076, 3.728],
    [-37.56628915313489, 14.395993127167076, 3.728]],
   'height': 2.1220000000000003,
   'thickness': 0.5984432752137359,
   'length': 0.902638866149941,
   'rotation': None,
   'type': 'door'},
  {'id': 'door_12',
   'center': [-55.841525103929484, 15.817708418505642, 3.742],
   'bbox_vertices': [[-55.976630478739295, 15.46310737019959, 3.742],
    [-55.77014429356693, 16.190401517011956, 3.742],
    [-55.70641972911967, 16.172309466811694, 3.742],
    [-55.91290591429203, 15.44501531999933, 3.742],
    [-55.976630478739295, 15.46310737019959, 3.742]],
   'height': 2.1160000000000005,
   'thickness': 0.7453861970126265,
   'length': 0.8,
   'rotation': None,
   'type': 'door'},
  {'id': 'door_13',
   'center': [-38.839399559121986, 14.412920787768341, 3.914],
   'bbox_vertices': [[-39.05850995514739, 14.40493134682952, 3.914],
    [-38.637030819381785, 14.328540162095802, 3.914],
    [-38.62028916309658, 14.420910228707163, 3.914],
    [-39.04176829886218, 14.49730141344088, 3.914],
    [-39.05850995514739, 14.40493134682952, 3.914]],
   'height': 1.9380000000000002,
   'thickness': 0.16876125134507802,
   'length': 0.8,
   'rotation': None,
   'type': 'door'},
  {'id': 'door_14',
   'center': [-35.594437007547945, 20.346042590519822, 3.758],
   'bbox_vertices': [[-35.94794050453419, 20.150645007509564, 3.758],
    [-35.20382115925766, 20.44882486959659, 3.758],
    [-35.2409335105617, 20.54144017353008, 3.758],
    [-35.985052855838234, 20.243260311443056, 3.758],
    [-35.94794050453419, 20.150645007509564, 3.758]],
   'height': 2.0460000000000003,
   'thickness': 0.3907951660205171,
   'length': 0.6312316965805763,
   'rotation': None,
   'type': 'door'},
  {'id': 'door_15',
   'center': [-53.101885337827355, 17.79310510683726, 3.786],
   'bbox_vertices': [[-53.766720496401085, 17.433175135950925, 3.786],
    [-53.35965413332401, 18.5038164547741, 3.786],
    [-52.437050179253625, 18.15303507772359, 3.786],
    [-52.8441165423307, 17.082393758900416, 3.786],
    [-53.766720496401085, 17.433175135950925, 3.786]],
   'height': 2.046,
   'thickness': 1.4214226958736838,
   'length': 1.1796703171474605,
   'rotation': None,
   'type': 'door'},
  {'id': 'door_16',
   'center': [-54.215599284318685, 15.85556964530438, 4.571],
   'bbox_vertices': [[-54.50012697323607, 15.822622362664973, 4.571],
    [-53.93209775601289, 15.814723438108839, 4.571],
    [-53.931071595401306, 15.888516927943789, 4.571],
    [-54.49910081262449, 15.896415852499922, 4.571],
    [-54.50012697323607, 15.822622362664973, 4.571]],
   'height': 0.6340000000000003,
   'thickness': 0.125,
   'length': 0.4190553778347649,
   'rotation': None,
   'type': 'door'},
  {'id': 'window_0',
   'center': [-49.60468316412065, 6.448395849255596, 4.594],
   'bbox_vertices': [[-53.06259469410743, 6.212827082765682, 4.594],
    [-46.14101698181599, 6.32325045173699, 4.594],
    [-46.14677163413388, 6.68396461574551, 4.594],
    [-53.06834934642532, 6.5735412467742025, 4.594],
    [-53.06259469410743, 6.212827082765682, 4.594]],
   'height': 1.3330000000000002,
   'thickness': 0.47113753297982797,
   'length': 6.9273323646093345,
   'rotation': None,
   'type': 'window'},
  {'id': 'window_1',
   'center': [-56.364505618284014, 15.372934301525948, 3.095],
   'bbox_vertices': [[-57.005694051501244, 15.189214196275111, 3.095],
    [-55.72520226196559, 15.182758323639808, 3.095],
    [-55.723317185066776, 15.556654406776786, 3.095],
    [-57.003808974602435, 15.563110279412088, 3.095],
    [-57.005694051501244, 15.189214196275111, 3.095]],
   'height': 3.562,
   'thickness': 0.3803519557722801,
   'length': 1.2823768664344684,
   'rotation': None,
   'type': 'window'},
  {'id': 'window_2',
   'center': [-35.54820282949129, 30.153614653477263, 4.519],
   'bbox_vertices': [[-38.96979649365527, 29.884558026147786, 4.519],
    [-32.120653717623654, 29.9758458229912, 4.519],
    [-32.12660916532731, 30.422671280806732, 4.519],
    [-38.97575194135892, 30.33138348396332, 4.519],
    [-38.96979649365527, 29.884558026147786, 4.519]],
   'height': 1.3929999999999998,
   'thickness': 0.5381132546589456,
   'length': 6.8550982237352684,
   'rotation': None,
   'type': 'window'},
  {'id': 'window_3',
   'center': [-28.05952792411444, 22.985801754639994, 4.4350000000000005],
   'bbox_vertices': [[-31.2784589877226, 22.7695732727201, 4.4350000000000005],
    [-24.835598251516263, 22.865180799595013, 4.4350000000000005],
    [-24.84059686050628, 23.202030236559885, 4.4350000000000005],
    [-31.283457596712616, 23.10642270968497, 4.4350000000000005],
    [-31.2784589877226, 22.7695732727201, 4.4350000000000005]],
   'height': 1.4589999999999996,
   'thickness': 0.4324569638397868,
   'length': 6.447859345196353,
   'rotation': None,
   'type': 'window'},
  {'id': 'window_4',
   'center': [-57.32198512130534, 22.516691236764608, 4.572],
   'bbox_vertices': [[-60.6057959641512, 22.258492757279452, 4.572],
    [-54.03358124246423, 22.325693760280885, 4.572],
    [-54.03817427845949, 22.774889716249756, 4.572],
    [-60.610389000146455, 22.707688713248324, 4.572],
    [-60.6057959641512, 22.258492757279452, 4.572]],
   'height': 1.263,
   'thickness': 0.5163969589703044,
   'length': 6.576807757682225,
   'rotation': None,
   'type': 'window'},
  {'id': 'window_5',
   'center': [-54.581020785376204, 13.530897460871156, 3.722],
   'bbox_vertices': [[-55.25937014802137, 13.314106281019981, 3.722],
    [-53.88591493072347, 13.376029374133248, 3.722],
    [-53.90267142273104, 13.74768864072233, 3.722],
    [-55.27612664002894, 13.685765547609064, 3.722],
    [-55.25937014802137, 13.314106281019981, 3.722]],
   'height': 2.3080000000000003,
   'thickness': 0.43358235970234915,
   'length': 1.3902117093054684,
   'rotation': None,
   'type': 'window'},
  {'id': 'window_6',
   'center': [-59.5264104336982, 13.482943117932509, 3.7680000000000002],
   'bbox_vertices': [[-60.395055249283324,
     13.310182304605785,
     3.7680000000000002],
    [-58.655433102102734, 13.322354817888428, 3.7680000000000002],
    [-58.65776561811309, 13.65570393125923, 3.7680000000000002],
    [-60.39738776529368, 13.643531417976588, 3.7680000000000002],
    [-60.395055249283324, 13.310182304605785, 3.7680000000000002]],
   'height': 2.889,
   'thickness': 0.3455216266534453,
   'length': 1.7419546631909455,
   'rotation': None,
   'type': 'window'},
  {'id': 'window_7',
   'center': [-42.40819417017463, 6.5567771055951525, 4.622],
   'bbox_vertices': [[-45.83843881405148, 6.357764756494404, 4.622],
    [-38.974340981254144, 6.4349718968351945, 4.622],
    [-38.97794952629778, 6.755789454695901, 4.622],
    [-45.84204735909511, 6.678582314355111, 4.622],
    [-45.83843881405148, 6.357764756494404, 4.622]],
   'height': 1.3660000000000005,
   'thickness': 0.398024698201497,
   'length': 6.8677063778409675,
   'rotation': None,
   'type': 'window'},
  {'id': 'window_8',
   'center': [-35.167651719938014, 6.675526491528233, 4.53],
   'bbox_vertices': [[-38.62041668935214, 6.455850688288119, 4.53],
    [-31.71159353760679, 6.515822913347054, 4.53],
    [-31.714886750523895, 6.895202294768346, 4.53],
    [-38.62370990226925, 6.835230069709411, 4.53],
    [-38.62041668935214, 6.455850688288119, 4.53]],
   'height': 1.335,
   'thickness': 0.4393516064802272,
   'length': 6.912116364662456,
   'rotation': None,
   'type': 'window'},
  {'id': 'window_9',
   'center': [-50.084162818676404, 29.768621023130976, 4.6000000000000005],
   'bbox_vertices': [[-53.5115846557799,
     29.333278913449387,
     4.6000000000000005],
    [-46.6361842176021, 29.549106175228676, 4.6000000000000005],
    [-46.65674098157291, 30.203963132812568, 4.6000000000000005],
    [-53.53214141975071, 29.98813587103328, 4.6000000000000005],
    [-53.5115846557799, 29.333278913449387, 4.6000000000000005]],
   'height': 1.1909999999999998,
   'thickness': 0.8706842193631807,
   'length': 6.895957202148608,
   'rotation': None,
   'type': 'window'},
  {'id': 'window_10',
   'center': [-42.76767083092984, 29.83747223332064, 4.594],
   'bbox_vertices': [[-46.18840019381312, 29.432355512508177, 4.594],
    [-39.33644929208075, 29.53378212332065, 4.594],
    [-39.34694146804655, 30.242588954133097, 4.594],
    [-46.19889236977893, 30.14116234332063, 4.594],
    [-46.18840019381312, 29.432355512508177, 4.594]],
   'height': 1.2080000000000002,
   'thickness': 0.81023344162492,
   'length': 6.862443077698181,
   'rotation': None,
   'type': 'window'}],
 'villa': []}

In [ ]:
openings_data_dict

In [ ]:
surfaces_data_dict

WORKING IN 2D

In [ ]:
import matplotlib.pyplot as plt
from shapely.geometry import LineString

plot_imported_data = True

if plot_imported_data:

    fig, ax = plt.subplots(figsize=(14, 12))

    walls = surfaces_data_dict['first_floor']

    base_center_lines_2d = []
    
    for wall in walls:
        vertices = wall["vertices"]
        # Take the bottom edge (first two vertices)
        p1, p2 = vertices[0], vertices[1]
        line = LineString([(p1[0], p1[1]), (p2[0], p2[1])])
        base_center_lines_2d.append(line)

    # Plot base center lines
    for line in base_center_lines_2d:
        x, y = line.xy
        ax.plot(x, y, color='black', linewidth=1)

    # Plot settings
    ax.set_title("2D Visualization of Base Center Lines", fontsize=14)
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    ax.grid(True)
    ax.set_aspect('equal', adjustable='box')
    plt.tight_layout()
    plt.show()



In [ ]:
plot_imported_data = True

if plot_imported_data:

    import matplotlib.pyplot as plt
    from shapely.geometry import LineString
    import numpy as np
    
    def angle_between_vectors(v1, v2):

        dot_product = np.dot(v1, v2)
        norm_v1 = np.linalg.norm(v1)
        norm_v2 = np.linalg.norm(v2)
        cos_theta = dot_product / (norm_v1 * norm_v2)
        return np.arccos(np.clip(cos_theta, -1.0, 1.0)) 


    def perpendicular_distance(line1, line2):
        return line1.distance(line2)
    
    fig, ax = plt.subplots(figsize=(14, 12))

    walls = surfaces_data_dict['first_floor']

    base_center_lines_2d = []

    for wall in walls:
        vertices = wall["vertices"]
        p1, p2 = vertices[0], vertices[1]
        line = LineString([(p1[0], p1[1]), (p2[0], p2[1])])

        if line.length < 0.15:
            continue

        is_parallel = False

        for existing_line in base_center_lines_2d:

            vec1 = np.array(existing_line.coords[1]) - np.array(existing_line.coords[0])
            vec2 = np.array(line.coords[1]) - np.array(line.coords[0])

            angle = angle_between_vectors(vec1, vec2)

            if np.isclose(angle, 0, atol=0.15) or np.isclose(angle, np.pi, atol=0.15):  

                dist = perpendicular_distance(existing_line, line)
                if dist < 0.15:
                    is_parallel = True

                    if line.length > existing_line.length:
                        base_center_lines_2d.remove(existing_line)
                    break

        if not is_parallel:
            base_center_lines_2d.append(line)

    for line in base_center_lines_2d:
        x, y = line.xy
        ax.plot(x, y, color='black', linewidth=1)

    ax.set_title("2D Visualization of Base Center Lines (Filtered)", fontsize=14)
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    ax.grid(True)
    ax.set_aspect('equal', adjustable='box')
    plt.tight_layout()
    plt.show()


TEST_01

In [ ]:
first_step_all_int = True

if first_step_all_int:

    from shapely.geometry import LineString, Point
    import numpy as np
    import matplotlib.pyplot as plt

    def angle_between_vectors(v1, v2):
        v1_u = v1 / np.linalg.norm(v1)
        v2_u = v2 / np.linalg.norm(v2)
        dot_product = np.clip(np.dot(v1_u, v2_u), -1.0, 1.0)
        return np.arccos(dot_product)

    def perpendicular_distance(line1, line2):
        mid_point = np.array(line2.interpolate(0.5, normalized=True).coords[0])
        return line1.distance(Point(mid_point))

    def extend_line(line, factor):
        coords = np.array(line.coords)
        p1, p2 = coords[0], coords[1]
        direction = p2 - p1
        if np.linalg.norm(direction) == 0:
            return line
        unit_dir = direction / np.linalg.norm(direction)
        new_p1 = p1 - unit_dir * factor
        new_p2 = p2 + unit_dir * factor
        return LineString([new_p1, new_p2])


    first_step_all_int = True

    if first_step_all_int:
        walls = surfaces_data_dict['first_floor']

        base_center_lines_2d = []

        for wall in walls:
            vertices = wall.get("vertices", [])
            if len(vertices) >= 2:
                p1, p2 = vertices[0], vertices[1]
                line = LineString([(p1[0], p1[1]), (p2[0], p2[1])])

                if line.length < 0.15:
                    continue

                is_parallel = False

                for existing_line in base_center_lines_2d:
                    vec1 = np.array(existing_line.coords[1]) - np.array(existing_line.coords[0])
                    vec2 = np.array(line.coords[1]) - np.array(line.coords[0])
                    angle = angle_between_vectors(vec1, vec2)

                    if np.isclose(angle, 0, atol=0.15) or np.isclose(angle, np.pi, atol=0.15):
                        dist = perpendicular_distance(existing_line, line)
                        if dist < 0.15:
                            is_parallel = True
                            if line.length > existing_line.length:
                                base_center_lines_2d.remove(existing_line)
                            break

                if not is_parallel:
                    base_center_lines_2d.append(line)

        print(f"🧱 Total base center lines: {len(base_center_lines_2d)}")

        step = 0.005
        max_scale = 1.0

        all_intersections = []
        seen_points = set()

        for factor in np.arange(step, max_scale + step, step):
            extended_lines = [extend_line(line, factor) for line in base_center_lines_2d]

            for i, line1 in enumerate(extended_lines):
                for j in range(i + 1, len(extended_lines)):
                    line2 = extended_lines[j]
                    result = line1.intersection(line2)

                    if not result.is_empty:
                        if result.geom_type == 'Point':
                            points = [result]
                        elif result.geom_type == 'MultiPoint':
                            points = list(result.geoms)
                        elif result.geom_type == 'GeometryCollection':
                            points = [geom for geom in result.geoms if geom.geom_type == 'Point']
                        else:
                            points = []

                        for pt in points:
                            pt_coords = (round(pt.x, 6), round(pt.y, 6))

                            if pt_coords not in seen_points:
                                seen_points.add(pt_coords)
                                all_intersections.append({
                                    "line_1": i,
                                    "line_2": j,
                                    "point": pt,
                                    "scale": round(factor, 3)
                                })

        print(f"✅ Found {len(all_intersections)} unique intersection points.")


        fig, ax = plt.subplots(figsize=(10, 10))

        for line in base_center_lines_2d:
            x, y = line.xy
            ax.plot(x, y, color='red')

        for inter in all_intersections:
            point = inter['point']
            if point.geom_type == 'Point':
                ax.plot(point.x, point.y, 'bo', markersize=3)

        ax.set_aspect('equal')
        ax.set_title("All Intersections of Extended Centerlines")
        ax.grid(True)
        plt.show()



In [ ]:
extended_lines = [extend_line(line, factor) for line in base_center_lines_2d]
print(extended_lines)

In [ ]:
intersection_comp_def = True

if intersection_comp_def:

    from shapely.geometry import LineString, Point
    import numpy as np
    import matplotlib.pyplot as plt
    from matplotlib.lines import Line2D

    # === Helper Functions ===
    def angle_between_vectors(v1, v2):
        v1_u = v1 / np.linalg.norm(v1)
        v2_u = v2 / np.linalg.norm(v2)
        dot_product = np.clip(np.dot(v1_u, v2_u), -1.0, 1.0)
        return np.arccos(dot_product)

    def perpendicular_distance(line1, line2):
        mid_point = np.array(line2.interpolate(0.5, normalized=True).coords[0])
        return line1.distance(Point(mid_point))

    def extend_line(line, factor=1.5):
        coords = np.array(line.coords)
        p1, p2 = coords[0], coords[1]
        direction = p2 - p1
        if np.linalg.norm(direction) == 0:
            return line
        unit_dir = direction / np.linalg.norm(direction)
        new_p1 = p1 - unit_dir * factor
        new_p2 = p2 + unit_dir * factor
        return LineString([new_p1, new_p2])

    def find_neighbors(intersections, target_point, neighborhood_radius):
        neighborhood = target_point.buffer(neighborhood_radius)
        return [inter for inter in intersections if inter['point'].within(neighborhood)]

    # === Main Process ===
    filter_all_intersections = True

    if filter_all_intersections:

        walls = surfaces_data_dict['first_floor']
        base_center_lines_2d = []

        for wall in walls:
            vertices = wall.get("vertices", [])
            if len(vertices) >= 2:
                p1, p2 = vertices[0], vertices[1]
                line = LineString([(p1[0], p1[1]), (p2[0], p2[1])])
                if line.length < 0.15:
                    continue

                is_parallel = False
                for existing_line in base_center_lines_2d:
                    vec1 = np.array(existing_line.coords[1]) - np.array(existing_line.coords[0])
                    vec2 = np.array(line.coords[1]) - np.array(line.coords[0])
                    angle = angle_between_vectors(vec1, vec2)

                    if np.isclose(angle, 0, atol=0.15) or np.isclose(angle, np.pi, atol=0.15):
                        dist = perpendicular_distance(existing_line, line)
                        if dist < 0.15:
                            is_parallel = True
                            if line.length > existing_line.length:
                                base_center_lines_2d.remove(existing_line)
                            break

                if not is_parallel:
                    base_center_lines_2d.append(line)

        print(f"🧱 Total base center lines: {len(base_center_lines_2d)}")

        step = 0.005
        max_scale = 1.0

        all_intersections = []
        seen_points = set()

        for factor in np.arange(step, max_scale + step, step):
            extended_lines = [extend_line(line, factor) for line in base_center_lines_2d]

            for i, line1 in enumerate(extended_lines):
                for j in range(i + 1, len(extended_lines)):
                    line2 = extended_lines[j]
                    result = line1.intersection(line2)

                    if not result.is_empty:
                        if result.geom_type == 'Point':
                            points = [result]
                        elif result.geom_type == 'MultiPoint':
                            points = list(result.geoms)
                        elif result.geom_type == 'GeometryCollection':
                            points = [geom for geom in result.geoms if geom.geom_type == 'Point']
                        else:
                            points = []

                        for pt in points:
                            pt_coords = (round(pt.x, 6), round(pt.y, 6))

                            if pt_coords not in seen_points:
                                seen_points.add(pt_coords)
                                all_intersections.append({
                                    "line_1": i,
                                    "line_2": j,
                                    "point": pt,
                                    "scale": round(factor, 3)
                                })

        print(f"✅ Found {len(all_intersections)} unique intersection points.")

        # === Filtering Intersections ===
        min_spacing = 0.5
        filtered_intersections = []
        rejected_points = []

        for inter in all_intersections:
            point = inter['point']
            distances = [point.distance(line) for line in base_center_lines_2d]
            min_dist = min(distances)
            if min_dist <= min_spacing:
                filtered_intersections.append(inter)
            else:
                rejected_points.append((point, min_dist))
                print(f"❌ Discarded: Point {point} — distance = {min_dist:.2f}")

        print(f" {len(filtered_intersections)} intersection points after distance filtering.")
        print(f"❌ Rejected {len(rejected_points)} intersection points.")

        # === Assign Intersections to Lines ===
        extended_lines = [extend_line(line, factor=1) for line in base_center_lines_2d]
        neighborhood_radius = 2
        valid_intersections = []

        for p in filtered_intersections:
            point = p['point']
            neighbors = find_neighbors(filtered_intersections, point, neighborhood_radius)

            if len(neighbors) > 3:
                closest_line = min(extended_lines, key=lambda line: point.distance(line))
                assigned_intersection = {'point': point, 'closest_line': closest_line}
                valid_intersections.append(assigned_intersection)

        print("\nValid Intersections assigned to closest lines:")
        for inter in valid_intersections:
            print(f"Point: {inter['point']} on Line with Length: {inter['closest_line'].length:.2f}")

        # === Plotting ===
        plot_filtered_points = True

        if plot_filtered_points:
            fig, ax = plt.subplots(figsize=(12, 12))

            for line in base_center_lines_2d:
                x, y = line.xy
                ax.plot(x, y, color='red', linewidth=1)

            for inter in all_intersections:
                point = inter['point']
                ax.plot(point.x, point.y, 'bo', markersize=3, alpha=0.2)

            for inter in filtered_intersections:
                point = inter['point']
                ax.plot(point.x, point.y, 'go', markersize=5)

            for point, dist in rejected_points:
                ax.plot(point.x, point.y, 'rx', markersize=4)
                ax.text(point.x, point.y, f"{dist:.1f}", fontsize=6, color='gray')

            ax.set_aspect('equal')
            ax.set_title("Distance-Filtered Intersections of Extended Centerlines", fontsize=15)
            ax.grid(True)

            legend_elements = [
                Line2D([0], [0], color='red', lw=2, label='Base Centerlines'),
                Line2D([0], [0], marker='o', color='w', label='All Intersections', markerfacecolor='blue', markersize=6),
                Line2D([0], [0], marker='o', color='w', label='Filtered (Valid)', markerfacecolor='green', markersize=6),
                Line2D([0], [0], marker='x', color='r', label='Rejected (Too Far)', markersize=6)
            ]
            ax.legend(handles=legend_elements)

            plt.show()


In [ ]:
overlap_wall_bboxes = True

if overlap_wall_bboxes:
    from shapely.geometry import LineString, Point
    from collections import defaultdict
    import matplotlib.pyplot as plt
    from matplotlib.lines import Line2D
    import numpy as np

    def snap_point_to_nearest(p, candidates, tolerance=1):
        """Snap a point to the nearest candidate within a tolerance."""
        nearest = min(candidates, key=lambda c: p.distance(c))
        if p.distance(nearest) <= tolerance:
            return nearest
        else:
            return p

    # Build a list of all filtered intersections as snapping candidates
    intersection_points = [inter['point'] for inter in filtered_intersections]

    wall_buffers = []
    wall_id_map = []

    for walls in walls_data_dict.values():
        for wall in walls:
            wall_id = wall.get("wall_id")
            start = wall.get("base_line_start")
            end = wall.get("base_line_end")

            if start and end:
                line = LineString([tuple(start[:2]), tuple(end[:2])])
                buffered = line.buffer(0.40, cap_style=2)
                wall_buffers.append(buffered)
                wall_id_map.append(wall_id)

    line_to_points = defaultdict(list)

    for inter in filtered_intersections:
        line_to_points[inter['line_1']].append(inter['point'])
        line_to_points[inter['line_2']].append(inter['point'])

    connection_segments_defined = []

    for idx, base_line in enumerate(base_center_lines_2d):
        pts = line_to_points[idx]
        if len(pts) < 2:
            continue

        pts_sorted = sorted(pts, key=lambda p: base_line.project(p))

        for i in range(len(pts_sorted) - 1):
            p_start = pts_sorted[i]
            p_end = pts_sorted[i + 1]

            p_start_snapped = snap_point_to_nearest(p_start, intersection_points)
            p_end_snapped = snap_point_to_nearest(p_end, intersection_points)

            seg = LineString([p_start_snapped, p_end_snapped])

            # ➡️ Always create the segment
            connection_segments_defined.append({
                "segment": seg,
                "wall_id": None  # Will be updated if intersecting a wall
            })



    print(f"✅ {len(connection_segments_defined)} connection segments intersecting wall bounding boxes.")

plot_filtered_points = True

if plot_filtered_points:
    fig, ax = plt.subplots(figsize=(12, 12))

    for line in base_center_lines_2d:
        x, y = line.xy
        ax.plot(x, y, color='red', linewidth=1)

    for inter in all_intersections:
        point = inter['point']
        ax.plot(point.x, point.y, 'bo', markersize=3, alpha=0.2)

    for inter in filtered_intersections:
        point = inter['point']
        ax.plot(point.x, point.y, 'go', markersize=5)

    for point, dist in rejected_points:
        ax.plot(point.x, point.y, 'rx', markersize=4)
        ax.text(point.x, point.y, f"{dist:.1f}", fontsize=6, color='gray')

    for conn in connection_segments_defined:
        segment = conn['segment']
        x, y = segment.xy
        ax.plot(x, y, color='purple', linewidth=2, linestyle='--')

    ax.set_aspect('equal')
    ax.set_title("Distance-Filtered Intersections of Extended Centerlines", fontsize=15)
    ax.grid(True)

    legend_elements = [
        Line2D([0], [0], color='red', lw=2, label='Base Centerlines'),
        Line2D([0], [0], marker='o', color='w', label='All Intersections', markerfacecolor='blue', markersize=6),
        Line2D([0], [0], marker='o', color='w', label='Filtered (Valid)', markerfacecolor='green', markersize=6),
        Line2D([0], [0], color='purple', lw=2, linestyle='--', label='Connection Segments')
    ]
    ax.legend(handles=legend_elements)

    plt.show()

    fig2, ax2 = plt.subplots(figsize=(12, 12))

    for inter in filtered_intersections:
        point = inter['point']
        ax2.plot(point.x, point.y, 'go', markersize=5)

    for conn in connection_segments_defined:
        segment = conn['segment']
        x, y = segment.xy
        ax2.plot(x, y, color='purple', linewidth=2, linestyle='--')

    ax2.set_aspect('equal')
    ax2.set_title("Final Connection Segments with Filtered Intersections", fontsize=15)
    ax2.grid(True)

    plt.show()


In [ ]:
overlap_wall_bboxes = True

if overlap_wall_bboxes:
    from shapely.geometry import LineString, Point
    from collections import defaultdict
    import matplotlib.pyplot as plt
    from matplotlib.lines import Line2D
    import numpy as np

    def snap_point_to_nearest(p, candidates, tolerance=1):
        """Snap a point to the nearest candidate within a tolerance."""
        nearest = min(candidates, key=lambda c: p.distance(c))
        if p.distance(nearest) <= tolerance:
            return nearest
        else:
            return p

    # Build a list of all filtered intersections as snapping candidates
    intersection_points = [inter['point'] for inter in filtered_intersections]

    wall_buffers = []
    wall_id_map = []

    for walls in walls_data_dict.values():
        for wall in walls:
            wall_id = wall.get("wall_id")
            start = wall.get("base_line_start")
            end = wall.get("base_line_end")

            if start and end:
                line = LineString([tuple(start[:2]), tuple(end[:2])])
                buffered = line.buffer(0.40, cap_style=2)
                wall_buffers.append(buffered)
                wall_id_map.append(wall_id)

    line_to_points = defaultdict(list)

    for inter in filtered_intersections:
        line_to_points[inter['line_1']].append(inter['point'])
        line_to_points[inter['line_2']].append(inter['point'])

    connection_segments_defined = []

    for idx, base_line in enumerate(base_center_lines_2d):
        pts = line_to_points[idx]
        if len(pts) < 2:
            continue

        pts_sorted = sorted(pts, key=lambda p: base_line.project(p))

        for i in range(len(pts_sorted) - 1):
            p_start = pts_sorted[i]
            p_end = pts_sorted[i + 1]

            p_start_snapped = snap_point_to_nearest(p_start, intersection_points)
            p_end_snapped = snap_point_to_nearest(p_end, intersection_points)

            seg = LineString([p_start_snapped, p_end_snapped])

            # ➡️ Always create the segment
            connection_segments_defined.append({
                "segment": seg,
                "wall_id": None  # Will be updated if intersecting a wall
            })


    print(f"✅ {len(connection_segments_defined)} connection segments intersecting wall bounding boxes.")

    from collections import Counter

    # Step 1: Build endpoint list
    all_endpoints = []
    for conn in connection_segments_defined:
        seg = conn["segment"]
        start = Point(seg.coords[0])
        end = Point(seg.coords[1])
        all_endpoints.append((round(start.x, 6), round(start.y, 6)))
        all_endpoints.append((round(end.x, 6), round(end.y, 6)))

    # Step 2: Count how many times each point appears
    endpoint_counter = Counter(all_endpoints)

    # Step 3: Keep only segments where both endpoints appear more than once
    filtered_segments_connected = []

    for conn in connection_segments_defined:
        seg = conn["segment"]
        start = Point(seg.coords[0])
        end = Point(seg.coords[1])
        start_key = (round(start.x, 6), round(start.y, 6))
        end_key = (round(end.x, 6), round(end.y, 6))

        if endpoint_counter[start_key] > 1 and endpoint_counter[end_key] > 1:
            filtered_segments_connected.append(conn)

    print(f"✅ {len(filtered_segments_connected)} segments kept (connected).")
    print(f"❌ {len(connection_segments_defined) - len(filtered_segments_connected)} segments removed (unconnected).")

    # Finally: replace for plotting
    connection_segments_defined = filtered_segments_connected

    connected_points_set = set()

    for conn in connection_segments_defined:
        
        seg = conn["segment"]
        start = (round(seg.coords[0][0], 3), round(seg.coords[0][1], 3))
        end = (round(seg.coords[1][0], 3), round(seg.coords[1][1], 3))
        connected_points_set.add(start)
        connected_points_set.add(end)

    # 2. Now rebuild only the intersection points that exist at these exact places
    filtered_intersections_connected = []

    for inter in filtered_intersections:
        p = inter['point']
        key = (round(p.x, 3), round(p.y, 3))
        if key in connected_points_set:
            filtered_intersections_connected.append(inter)

    print(f"✅ {len(filtered_intersections_connected)} intersection points kept after strict filtering.")

plot_filtered_points = True

if plot_filtered_points:
    fig, ax = plt.subplots(figsize=(12, 12))

    for line in base_center_lines_2d:
        x, y = line.xy
        ax.plot(x, y, color='red', linewidth=1)

    for inter in all_intersections:
        point = inter['point']
        ax.plot(point.x, point.y, 'bo', markersize=3, alpha=0.2)

    for inter in filtered_intersections_connected:

        point = inter['point']
        ax.plot(point.x, point.y, 'go', markersize=6)

    for point, dist in rejected_points:
        ax.plot(point.x, point.y, 'rx', markersize=4)
        ax.text(point.x, point.y, f"{dist:.1f}", fontsize=6, color='gray')

    for conn in connection_segments_defined:
        segment = conn['segment']
        x, y = segment.xy
        ax.plot(x, y, color='purple', linewidth=2, linestyle='--')

    ax.set_aspect('equal')
    ax.set_title("Distance-Filtered Intersections of Extended Centerlines", fontsize=15)
    ax.grid(True)

    legend_elements = [
        Line2D([0], [0], color='red', lw=2, label='Base Centerlines'),
        Line2D([0], [0], marker='o', color='w', label='All Intersections', markerfacecolor='blue', markersize=6),
        Line2D([0], [0], marker='o', color='w', label='Filtered (Valid)', markerfacecolor='green', markersize=6),
        Line2D([0], [0], color='purple', lw=2, linestyle='--', label='Connection Segments')
    ]
    ax.legend(handles=legend_elements)

    plt.show()

    fig2, ax2 = plt.subplots(figsize=(12, 12))

    # ✅ Correct! Only plot cleaned, connected points
    for inter in filtered_intersections_connected:
        point = inter['point']
        ax2.plot(point.x, point.y, 'go', markersize=5)

    # Plot final connection segments
    for conn in connection_segments_defined:
        segment = conn['segment']
        x, y = segment.xy
        ax2.plot(x, y, color='red', linewidth=2, linestyle='--')

        # Plot filtered connected intersection points and number them
    for idx, inter in enumerate(filtered_intersections_connected):
        point = inter['point']
        ax.plot(point.x, point.y, 'go', markersize=5)
        ax.text(point.x + 0.2, point.y + 0.2, str(idx), fontsize=8, color='black')  # Offset text to avoid covering point

    ax2.set_aspect('equal')
    ax2.set_title("Final Connection Segments with Filtered Intersections", fontsize=15)
    ax2.grid(True)

    plt.show()



In [ ]:
final_segments_comp = True

if final_segments_comp:    

    import matplotlib.pyplot as plt
    from shapely.geometry import LineString, Point
    from collections import defaultdict, Counter
    import numpy as np
    from matplotlib.lines import Line2D

    # --- Step 1: Build wall buffers from walls_data_dict ---
    wall_buffers = []
    wall_id_map = []

    for walls in walls_data_dict.values():
        for wall in walls:
            wall_id = wall.get("wall_id")
            start = wall.get("base_line_start")
            end = wall.get("base_line_end")

            if start and end:
                line = LineString([tuple(start[:2]), tuple(end[:2])])
                buffered = line.buffer(0.40, cap_style=2)
                wall_buffers.append(buffered)
                wall_id_map.append(wall_id)


    line_to_points = defaultdict(list)

    for inter in filtered_intersections:
        line_to_points[inter['line_1']].append(inter['point'])
        line_to_points[inter['line_2']].append(inter['point'])

    connection_segments_defined = []

    for idx, base_line in enumerate(base_center_lines_2d):
        pts = line_to_points[idx]
        if len(pts) < 2:
            continue


        pts_sorted = sorted(pts, key=lambda p: base_line.project(p))

        for i in range(len(pts_sorted) - 1):
            p_start = pts_sorted[i]
            p_end = pts_sorted[i + 1]

            p_start_snapped = snap_point_to_nearest(p_start, intersection_points)
            p_end_snapped = snap_point_to_nearest(p_end, intersection_points)

            seg = LineString([p_start_snapped, p_end_snapped])

            connection_segments_defined.append({
                "segment": seg,
                "wall_id": None
            })

    print(f"✅ {len(connection_segments_defined)} connection segments created.")

    all_endpoints = []

    for conn in connection_segments_defined:
        seg = conn["segment"]
        start = Point(seg.coords[0])
        end = Point(seg.coords[1])
        all_endpoints.append((round(start.x, 6), round(start.y, 6)))
        all_endpoints.append((round(end.x, 6), round(end.y, 6)))

    endpoint_counter = Counter(all_endpoints)

    filtered_segments_connected = []

    for conn in connection_segments_defined:
        seg = conn["segment"]
        start = Point(seg.coords[0])
        end = Point(seg.coords[1])
        start_key = (round(start.x, 6), round(start.y, 6))
        end_key = (round(end.x, 6), round(end.y, 6))

        if endpoint_counter[start_key] > 1 and endpoint_counter[end_key] > 1:
            filtered_segments_connected.append(conn)

    print(f"✅ {len(filtered_segments_connected)} segments kept (connected).")
    print(f"❌ {len(connection_segments_defined) - len(filtered_segments_connected)} segments removed (unconnected).")

    connection_segments_defined = filtered_segments_connected

    connected_points_set = set()

    for conn in connection_segments_defined:
        seg = conn["segment"]
        start = (round(seg.coords[0][0], 6), round(seg.coords[0][1], 6))
        end = (round(seg.coords[1][0], 6), round(seg.coords[1][1], 6))
        connected_points_set.add(start)
        connected_points_set.add(end)

    filtered_intersections_connected = []

    for inter in filtered_intersections:
        p = inter['point']
        key = (round(p.x, 6), round(p.y, 6))
        if key in connected_points_set:
            filtered_intersections_connected.append(inter)

    print(f"✅ {len(filtered_intersections_connected)} intersection points kept after strict filtering.")


    # --- Step 6: Plotting the final result ---
    fig, ax = plt.subplots(figsize=(12, 12))

    # Plot connection segments
    for conn in connection_segments_defined:
        segment = conn['segment']
        x, y = segment.xy
        ax.plot(x, y, color='red', linewidth=2, linestyle='--')

        # Label wall_id at midpoint if available
        wall_id = conn.get('wall_id', None)
        if wall_id is not None:
            mid_x = (x[0] + x[1]) / 2
            mid_y = (y[0] + y[1]) / 2
            ax.text(mid_x, mid_y, str(wall_id), fontsize=8, color='blue', ha='center')

    # Plot filtered connected intersection points and number them
    for idx, inter in enumerate(filtered_intersections_connected):
        point = inter['point']
        ax.plot(point.x, point.y, 'go', markersize=5)
        ax.text(point.x + 0.2, point.y + 0.2, str(idx), fontsize=8, color='black')  # Offset text to avoid covering point

    ax.set_aspect('equal')
    ax.set_title("✅ Final Connected Segments with Intersection Indices and Wall IDs", fontsize=15)
    ax.grid(True)

    plt.show()

    final_segments = connection_segments_defined.copy()
    final_intersections = filtered_intersections_connected.copy()

    print(f"🧩 {len(final_segments)} segments frozen as final.")
    print(f"🧩 {len(final_intersections)} intersection points frozen as final.")


    from shapely.geometry import LineString

    for conn in final_segments:

        seg = conn['segment']
        
        max_overlap_length = 0.1
        best_wall_id = None

        for bbox, wall_id in zip(wall_buffers, wall_id_map):
            if seg.intersects(bbox):
                overlap = seg.intersection(bbox)
                if not overlap.is_empty:
                    if overlap.geom_type == 'LineString':
                        overlap_length = overlap.length
                    elif overlap.geom_type == 'MultiLineString':
                        overlap_length = sum(part.length for part in overlap)
                    else:
                        overlap_length = 0  # Other types not considered

                    if overlap_length > max_overlap_length:
                        max_overlap_length = overlap_length
                        best_wall_id = wall_id

        conn['wall_id'] = best_wall_id  # Assign the wall_id with max overlap

    print(f"🏷️ Wall IDs assigned to {len(final_segments)} final segments using maximum overlap.")



In [ ]:
rays_tracing = True
plot_ray_tracing = True

line_length = 0.75
min_hits_per_segment = 10

rays_tracing = True
plot_ray_tracing = True

if rays_tracing:

    import matplotlib.pyplot as plt
    from shapely.geometry import Point, LineString

    all_x = [p.x for conn in connection_segments_defined for p in [Point(conn['segment'].coords[0]), Point(conn['segment'].coords[1])]]
    all_y = [p.y for conn in connection_segments_defined for p in [Point(conn['segment'].coords[0]), Point(conn['segment'].coords[1])]]
    center_x = np.mean(all_x)
    center_y = np.mean(all_y)
    center_point = Point(center_x, center_y)

    print(f"📍 Center of scene: ({center_x:.2f}, {center_y:.2f})")

    num_rays = 360  
    angles = np.linspace(0, 2 * np.pi, num_rays, endpoint=False)
    max_ray_length = 500

    wall_lines = [conn['segment'] for conn in connection_segments_defined]

    ray_hit_points = []
    segment_hit = []

    for angle in angles:
        dx = np.cos(angle)
        dy = np.sin(angle)
        ray = LineString([
            (center_x, center_y),
            (center_x + dx * max_ray_length, center_y + dy * max_ray_length)
        ])

        ray_intersections = []
        ray_intersections_info = []

        for idx, wall in enumerate(wall_lines):
            inter = ray.intersection(wall)
            if not inter.is_empty:
                if inter.geom_type == 'Point':
                    ray_intersections.append(inter)
                    ray_intersections_info.append((inter, idx))
                elif inter.geom_type == 'MultiPoint':
                    for pt in inter.geoms:
                        ray_intersections.append(pt)
                        ray_intersections_info.append((pt, idx))

        if ray_intersections:
            furthest_inter = max(ray_intersections, key=lambda pt: center_point.distance(pt))
            furthest_idx = None
            for pt, idx in ray_intersections_info:
                if pt.equals(furthest_inter):
                    furthest_idx = idx
                    break
            if furthest_idx is not None:
                ray_hit_points.append(furthest_inter)
                segment_hit.append(furthest_idx)


    filtered_ray_hit_points = []
    segment_hit_counter = {}
    for i, pt in enumerate(ray_hit_points):
        distances = [pt.distance(other_pt) for j, other_pt in enumerate(ray_hit_points) if i != j]
        if distances and min(distances) <= line_length:
            filtered_ray_hit_points.append(pt)
            seg_id = segment_hit[i]
            if seg_id not in segment_hit_counter:
                segment_hit_counter[seg_id] = []
            segment_hit_counter[seg_id].append(pt)

    segment_hit_final = [seg_id for seg_id, pts in segment_hit_counter.items() if len(pts) >= min_hits_per_segment]
    external_segments = [connection_segments_defined[i]['segment'] for i in segment_hit_final]

    point_to_segments = {}
    for i, seg in enumerate(external_segments):
        p_start = tuple(np.round(seg.coords[0], 5))
        p_end = tuple(np.round(seg.coords[1], 5))
        point_to_segments.setdefault(p_start, []).append((i, p_end))
        point_to_segments.setdefault(p_end, []).append((i, p_start))

    visited = set()
    ordered_points = []

    start_point = list(point_to_segments.keys())[0]
    current_point = start_point
    ordered_points.append(current_point)

    while len(visited) < len(external_segments):
        candidates = point_to_segments.get(current_point, [])
        found = False
        for idx, next_point in candidates:
            if idx not in visited:
                visited.add(idx)
                ordered_points.append(next_point)
                current_point = next_point
                found = True
                break

        if not found:
            break

    if ordered_points[-1] != start_point:
        ordered_points.append(start_point)

    final_points = [ordered_points[0]]
    for i in range(1, len(ordered_points)):
        p1 = Point(ordered_points[i-1])
        p2 = Point(ordered_points[i])
        dist = p1.distance(p2)
        if dist > line_length:
            bridge = LineString([p1, p2])
            final_points.append(tuple(p2.coords[0]))
        else:
            final_points.append(ordered_points[i])

    final_polygon = LineString(final_points)

    fig, ax = plt.subplots(figsize=(10, 10))

    for conn in connection_segments_defined:
        segment = conn['segment']
        x, y = segment.xy
        ax.plot(x, y, color='gray', linewidth=1, linestyle='--')

    x, y = final_polygon.xy


    ax.plot(center_x, center_y, 'bx', markersize=10, label='Center')

    for hit in filtered_ray_hit_points:
        ax.plot([center_x, hit.x], [center_y, hit.y], color='orange', linestyle=':', linewidth=0.5)
        ax.plot(hit.x, hit.y, 'ro', markersize=3)

    ax.set_aspect('equal')
    ax.set_title(f"✅ Radial Raycast (Filtered - 2 Hit Min Per Segment)", fontsize=15)
    ax.grid(True)
    ax.legend()

    margin = 5
    ax.set_xlim(center_x - 20, center_x + 20)
    ax.set_ylim(center_y - 20, center_y + 20)

    plt.tight_layout()
    plt.show()


plotly3d_rays = False

if plotly3d_rays:

    import plotly.graph_objs as go
    import numpy as np

    # Step 1 — Prepare lists for plotly traces
    wall_traces = []
    ray_traces = []

    # Wall segments
    for conn in connection_segments_defined:
        segment = conn['segment']
        x, y = segment.xy
        wall_traces.append(
            go.Scatter3d(
                x=list(x),  # <<< Make sure to convert x to list
                y=list(y),  # <<< Make sure to convert y to list
                z=[0, 0],   # flat in Z
                mode='lines',
                line=dict(color='black', width=5),
                name='Wall Segment'
            )
        )

    # Rays from center to hit points
    for hit in ray_hit_points:
        ray_traces.append(
            go.Scatter3d(
                x=[center_x, hit.x],
                y=[center_y, hit.y],
                z=[0, 0],  # flat
                mode='lines',
                line=dict(color='orange', width=1.2, dash='dot'),
                name='Ray',
                showlegend=False
            )
        )

    # Hit points
    hit_points_trace = go.Scatter3d(
        x=[p.x for p in ray_hit_points],
        y=[p.y for p in ray_hit_points],
        z=[0] * len(ray_hit_points),
        mode='markers',
        marker=dict(size=3, color='red'),
        name='Ray Hit Points'
    )

    # Center point
    center_trace = go.Scatter3d(
        x=[center_x],
        y=[center_y],
        z=[0],
        mode='markers',
        marker=dict(size=4, color='blue', symbol='x'),
        name='Center'
    )

    # Step 2 — Build plot
    fig = go.Figure(
        data=wall_traces + ray_traces + [hit_points_trace, center_trace]
    )

    fig.update_layout(
        title="✅ Radial Raycast in Plotly 3D",
        width=1200,
        height=1000,
        showlegend=True,
        scene=dict(
            aspectmode='data',
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Z',
            camera=dict(eye=dict(x=1.2, y=1.2, z=0.8)),  # nice default zoom
        )
    )

    fig.show()


In [ ]:
wall_closure = True

if wall_closure:

    all_x = [p.x for conn in connection_segments_defined for p in [Point(conn['segment'].coords[0]), Point(conn['segment'].coords[1])]]
    all_y = [p.y for conn in connection_segments_defined for p in [Point(conn['segment'].coords[0]), Point(conn['segment'].coords[1])]]
    center_x = np.mean(all_x)
    center_y = np.mean(all_y)
    center_point = Point(center_x, center_y)

    num_rays = 360
    angles = np.linspace(0, 2 * np.pi, num_rays, endpoint=False)
    max_ray_length = 500

    wall_lines = [conn['segment'] for conn in connection_segments_defined]

    ray_hit_points = []
    segment_hit = []

    for angle in angles:
        dx = np.cos(angle)
        dy = np.sin(angle)
        ray = LineString([(center_x, center_y), (center_x + dx * max_ray_length, center_y + dy * max_ray_length)])
        ray_intersections = []
        ray_intersections_info = []
        for idx, wall in enumerate(wall_lines):
            inter = wall.intersection(ray)
            if not inter.is_empty:
                if inter.geom_type == 'Point':
                    ray_intersections.append(inter)
                    ray_intersections_info.append((inter, idx))
                elif inter.geom_type == 'MultiPoint':
                    for pt in inter.geoms:
                        ray_intersections.append(pt)
                        ray_intersections_info.append((pt, idx))
        if ray_intersections:
            furthest_inter = max(ray_intersections, key=lambda pt: center_point.distance(pt))
            for pt, idx in ray_intersections_info:
                if pt.equals(furthest_inter):
                    ray_hit_points.append(furthest_inter)
                    segment_hit.append(idx)
                    break

    filtered_ray_hit_points = []
    segment_hit_counter = {}

    for i, pt in enumerate(ray_hit_points):
        distances = [pt.distance(other_pt) for j, other_pt in enumerate(ray_hit_points) if i != j]
        if distances and min(distances) <= line_length:
            filtered_ray_hit_points.append(pt)
            seg_id = segment_hit[i]
            if seg_id not in segment_hit_counter:
                segment_hit_counter[seg_id] = []
            segment_hit_counter[seg_id].append(pt)

    segment_hit_final = [seg_id for seg_id, pts in segment_hit_counter.items() if len(pts) >= min_hits_per_segment]
    external_segments = [connection_segments_defined[i]['segment'] for i in segment_hit_final]

    line_length = 0.75

    all_points = [Point(pt) for seg in external_segments for pt in seg.coords]
    unique_coords = list(set((round(p.x, 5), round(p.y, 5)) for p in all_points))
    unique_points = [Point(x, y) for x, y in unique_coords]

    # Compute centroid for angle reference
    centroid = Point(np.mean([p.x for p in unique_points]), np.mean([p.y for p in unique_points]))

    # Sort points counterclockwise by angle from centroid
    def angle_from_centroid(p):
        return np.arctan2(p.y - centroid.y, p.x - centroid.x)

    sorted_points = sorted(unique_points, key=angle_from_centroid)

    # Detect gaps between consecutive sorted points
    bridging_segments = []
    for i in range(len(sorted_points)):
        p1 = sorted_points[i]
        p2 = sorted_points[(i + 1) % len(sorted_points)]
        if p1.distance(p2) > line_length:
            bridging_segments.append(LineString([p1, p2]))

    # Plotting
    fig, ax = plt.subplots(figsize=(8, 8))

    for idx, inter in enumerate(filtered_intersections_connected):
        point = inter['point']
        ax.plot(point.x, point.y, 'go', markersize=5)
        ax.text(point.x + 0.2, point.y + 0.2, str(idx), fontsize=8, color='black')  # Offset text to avoid covering point


    for hit in filtered_ray_hit_points:
        ax.plot([center_x, hit.x], [center_y, hit.y], color='orange', linestyle=':', linewidth=0.5)
        ax.plot(hit.x, hit.y, 'ro', markersize=2)

    for seg in external_segments:
        x, y = seg.xy
        ax.plot(x, y, color='black', linewidth=1)

    # Draw sorted points
    for pt in sorted_points:
        ax.plot(pt.x, pt.y, 'ro', markersize=4)

    # Draw bridging segments
    for seg in bridging_segments:
        x, y = seg.xy
        ax.plot(x, y, color='blue', linestyle='--', linewidth=2)

    ax.set_title("Counterclockwise Ordered Points with Bridging Segments")
    ax.set_aspect('equal')
    plt.grid(True)
    plt.tight_layout()
    plt.show()



MANUAL ADD WALL_CLOSURE (In case)

In [ ]:
manual_wall_closure_needed = True

if manual_wall_closure_needed:
        

    def add_closure_segments(id1, id2, intersection_points, connection_segments_defined, closure_segments_list):

        p1 = intersection_points[id1]['point']
        p2 = intersection_points[id2]['point']
        
        seg = LineString([p1, p2])
        
        new_seg = {
            "segment": seg,
            "wall_id": None,
            "closure": True  # ✅ Mark this segment
        }
        
        connection_segments_defined.append(new_seg)
        closure_segments_list.append(new_seg)

        print(f"➕ Closure segment added between points {id1} and {id2}.")

    closure_segments_list = []

    add_closure_segments(58, 27, filtered_intersections_connected, connection_segments_defined, closure_segments_list)
    add_closure_segments(29, 42, filtered_intersections_connected, connection_segments_defined, closure_segments_list)
    add_closure_segments(54, 7, filtered_intersections_connected, connection_segments_defined, closure_segments_list)

    import matplotlib.pyplot as plt

    fig, ax = plt.subplots(figsize=(14, 14))

    for conn in connection_segments_defined:
        segment = conn['segment']
        x, y = segment.xy
        
        if conn.get('closure', False):

            ax.plot(x, y, color='blue', linewidth=2, linestyle='-')
        else:

            ax.plot(x, y, color='red', linewidth=2, linestyle='--')

            wall_id = conn.get('wall_id')
            if wall_id is not None:
                mid_x = (x[0] + x[1]) / 2
                mid_y = (y[0] + y[1]) / 2
                ax.text(mid_x, mid_y, str(wall_id), fontsize=8, color='purple', ha='center')

    for idx, inter in enumerate(filtered_intersections_connected):
        point = inter['point']
        ax.plot(point.x, point.y, 'go', markersize=5)
        ax.text(point.x + 0.2, point.y + 0.2, str(idx), fontsize=8, color='black')

    ax.set_aspect('equal')
    ax.set_title("✅ Final Connected Segments: Walls (Red Dashed) + Closures (Blue) + Wall IDs", fontsize=15)
    ax.grid(True)

    plt.show()

    from shapely.geometry import LineString

    def add_closure_segments(id1, id2, intersection_points, connection_segments_defined, closure_segments_list):
        p1 = intersection_points[id1]['point']
        p2 = intersection_points[id2]['point']
        
        seg = LineString([p1, p2])
        
        new_seg = {
            "segment": seg,
            "wall_id": "closure",  # 🛑 Assign special wall_id 'closure'
            "closure": True        # ✅ Also mark explicitly
        }
        
        connection_segments_defined.append(new_seg)
        closure_segments_list.append(new_seg)

        print(f"➕ Closure segment added between points {id1} and {id2}.")


In [ ]:
dcel_algorithm_improved = True

if dcel_algorithm_improved:


    from shapely.geometry import LineString
    from collections import defaultdict
    import math
    import random
    import plotly.graph_objects as go


    def snap_coords(coords, precision=6):
        return (round(coords[0], precision), round(coords[1], precision))

    def compute_angle(p0, p1):
        dx, dy = p1[0] - p0[0], p1[1] - p0[1]
        angle = math.atan2(dy, dx)
        return angle if angle >= 0 else angle + 2 * math.pi

    def get_destination(eid, half_edges):
        return half_edges[half_edges[eid]['twin']]['origin']

    def build_dcel_from_segments(segments, snap_precision=6):
        vertices, point_to_vid, half_edges = {}, {}, {}
        vertex_id, edge_id = 0, 0

        for conn in segments:
            coords = list(conn['segment'].coords)
            for i in range(len(coords) - 1):
                start, end = snap_coords(coords[i], snap_precision), snap_coords(coords[i + 1], snap_precision)
                for pt in (start, end):
                    if pt not in point_to_vid:
                        point_to_vid[pt] = vertex_id
                        vertices[vertex_id] = {"coordinates": pt, "incident_edge": None}
                        vertex_id += 1
                sid, eid = point_to_vid[start], point_to_vid[end]
                he1, he2 = edge_id, edge_id + 1
                half_edges[he1] = {"origin": sid, "twin": he2, "next": None, "prev": None, "incident_face": None}
                half_edges[he2] = {"origin": eid, "twin": he1, "next": None, "prev": None, "incident_face": None}
                if vertices[sid]["incident_edge"] is None: vertices[sid]["incident_edge"] = he1
                if vertices[eid]["incident_edge"] is None: vertices[eid]["incident_edge"] = he2
                edge_id += 2

        outgoing = defaultdict(list)
        for eid, edge in half_edges.items():
            outgoing[edge["origin"]].append(eid)

        for origin_id, edges in outgoing.items():
            origin_coords = vertices[origin_id]["coordinates"]
            edges_with_angles = [(eid, compute_angle(origin_coords, vertices[get_destination(eid, half_edges)]["coordinates"])) for eid in edges]
            sorted_edges = sorted(edges_with_angles, key=lambda x: x[1])
            for i in range(len(sorted_edges)):
                curr, next_ = sorted_edges[i][0], sorted_edges[(i + 1) % len(sorted_edges)][0]
                half_edges[curr]["next"] = next_
                half_edges[next_]["prev"] = curr

        faces, visited, face_id = {}, set(), 0
        for eid in half_edges:
            if eid in visited:
                continue
            loop, current = [], eid
            while current not in loop and current is not None:
                loop.append(current)
                visited.add(current)
                current = half_edges[current]["next"]
            if current == eid and len(loop) > 2:
                for he in loop:
                    half_edges[he]["incident_face"] = face_id
                faces[face_id] = {"outer_component": eid}
                face_id += 1

        return vertices, half_edges, faces

    def plot_dcel_faces(vertices, half_edges, faces, segments):
        fig = go.Figure()

        def get_face_coords(start_eid):
            coords = []
            visited = set()
            current = start_eid
            while current not in visited and current is not None:
                visited.add(current)
                coords.append(vertices[half_edges[current]['origin']]['coordinates'])
                current = half_edges[current]['next']
            return coords

        for conn in segments:
            x, y = zip(*conn['segment'].coords)
            fig.add_trace(go.Scatter3d(
                x=list(x), y=list(y), z=[0]*len(x),
                mode="lines", line=dict(color='gray', width=2),
                showlegend=False
            ))

        for fid, fdata in faces.items():
            coords = get_face_coords(fdata['outer_component'])
            if len(coords) < 3:
                continue
            x, y = zip(*coords)
            z = [0.01] * len(x)
            i = [0] * (len(x) - 2)
            j = list(range(1, len(x) - 1))
            k = list(range(2, len(x)))

            fig.add_trace(go.Mesh3d(
                x=x, y=y, z=z,
                i=i, j=j, k=k,
                color=f'rgb({random.randint(100,255)},{random.randint(100,255)},{random.randint(100,255)})',
                opacity=0.6,
                hovertext=f"Face {fid}",
                showlegend=False
            ))

            cx = sum(x) / len(x)
            cy = sum(y) / len(y)
            fig.add_trace(go.Scatter3d(
                x=[cx], y=[cy], z=[0.05],
                mode="text",
                text=[f"F{fid}"],
                textfont=dict(size=10, color="black"),
                showlegend=False
            ))

        fig.update_layout(
            title="✅ DCEL Face Visualization",
            scene=dict(
                xaxis=dict(title="x"),
                yaxis=dict(title="y"),
                zaxis=dict(visible=False),
                aspectmode="data"
            ),
            height=700
        )

        fig.show()

    # Run
    vertices, half_edges, faces = build_dcel_from_segments(connection_segments_defined)
    print(f"✅ Segments: {len(connection_segments_defined)}, Vertices: {len(vertices)}, Faces: {len(faces)}")
    plot_dcel_faces(vertices, half_edges, faces, connection_segments_defined)


GRAPH

In [ ]:
graph_networx = True

if graph_networx:
    
    import networkx as nx
    from shapely.geometry import LineString
    import matplotlib.pyplot as plt

    def build_wall_intersections_graph(connection_segments):

        G = nx.Graph()

        for seg_data in connection_segments:
            seg = seg_data.get("segment")
            
            if seg and isinstance(seg, LineString):
                coords = list(seg.coords)

                if len(coords) == 2:  # Only process direct 2-point segments
                    p1, p2 = tuple(coords[0]), tuple(coords[1])
                    G.add_edge(p1, p2, weight=seg.length)

        print(f"📊 Graph constructed with {G.number_of_nodes()} nodes and {G.number_of_edges()} edges.")
        return G

    def plot_wall_graph(G):
        """
        Plot a NetworkX graph of wall intersections.
        """
        pos = {node: node for node in G.nodes}

        plt.figure(figsize=(8, 8))
        nx.draw(
            G,
            pos,
            node_size=30,
            node_color='green',
            edge_color='gray',
            width=1,
            with_labels=False
        )

        plt.title("📌 Graph of Connected Intersection Points (via NetworkX)", fontsize=14)
        plt.axis("equal")
        plt.grid(True)
        plt.show()

    final_segments = connection_segments_defined  # ← Your latest segments
    G = build_wall_intersections_graph(final_segments)
    plot_wall_graph(G)


OPENINGS COMPUTATION

In [ ]:
import_openigs_step_1 = True

if import_openigs_step_1:

    from shapely.geometry import LineString

    def project_openings_2d(openings_data_dict):
        """Projects 3D openings data to 2D."""
        projected_openings = {}

        for name, openings in openings_data_dict.items():
            projected_openings[name] = []

            for opening in openings:
                if opening.get("type") == "door" or opening.get("type") == "window":
                    center = opening.get("center")
                    if center and len(center) >= 2:  # Ensure there is at least 2D data for center
                        projected_openings[name].append({
                            "id": opening.get("id"),
                            "center": (center[0], center[1]),  # Project to 2D (x, y)
                            "bbox": [(vertex[0], vertex[1]) for vertex in opening.get("bbox_vertices")]  # Project bbox to 2D
                        })

        return projected_openings

    def trace_openings_edges(projected_openings_bbox):
        """Traces the edges of projected openings."""
        traced_edges = []

        for name, openings in projected_openings_bbox.items():
            for opening in openings:
                bbox = opening.get("bbox")
                if bbox and len(bbox) >= 4:  # Ensure there are at least 4 vertices for a polygon
                    line = LineString(bbox)
                    traced_edges.append({
                        "id": opening.get("id"),
                        "line": line
                    })

        return traced_edges

    def find_intersections(project_openings_2d, connection_segments_defined):
        """Finds intersections between projected openings and connection segments."""
        intersections = []

        for opening in project_openings_2d.values():
            for seg_data in connection_segments_defined:
                seg = seg_data["segment"]
                for open_data in opening:
                    line = LineString(open_data["bbox"])
                    if line.intersects(seg):
                        inter = line.intersection(seg)
                        if not inter.is_empty:
                            intersections.append({
                                "opening_id": open_data["id"],
                                "segment": seg,
                                "intersection": inter
                            })

        return intersections

    print(f'N. of segments: {len(connection_segments_defined)}')


In [ ]:
import_openigs_step_1 = True

if import_openigs_step_1:

    from shapely.geometry import LineString

    def project_openings_2d(openings_data_dict):
        """Projects 3D openings data to 2D."""
        projected_openings = {}

        for name, openings in openings_data_dict.items():
            projected_openings[name] = []

            for opening in openings:
                if opening.get("type") == "door" or opening.get("type") == "window":
                    center = opening.get("center")
                    if center and len(center) >= 2:
                        projected_openings[name].append({
                            "id": opening.get("id"),
                            "center": (center[0], center[1]), 
                            "bbox": [(vertex[0], vertex[1]) for vertex in opening.get("bbox_vertices")] 
                        })

        return projected_openings
    
    from shapely.geometry import MultiPoint, Polygon

    def build_polygon_from_graph(G):

        points = [Point(node) for node in G.nodes]
        hull_polygon = MultiPoint(points).convex_hull
        return hull_polygon

    def align_bbox_to_wall(openings_data_dict, connection_segments_defined):

        aligned_openings = {}

        for name, openings in openings_data_dict.items():
            aligned_openings[name] = []

            for opening in openings:
                center = opening.get("center")
                bbox = opening.get("bbox")
                if not center or not bbox or len(bbox) < 4:
                    continue

                center_point = Point(center)
                bbox_points = [Point(p) for p in bbox]

                main_axis_bbox = LineString([bbox_points[0], bbox_points[2]])

                min_dist = float('inf')
                closest_wall = None

                for conn in connection_segments_defined:
                    wall = conn["segment"]
                    dist = center_point.distance(wall)
                    if dist < min_dist:
                        min_dist = dist
                        closest_wall = wall

                if closest_wall is None:
                    continue

                wall_direction = np.array(closest_wall.coords[1]) - np.array(closest_wall.coords[0])
                wall_direction = wall_direction / np.linalg.norm(wall_direction)

                bbox_center = np.mean(bbox, axis=0)
                aligned_bbox = []

                half_length = 0.5 * (Point(bbox[0]).distance(Point(bbox[1])) - 0.09 * 2)
                half_thickness = 0.075

                perpendicular = np.array([-wall_direction[1], wall_direction[0]])

                corner_offsets = [
                    -half_length * wall_direction - half_thickness * perpendicular,
                    half_length * wall_direction - half_thickness * perpendicular,
                    half_length * wall_direction + half_thickness * perpendicular,
                    -half_length * wall_direction + half_thickness * perpendicular
                ]

                for offset in corner_offsets:
                    aligned_bbox.append((bbox_center[0] + offset[0], bbox_center[1] + offset[1]))

                aligned_openings[name].append({
                    "id": opening.get("id"),
                    "center": center,
                    "aligned_bbox": aligned_bbox
                })

        return aligned_openings
        
    def trace_openings_edges(projected_openings_bbox):
        """Traces the edges of projected openings."""
        traced_edges = []

        for name, openings in projected_openings_bbox.items():
            for opening in openings:
                bbox = opening.get("bbox")
                if bbox and len(bbox) >= 4:  # Ensure there are at least 4 vertices for a polygon
                    line = LineString(bbox)
                    traced_edges.append({
                        "id": opening.get("id"),
                        "line": line
                    })

        return traced_edges

    def find_intersections(project_openings_2d, connection_segments_defined):
        """Finds intersections between projected openings and connection segments."""
        intersections = []

        for opening in project_openings_2d.values():
            for seg_data in connection_segments_defined:
                seg = seg_data["segment"]
                for open_data in opening:
                    line = LineString(open_data["bbox"])
                    if line.intersects(seg):
                        inter = line.intersection(seg)
                        if not inter.is_empty:
                            intersections.append({
                                "opening_id": open_data["id"],
                                "segment": seg,
                                "intersection": inter
                            })

        return intersections
    
    
    from shapely.geometry import Polygon, MultiPoint

    def filter_openings_inside_polygon_with_buffer(openings_bbox_dict, scene_polygon, buffer_distance=0.5, outside_ratio_threshold=0.55):
        filtered_openings = {}

        buffered_polygon = scene_polygon.buffer(buffer_distance)

        for name, openings in openings_bbox_dict.items():
            valid_openings = []
            for opening in openings:
                bbox_points = [Point(pt) for pt in opening.get("bbox", [])]
                if not bbox_points:
                    continue

                outside_count = sum(1 for pt in bbox_points if not buffered_polygon.contains(pt))
                ratio_outside = outside_count / len(bbox_points)

                if ratio_outside <= outside_ratio_threshold:
                    valid_openings.append(opening)
                else:
                    print(f"🚫 Skipping opening {opening['id']} — {int(ratio_outside * 100)}% outside buffered polygon.")

            if valid_openings:
                filtered_openings[name] = valid_openings

        return filtered_openings


    
    projected_openings_bbox = project_openings_2d(openings_data_dict)
    print(f'Number of projected openings: {len(projected_openings_bbox)}')

    from shapely.geometry import LineString, Polygon

    closure_lines = external_segments + bridging_segments
    from shapely.ops import linemerge, polygonize

    merged = linemerge(closure_lines)
    polygons = list(polygonize(merged))
    scene_polygon = polygons[0] if polygons else None


    projected_openings_bbox = filter_openings_inside_polygon_with_buffer(
        projected_openings_bbox, 
        scene_polygon, 
        buffer_distance=0.2, 
        outside_ratio_threshold=0.75
    )

    filtered_dict = {
        name: openings
        for name, openings in projected_openings_bbox.items()
        if len(openings) > 0
    }

    traced_edges = trace_openings_edges(filtered_dict)
    print(f'Number of traced edges: {len(traced_edges)}')
    print(f'Number of projected openings after filtering: {len(filtered_dict)}')

    aligned_openings = align_bbox_to_wall(filtered_dict, connection_segments_defined)

    intersections = find_intersections(filtered_dict, connection_segments_defined)
    print(f'Number of intersections found: {len(intersections)}')

    plot1 =  True

    if plot1:
            
        fig, ax = plt.subplots(figsize=(12, 12))

        for name, openings in aligned_openings.items():
            for opening in openings:
                aligned_bbox = opening.get("aligned_bbox")
                if aligned_bbox:
                    aligned_line = LineString(aligned_bbox + [aligned_bbox[0]])
                    x, y = aligned_line.xy
                    ax.plot(x, y, color='blue')

        for conn in connection_segments_defined:
            seg = conn["segment"]
            x, y = seg.xy
            ax.plot(x, y, color='gray', linestyle='--')

        ax.set_aspect('equal')
        ax.set_title("Aligned Openings to Walls")
        plt.grid(True)
        plt.show()

    print(f'N. of segments: {len(connection_segments_defined)}')


In [ ]:
def_intersections_openings_walls = True

if def_intersections_openings_walls:

    import math
    import json
    from shapely.geometry import LineString, Point, Polygon, MultiPoint, GeometryCollection
    from shapely.affinity import rotate, translate
    from collections import defaultdict
    import matplotlib.pyplot as plt
    from pathlib import Path


    def angle_of_segment(segment):
        x1, y1 = segment.coords[0]
        x2, y2 = segment.coords[-1]
        return math.degrees(math.atan2(y2 - y1, x2 - x1))

    def opening_bbox_angle(bbox):
        if len(bbox) < 2:
            return 0
        x1, y1 = bbox[0]
        x2, y2 = bbox[1]
        return math.degrees(math.atan2(y2 - y1, x2 - x1))

    def is_aligned_angle(angle, tolerance=5):
        angle = angle % 360
        return any(abs(angle - base) <= tolerance for base in [0, 90, 180, 270])


    def project_openings_2d(openings_data_dict):
        projected_openings = {}
        for name, openings in openings_data_dict.items():
            projected_openings[name] = []
            for opening in openings:
                if opening.get("type") in ["door", "window"]:
                    center = opening.get("center")
                    if center and len(center) >= 2:
                        projected_openings[name].append({
                            "id": opening.get("id"),
                            "center": (center[0], center[1]),
                            "bbox": [(v[0], v[1]) for v in opening.get("bbox_vertices")],
                            "height": opening.get("height"),
                            "thickness": opening.get("thickness"),
                            "length": opening.get("length"),
                        })
        return projected_openings


    def rotate_openings_to_segments(projected_openings_bbox, connection_segments_defined, tolerance_deg=5):
        rotated_openings = {}
        for name, openings in projected_openings_bbox.items():
            rotated_openings[name] = []
            for opening in openings:
                opening_center = Point(opening["center"])
                opening_bbox = opening["bbox"]
                if not connection_segments_defined:
                    rotated_openings[name].append(opening)
                    continue
                # Find the closest wall segment
                closest_seg = min(connection_segments_defined, key=lambda s: s["segment"].distance(opening_center))
                seg_line = closest_seg["segment"]
                target_angle = angle_of_segment(seg_line)
                original_angle = opening_bbox_angle(opening_bbox)
                angle_diff = abs((original_angle - target_angle + 180) % 360 - 180)
                bbox_line = LineString(opening_bbox)
                if angle_diff <= tolerance_deg:
                    closest_point = seg_line.interpolate(seg_line.project(opening_center))
                    dx = closest_point.x - opening_center.x
                    dy = closest_point.y - opening_center.y
                    transformed = translate(bbox_line, xoff=dx, yoff=dy)
                else:
                    rotated_bbox = rotate(bbox_line, target_angle - original_angle, origin=opening_center, use_radians=False)
                    new_center = Point(rotated_bbox.centroid)
                    closest_point = seg_line.interpolate(seg_line.project(new_center))
                    dx = closest_point.x - new_center.x
                    dy = closest_point.y - new_center.y
                    transformed = translate(rotated_bbox, xoff=dx, yoff=dy)
                rotated_openings[name].append({
                    **opening,
                    "center": (transformed.centroid.x, transformed.centroid.y),
                    "bbox": list(transformed.coords)
                })

        return rotated_openings


    def filter_openings_by_geometry(openings, min_length=0.55, min_thickness=0.125, min_height=2.0):
        
        filtered = []

        def infer_type(opening_id):
            if opening_id.startswith("door"):
                return "door"
            elif opening_id.startswith("window"):
                return "window"
            return "unknown"
        
        for opening in openings:
            bbox = opening["bbox"]
            if len(bbox) < 2:
                continue
            angle = opening_bbox_angle(bbox)
            if not is_aligned_angle(angle):
                print(f"🚫 Skipping {opening['id']} due to rotation ({angle:.2f}°)")
                continue
            poly = Polygon(bbox)
            minx, miny, maxx, maxy = poly.bounds
            length = maxx - minx
            thickness = maxy - miny

            opening_type = infer_type(opening["id"])
            opening["type"] = opening_type
            
            if opening_type == "door":
                if length < min_length:
                    length = 0.80
                else:
                    length = max(length - 0.15, min_length)
                if thickness < min_thickness:
                    thickness = min_thickness
                if opening.get("height", 0) < min_height:
                    opening["height"] = min_height
            else:  # windows
                if length < min_length or thickness < min_thickness:
                    print(f"🚫 Skipping {opening['id']} due to size — L: {length:.2f}, T: {thickness:.2f}")
                    continue

            area = poly.area
            if area < 0.15:
                print(f"🚫 Skipping {opening['id']} due to small area ({area:.2f})")
                continue

            opening["length"] = length
            opening["thickness"] = thickness
            filtered.append(opening)

        return filtered

    def filter_overlaps(openings):

        accepted = []
        seen_polygons = []
        for opening in openings:
            poly = Polygon(opening["bbox"])
            if not any(poly.intersects(other) for other in seen_polygons):
                accepted.append(opening)
                seen_polygons.append(poly)
            else:
                print(f"❌ Overlapping {opening['id']} skipped")
        return accepted

    def trace_openings_edges(openings_by_scene):

        traced = []
        
        for opening in openings_by_scene["scene"]:
            traced.append({
                "id": opening["id"],
                "line": LineString(opening["bbox"])
            })
        return traced

    def find_intersections(openings_by_scene, connection_segments_defined):

        intersections = []
        for opening in openings_by_scene["scene"]:
            opening_line = LineString(opening["bbox"])  # Convert opening bbox to line
            for seg_data in connection_segments_defined:
                seg = seg_data["segment"]
                if opening_line.intersects(seg):
                    inter = opening_line.intersection(seg)
                    if not inter.is_empty:
                        intersections.append({
                            "opening_id": opening["id"],
                            "segment": seg,
                            "intersection": inter
                        })
        return intersections

    def export_intersections(intersections, all_connection_segments, filtered_openings):
        
        from collections import defaultdict
        intersections_export = []
        summary_by_segment = defaultdict(lambda: {
            "segment_coords": [],
            "number_of_intersections": 0,
            "opening_ids": []
        })

        for intersection in intersections:
            opening_id = intersection.get("opening_id")
            segment = intersection.get("segment")
            intersection_geom = intersection.get("intersection")

            opening_details = next(
                (opening for opening in filtered_openings if opening.get("id") == opening_id),
                {}
            )

            segment_details = next(
                (seg for seg in all_connection_segments if seg.get("segment") and seg["segment"].equals(segment)),
                {}
            )

            segment_id = segment_details.get("wall_id", "unknown")
            segment_coords = list(segment.coords) if segment else []

            summary = summary_by_segment[segment_id]
            summary["segment_coords"] = segment_coords
            summary["number_of_intersections"] += 1
            summary["opening_ids"].append(opening_id)

            export_data = {
                "opening_id": opening_id,
                "opening_type": opening_details.get("type", "unknown"),
                "segment_id": segment_id,
                "segment_coords": segment_coords,
                "intersection": intersection_geom.wkt if intersection_geom else None,
                "intersection_coords": (
                    [(pt.x, pt.y) for pt in intersection_geom.geoms] if isinstance(intersection_geom, (MultiPoint, GeometryCollection))
                    else list(intersection_geom.coords) if isinstance(intersection_geom, LineString)
                    else [(intersection_geom.x, intersection_geom.y)] if isinstance(intersection_geom, Point)
                    else None
                ),
                **{k: v for k, v in opening_details.items() if k != "id"},
                **{k: v for k, v in segment_details.items() if k not in ["segment", "segment_id"]},
            }

            intersections_export.append(export_data)
        
        return summary_by_segment
    
    def align_openings_to_closest_wall(projected_openings_bbox, connection_segments_defined):

        aligned_openings = {}

        for name, openings in projected_openings_bbox.items():
            aligned_openings[name] = []

            for opening in openings:
                center = Point(opening["center"])
                bbox = opening["bbox"]
                if not bbox or len(bbox) < 2:
                    continue

                bbox_line = LineString(bbox)

                closest_seg = min(connection_segments_defined, key=lambda s: s["segment"].distance(center))
                wall = closest_seg["segment"]

                wall_vec = np.array(wall.coords[1]) - np.array(wall.coords[0])
                wall_angle = math.degrees(math.atan2(wall_vec[1], wall_vec[0]))

                bbox_vec = np.array(bbox[1]) - np.array(bbox[0])
                bbox_angle = math.degrees(math.atan2(bbox_vec[1], bbox_vec[0]))

                rotation_needed = wall_angle - bbox_angle

                rotated_bbox = rotate(bbox_line, rotation_needed, origin=center, use_radians=False)

                new_center = rotated_bbox.centroid
                projected_center_on_wall = wall.interpolate(wall.project(new_center))

                dx = projected_center_on_wall.x - new_center.x
                dy = projected_center_on_wall.y - new_center.y

                final_bbox = translate(rotated_bbox, xoff=dx, yoff=dy)

                aligned_openings[name].append({
                    "id": opening.get("id"),
                    "center": (projected_center_on_wall.x, projected_center_on_wall.y),
                    "bbox": list(final_bbox.coords),
                    "height": opening.get("height"),
                    "thickness": opening.get("thickness"),
                    "length": opening.get("length"),
                    "type": opening.get("type", "unknown")
                })

        return aligned_openings
    

In [ ]:
intersections_openings_walls = True

if intersections_openings_walls:

    all_connection_segments = connection_segments_defined.copy()

    rotated_openings_bbox = rotate_openings_to_segments(projected_openings_bbox, connection_segments_defined)
    all_rotated = [o for opening_list in rotated_openings_bbox.values() for o in opening_list]
    filtered = filter_openings_by_geometry(all_rotated)
    filtered = filter_overlaps(filtered)
    filtered_dict = {"scene": filtered}
    traced_edges = trace_openings_edges(filtered_dict)
    intersections_openings_walls = find_intersections(filtered_dict, connection_segments_defined)
    projected_openings_bbox = project_openings_2d(openings_data_dict)



    intersections_by_segment_wkt = defaultdict(list)
    intersections_by_segment_id = defaultdict(list)

    for inter in intersections_openings_walls:
        segment = inter["segment"]
        segment_wkt = segment.wkt
        intersections_by_segment_wkt[segment_wkt].append(inter)

    summary_by_wall = {}
    intersections_export = []

    for seg_data in connection_segments_defined:
        segment = seg_data["segment"]
        segment_id = seg_data.get("wall_id", "unknown")
        segment_coords = list(segment.coords)
        segment_wkt = segment.wkt

        inter_list = intersections_by_segment_wkt.get(segment_wkt, [])

        summary_by_wall[segment_id] = {
            "segment_coords": segment_coords,
            "number_of_intersections": len(inter_list),
            "opening_ids": [inter["opening_id"] for inter in inter_list]
        }

        for inter in inter_list:
            opening_id = inter.get("opening_id")
            opening_details = next((o for o in filtered if o["id"] == opening_id), {})

            intersection_geom = inter.get("intersection")

            if isinstance(intersection_geom, MultiPoint):
                intersection_coords = [(pt.x, pt.y) for pt in intersection_geom.geoms]
            elif isinstance(intersection_geom, GeometryCollection):
                intersection_coords = []
                for geom in intersection_geom.geoms:
                    if isinstance(geom, Point):
                        intersection_coords.append((geom.x, geom.y))
                    elif isinstance(geom, LineString):
                        intersection_coords.extend(list(geom.coords))
            elif isinstance(intersection_geom, LineString):
                intersection_coords = list(intersection_geom.coords)
            elif isinstance(intersection_geom, Point):
                intersection_coords = [(intersection_geom.x, intersection_geom.y)]
            else:
                intersection_coords = None

            export_entry = {
                "opening_id": opening_id,
                "segment_id": segment_id,
                "segment_coords": segment_coords,
                "intersection_coords": intersection_coords,
                **{k: v for k, v in opening_details.items() if k != "id"},
                **{k: v for k, v in seg_data.items() if k not in ["segment", "segment_id"]},
            }

            intersections_export.append(export_entry)
            intersections_by_segment_id[segment_id].append(export_entry)

    print(f"📌 Summary by wall:\n{json.dumps(summary_by_wall, indent=2)}")
    # print(f"📌 Example intersection entry:\n{json.dumps(intersections_export[0], indent=2, default=str)}")
    print(f'number of segments {len(connection_segments_defined)}')

    plot_intersections_openings = True

    if plot_intersections_openings:
            
        fig, ax = plt.subplots(figsize=(12, 12))

        for idx, edge in enumerate(traced_edges):
            x, y = edge['line'].xy
            ax.plot(x, y, color='purple', linewidth=1, linestyle='-', alpha=0.7, label="Opening Edge" if idx == 0 else "")

        for idx, seg_data in enumerate(all_connection_segments):
            seg = seg_data["segment"]
            x, y = seg.xy
            ax.plot(x, y, color='gray', linewidth=1, linestyle=':', alpha=0.8, label="All Segments" if idx == 0 else "")

        for i, intersection in enumerate(intersections):
            inter = intersection['intersection']
            if isinstance(inter, Point):
                ax.plot(inter.x, inter.y, 'go', markersize=5, label="Intersection" if i == 0 else "")
            elif isinstance(inter, MultiPoint):
                for j, pt in enumerate(inter.geoms):
                    ax.plot(pt.x, pt.y, 'go', markersize=5, label="Intersection" if i == 0 and j == 0 else "")
            elif isinstance(inter, LineString):
                x, y = inter.xy
                ax.plot(x, y, 'g-', linewidth=2, alpha=0.6, label="Intersection" if i == 0 else "")
            elif isinstance(inter, GeometryCollection):
                for j, geom in enumerate(inter.geoms):
                    if isinstance(geom, Point):
                        ax.plot(geom.x, geom.y, 'go', markersize=5, label="Intersection" if i == 0 and j == 0 else "")
                    elif isinstance(geom, LineString):
                        x, y = geom.xy
                        ax.plot(x, y, 'g-', linewidth=2, alpha=0.6, label="Intersection" if i == 0 and j == 0 else "")


        ax.set_aspect('equal')
        ax.set_title("📍 Cleaned Intersections Between Openings and Connection Segments")
        ax.grid(True)


        handles, labels = ax.get_legend_handles_labels()
        by_label = dict(zip(labels, handles))
        ax.legend(by_label.values(), by_label.keys())

        plt.show()


In [ ]:
summary_by_wall

In [ ]:
intersection_number_comp = True

if intersection_number_comp:

    openings_intersections = []

    for opening in intersections_export:
        intersection_coords = opening.get('intersection_coords')
        openings_intersections.append(intersection_coords)

    print(f'intersections {openings_intersections}\n number {len(openings_intersections)}')


TOPOLOGIC _MAP

In [ ]:
topologic_step_01 = True

if topologic_step_01:

        segments = []

        for seg_data in connection_segments_defined:
                wall_id = seg_data['wall_id']
                segment = seg_data['segment']
                segment_coords = list(segment.coords)
                segments.append(segment_coords)

        print(f'Segment {segments}')

        segs = []

        for seg_data in connection_segments_defined:
                        
                segment = seg_data['segment']
                segs.append(segments)

                print(segs)
                print(len(segs))


TEST 02


In [ ]:
topologic_map_computation_complete = True

if topologic_map_computation_complete:

    wall_lookup = {wall['wall_id']: wall for wall in walls_data_dict['first_floor']}
    openings_lookup = {opening['id']: opening for opening in openings_data_dict['first_floor']}

    topologic_map = {}
    z_coordinates_walls = []
    z_coordinates_openings = []

    from shapely.geometry import Point

    for seg_data in connection_segments_defined:

        segment = seg_data['segment']

        if seg_data in closure_segments_list:
            wall_id = "enclosure"  
        else:
            wall_id = seg_data['wall_id']

        wall_data = wall_lookup.get(wall_id, {}) if isinstance(wall_id, int) else {}

        wall_center = wall_data.get('center')
        wall_height = wall_data.get('height')
        wall_length = wall_data.get('length')
        wall_thickness = wall_data.get('thickness')
        surface_vertices = wall_data.get('surface_vertices')

        z_min = None
        z_max = None

        if surface_vertices:
            z_coords_list = [vertex[2] for vertex in surface_vertices if len(vertex) >= 3]
            if z_coords_list:
                z_min = np.min(z_coords_list)
                z_max = np.max(z_coords_list)
                z_coordinates_walls.extend(z_coords_list)

        if wall_id not in topologic_map:

            topologic_map[wall_id] = {
                'wall_id': wall_id,
                'segment_coords_list': [],
                'wall_center': wall_center,
                'wall_height': wall_height,
                'wall_length': wall_length,
                'wall_thickness': wall_thickness,
                'surface_vertices': surface_vertices,
                'z_min': z_min,
                'z_max': z_max,
                'openings': []
            }

        topologic_map[wall_id]['segment_coords_list'].append(list(segment.coords))

    wall_midpoints = {}

    for real_wall_id, wall_data in topologic_map.items():

        if real_wall_id != "enclosure" and wall_data['segment_coords_list']:

            segment = wall_data['segment_coords_list'][0]
            mid_x = (segment[0][0] + segment[1][0]) / 2
            mid_y = (segment[0][1] + segment[1][1]) / 2
            wall_midpoints[real_wall_id] = Point(mid_x, mid_y)

    for wall_id, wall_data in topologic_map.items():

        if wall_data['z_min'] is None or wall_data['z_max'] is None:

            if wall_data['segment_coords_list']:
            
                segment = wall_data['segment_coords_list'][0]
                mid_x = (segment[0][0] + segment[1][0]) / 2
                mid_y = (segment[0][1] + segment[1][1]) / 2
                midpoint = Point(mid_x, mid_y)

                nearest_id = None
                min_dist = float('inf')

                for real_id, real_mid in wall_midpoints.items():
                    dist = midpoint.distance(real_mid)
                    if dist < min_dist:
                        min_dist = dist
                        nearest_id = real_id

                if nearest_id is not None:
                    wall_data['z_min'] = topologic_map[nearest_id]['z_min']
                    wall_data['z_max'] = topologic_map[nearest_id]['z_max']
                    print(f"🛠️ Wall {wall_id} z-range assigned from Wall {nearest_id}.")

    print("\n🧱 Topologic Map — Wall Summary:")
    print("-" * 50)

    for wall_id, wall_data in topologic_map.items():

        segment_count = len(wall_data.get('segment_coords_list', []))
        z_min = wall_data.get('z_min')
        z_max = wall_data.get('z_max')
        if z_min is not None and z_max is not None:
            print(f"Wall ID {wall_id}: {segment_count} segments, z_min={z_min:.3f} z_max={z_max:.3f}")
        else:
            print(f"Wall ID {wall_id}: {segment_count} segments, no z-range available")

    openings_collected = []  

    already_processed_openings = set()

    for opening in intersections_export:
        opening_id = opening['opening_id']

        if opening_id in already_processed_openings:
            continue
        already_processed_openings.add(opening_id)

        wall_id = opening['segment_id']
        opening_type = opening.get('type')
        opening_center = opening.get('center')
        opening_height = opening.get('height')
        opening_length = opening.get('length')
        opening_thickness = opening.get('thickness')
        intersection_coords = opening.get('intersection_coords')

        opening_info = openings_lookup.get(opening_id)

        if not opening_info:
            print(f"⚠️ Opening {opening_id} not found in openings_lookup.")
            continue

        opening_bbox = opening_info.get('bbox_vertices')

        if not opening_bbox:
            print(f"⚠️ No bbox for opening {opening_id}")
            continue

        # z_coords = [point[2] for point in opening_bbox if isinstance(point, (list, tuple)) and len(point) >= 3]

        # if not z_coords:
        #     print(f"⚠️ No valid Z-coordinates for opening {opening_id}")
        #     continue

        # z_min = np.min(z_coords)

        # if opening_height and opening_height > 0:
        #     z_max = z_min + opening_height
        # else:
        #     z_max = np.max(z_coords)

        # # Optional: check for suspiciously flat openings
        # if abs(z_max - z_min) < 0.01:
        #     print(f"⚠️ Very thin Z-range for opening {opening_id}: {z_min:.3f}–{z_max:.3f}")

        # Use clean Z from center[2] and height from openings_data_dict
        center = opening_info.get("center")
        height = opening_info.get("height")
        length = opening_info.get("length")
        thickness = opening_info.get("thickness")

        if not center or height is None:
            print(f"⚠️ Missing center or height for opening {opening_id}")
            continue

        z_min = center[2]
        z_max = z_min + height

        if abs(z_max - z_min) < 0.01:
            print(f"⚠️ Very thin Z-range for opening {opening_id}: {z_min:.3f}–{z_max:.3f}")

        if not intersection_coords or len(intersection_coords) < 2:
            print(f"⚠️ Intersection coords invalid for opening {opening_id}")
            continue

        opening_data = {
            'opening_id': opening_id,
            'type': opening_type,
            'center': center,                     # <- use reliable version
            'height': height,
            'length': length,
            'thickness': thickness,
            'bbox': opening_bbox,
            'z_min': z_min,
            'z_max': z_max,
            'intersection_coords': intersection_coords
        }


        openings_collected.append(opening_data)

        if wall_id in topologic_map:
            topologic_map[wall_id]['openings'].append(opening_data)
        else:
            print(f"⚠️ Wall ID {wall_id} not found in topologic_map.")


In [ ]:
topologic_map

FROM_2D TO 3D

In [ ]:
from_2d_to_3d = True

if from_2d_to_3d:

    from shapely.geometry import LineString, Point
    from scipy.spatial import KDTree
    import numpy as np

    first_check = True

    if first_check:
            
        good_walls = []
        good_coords = []

        for wall_key, wall_info in topologic_map.items():

            if wall_info is None or 'segment_coords_list' not in wall_info:
                continue
            
            wall_id = wall_info.get('wall_id')
            z_min = wall_info.get('z_min')
            wall_height = wall_info.get('wall_height')

            if isinstance(wall_id, int) and z_min is not None and wall_height is not None:

                segment = wall_info['segment_coords_list'][0]
                (x1, y1), (x2, y2) = segment
                cx, cy = (x1 + x2)/2, (y1 + y2)/2
                good_coords.append([cx, cy])
                good_walls.append({
                    'wall_id': wall_id,
                    'z_min': z_min,
                    'wall_height': wall_height
                })

    if good_coords:

        kdtree = KDTree(good_coords)

    for wall_key, wall_info in topologic_map.items():
        
        if wall_info is None or 'segment_coords_list' not in wall_info:
            continue

        wall_id = wall_info.get('wall_id')
        wall_segments = wall_info.get('segment_coords_list', [])

        z_min = wall_info.get('z_min')
        wall_height = wall_info.get('wall_height')
        z_max = wall_info.get('z_max')

        if z_min is None or wall_height is None:
            if wall_segments:
                segment = wall_segments[0]
                (x1, y1), (x2, y2) = segment
                cx, cy = (x1 + x2)/2, (y1 + y2)/2

                if good_coords:
                    dist, idx = kdtree.query([cx, cy])

                    if dist < 6.0:  
                        nearest = good_walls[idx]
                        wall_info['z_min'] = nearest['z_min']
                        wall_info['wall_height'] = nearest['wall_height']
                        wall_info['z_max'] = wall_info['z_min'] + wall_info['wall_height']
                        print(f"✅ Wall {wall_id} fixed with nearest wall {nearest['wall_id']} (dist={dist:.2f}m)")

                    else:
                        wall_info['z_min'] = 0.0
                        wall_info['wall_height'] = 3.0
                        wall_info['z_max'] = 3.0
                        print(f"⚠️ Wall {wall_id} too far, default height used")
                        
                else:
                    wall_info['z_min'] = 0.0
                    wall_info['wall_height'] = 3.0
                    wall_info['z_max'] = 3.0


        if wall_info['z_min'] > wall_info['z_max']:
            wall_info['z_min'], wall_info['z_max'] = wall_info['z_max'], wall_info['z_min']
            print(f"🔄 Swapped z_min and z_max for wall {wall_id}")

        if wall_info['wall_height'] > 6.0:
            print(f"⚠️ Wall {wall_id} too high ({wall_info['wall_height']}m), capping to 3.0m")
            wall_info['wall_height'] = 3.0
            wall_info['z_max'] = wall_info['z_min'] + 3.0

    all_segments = []

    for wall_key, wall_info in topologic_map.items():
        if wall_info is None or 'segment_coords_list' not in wall_info:
            continue

        wall_id = wall_info.get('wall_id')
        wall_segments = wall_info['segment_coords_list']

        wall_z_min = wall_info.get('z_min')
        wall_height = wall_info.get('wall_height')

        if wall_z_min is None:
            wall_z_min = 0.0
        if wall_height is None:
            wall_height = (wall_z_max - wall_z_min)

        wall_z_max = wall_info.get('z_max')
        if wall_z_max is None:
            wall_z_max = wall_z_min + wall_height

        for segment_coords in wall_segments:
            segment = LineString(segment_coords)

            all_segments.append({
                'wall_id': wall_id,
                'wall_key': wall_key,
                'segment': segment,
                'wall_z_min': wall_z_min,
                'wall_z_max': wall_z_max,
                'wall_height': wall_height,
                'openings': []
            })

        for segment_coords in wall_segments:
            segment = LineString(segment_coords)

            all_segments.append({
                'wall_id': wall_id,
                'wall_key': wall_key,
                'segment': segment,
                'wall_z_min': wall_z_min,
                'wall_z_max': wall_z_max,
                'wall_height': wall_height,
                'openings': []  # To be filled below
            })

    for wall_key, wall_info in topologic_map.items():

        if wall_info is None:
            continue

        for opening in wall_info.get('openings', []):
            intersection_coords = opening.get('intersection_coords', [])
            if not intersection_coords or len(intersection_coords) < 2:
                continue

            opening_segment = LineString(intersection_coords)

            for seg in all_segments:
                if seg['wall_key'] != wall_key:
                    continue

                wall_segment = seg['segment']

                if wall_segment.distance(opening_segment) < 0.05:  # 5cm tolerance
                    seg['openings'].append({
                        'opening_id': opening.get('opening_id'),
                        'type': opening.get('type', 'unknown'),
                        'z_min': opening.get('z_min', seg['wall_z_min']),
                        'z_max': opening.get('z_max', seg['wall_z_max']),
                        'intersection_coords': intersection_coords
                    })
                    break  
    print(f"✅ Total segments prepared: {len(all_segments)}")

    from shapely.geometry import Polygon
    from shapely.geometry import Polygon

    opening_wkts = []
    wall_wkts = []

    for idx, seg_data in enumerate(all_segments):

        segment = seg_data['segment']
        wall_id = seg_data['wall_id']
        wall_key = seg_data['wall_key']
        wall_z_min = seg_data['wall_z_min']
        wall_z_max = seg_data['wall_z_max']
        openings = seg_data['openings']

        (x0, y0), (x1, y1) = segment.coords

        # Create wall polygon (vertical rectangle)
        polygon_wall = Polygon([
            (x0, y0, wall_z_min),
            (x1, y1, wall_z_min),
            (x1, y1, wall_z_max),
            (x0, y0, wall_z_max),
            (x0, y0, wall_z_min)
        ])

        wall_wkts.append({
            'wall_id': wall_id,
            'wall_key': wall_key,
            'geometry': polygon_wall.wkt
        })
        
        for opening in openings:
            intersection_coords = opening.get('intersection_coords', [])
            if not intersection_coords or len(intersection_coords) != 2:
                continue

            (ox1, oy1), (ox2, oy2) = intersection_coords
            z_min_opening = opening.get('z_min', wall_z_min)
            z_max_opening = opening.get('z_max', wall_z_min + 2.0)  # default opening height 2m if missing

            if z_min_opening > z_max_opening:
                z_min_opening, z_max_opening = z_max_opening, z_min_opening

            polygon_opening = Polygon([
                (ox1, oy1, z_min_opening),
                (ox2, oy2, z_min_opening),
                (ox2, oy2, z_max_opening),
                (ox1, oy1, z_max_opening),
                (ox1, oy1, z_min_opening)
            ])

            
            opening_wkts.append({
                'wall_id': wall_id,
                'opening_id': opening.get('opening_id', 'unknown'),
                'type': opening.get('type', 'unknown'),
                'geometry': polygon_opening.wkt
            })

    print(f"\n📄 Generated {len(wall_wkts)} wall polygons and {len(opening_wkts)} opening polygons!")

    if wall_wkts:
        print("\nWall polygon:")
        print(wall_wkts)
        print(f"\nNum of walls: {len(wall_wkts)}")

    if opening_wkts:
        print("\nOpening polygon:")
        print(opening_wkts)


In [ ]:
opening_wkts

DCEL TO POLYGON ROOMS

In [ ]:
dcel_for_rooms = True

if dcel_for_rooms:

    import networkx as nx
    import matplotlib.pyplot as plt
    from shapely.geometry import LineString, Polygon, MultiPolygon
    from shapely.ops import polygonize_full, unary_union

    # Parameters
    l1 = 0.9
    l2 = 0.9
    min_area = l1 * l2

    def build_wall_intersections_graph(segments):
        G = nx.Graph()
        for seg_data in segments:
            seg = seg_data.get("segment")
            if seg and isinstance(seg, LineString):
                coords = list(seg.coords)
                if len(coords) == 2:
                    p1, p2 = tuple(coords[0]), tuple(coords[1])
                    G.add_edge(p1, p2, weight=seg.length)
        return G

    def find_rooms_from_graph(G):
        print("🔍 Extracting rooms via polygonization...")
        edge_lines = [LineString([u, v]) for u, v in G.edges if u != v]
        polygons_gc, dangles, cuts, invalids = polygonize_full(edge_lines)
        valid_rooms = []
        for idx, poly in enumerate(polygons_gc.geoms):
            if poly.is_valid and poly.area > 0.01:
                valid_rooms.append(poly)
            else:
                print(f"⚠️ Skipping invalid or small polygon {idx}")
        print(f"✅ Detected {len(valid_rooms)} valid room polygons.")
        return valid_rooms

    def merge_small_rooms(rooms, min_area):
        big_rooms = []
        small_rooms = []
        for room in rooms:
            if room.area < min_area:
                print(f"🟡 Small room found (area={room.area:.4f})")
                small_rooms.append(room)
            else:
                big_rooms.append(room)

        print(f"✅ Found {len(small_rooms)} small rooms to merge.")

        for small in small_rooms:
            if not big_rooms:
                print("⚠️ No big rooms available to merge with.")
                continue

            # Prefer touching big rooms
            touching_idxs = [i for i, big in enumerate(big_rooms) if small.touches(big)]
            if touching_idxs:
                nearest_idx = touching_idxs[0]
            else:
                # fallback to nearest centroid
                distances = [small.centroid.distance(big.centroid) for big in big_rooms]
                nearest_idx = distances.index(min(distances))

            nearest_big = big_rooms[nearest_idx]
            merged = nearest_big.union(small)

            if merged.is_empty:
                print("⚠️ Merged room is empty, skipping.")
                continue
            if merged.type == 'GeometryCollection':
                merged = unary_union([geom for geom in merged.geoms if isinstance(geom, (Polygon, MultiPolygon))])
            if not merged.is_valid:
                print("⚠️ Merged room is invalid, skipping.")
                continue

            big_rooms[nearest_idx] = merged
            print(f"✅ Merged small room into room {nearest_idx}")

        return big_rooms
    
    def check_dcel_validity(vertices, half_edges, snap_precision=9):
        """
        For every half-edge 'e' in half_edges:
        1) Verify 'twin' exists, and that twin.twin == e
        2) Let e_origin = vertices[e['origin']];   e_dest = vertices[ twin['origin'] ]
            Let t_origin = e_dest;   t_dest = e_origin
        3) Check (e_origin == t_dest) and (e_dest == t_origin)
        4) Check that LineString(e_origin→e_dest) equals LineString(t_origin→t_dest) using .equals()
        5) Check that e.incident_face != twin.incident_face (if neither is None)
        """
        errors = []

        def _snap(pt):
            return tuple(round(c, snap_precision) for c in pt)

        for eid, edge in half_edges.items():
            twin_id = edge.get("twin")
            if twin_id is None:
                errors.append(f"Half-edge {eid} has no twin.")
                continue
            if twin_id not in half_edges:
                errors.append(f"Half-edge {eid} has missing twin {twin_id}.")
                continue

            twin = half_edges[twin_id]
            # 1) Mutual‐twin check
            if twin.get("twin") != eid:
                errors.append(f"Half-edge {eid} and twin {twin_id} are not mutually linked.")

            # 2) Fetch coordinates (with snapping)
            try:
                e_origin_coords = _snap(vertices[edge["origin"]]["coordinates"])
            except KeyError:
                errors.append(f"Half-edge {eid} refers to unknown origin {edge['origin']}.")
                continue

            try:
                e_dest_coords = _snap(vertices[twin["origin"]]["coordinates"])
            except KeyError:
                errors.append(f"Half-edge {eid} has twin {twin_id} with unknown origin {twin['origin']}.")
                continue

            # 3) Derive twin’s endpoints
            t_origin_coords = e_dest_coords
            t_dest_coords = e_origin_coords

            # 4) Check origin/destination consistency
            if e_origin_coords != t_dest_coords:
                errors.append(
                    f"Half-edge {eid}: origin {e_origin_coords} != twin {twin_id} destination {t_dest_coords}."
                )
            if e_dest_coords != t_origin_coords:
                errors.append(
                    f"Half-edge {eid}: destination {e_dest_coords} != twin {twin_id} origin {t_origin_coords}."
                )

            # 5) Geometry check (use .equals, not equals_exact, because the twin is reversed)
            try:
                seg_e = LineString([e_origin_coords, e_dest_coords])
                seg_t = LineString([t_origin_coords, t_dest_coords])
                if not seg_e.equals(seg_t):
                    errors.append(f"Half-edge {eid} and twin {twin_id} geometry mismatch.")
            except Exception as ex:
                errors.append(f"Error comparing geometry for edge {eid} vs twin {twin_id}: {ex}")

            # 6) Incident-face check
            f_e = edge.get("incident_face")
            f_t = twin.get("incident_face")
            if f_e is not None and f_e == f_t:
                errors.append(f"Half-edge {eid} and twin {twin_id} belong to same face {f_e}.")

        if errors:
            print(f"❌ DCEL check failed: {len(errors)} issue(s) found")
            for err in errors[:100]:
                print(" -", err)
            if len(errors) > 100:
                print(f"...and {len(errors) - 100} more.")
        else:
            print("✅ DCEL passed all validity checks.")


    def extract_wall_segments_from_rooms(rooms):
        new_segments = []
        for room_idx, room in enumerate(rooms):
            polys = list(room.geoms) if isinstance(room, MultiPolygon) else [room]
            for poly in polys:
                exterior = list(poly.exterior.coords)
                for i in range(len(exterior) - 1):
                    p1, p2 = exterior[i], exterior[i + 1]
                    line = LineString([p1, p2])
                    new_segments.append({"segment": line, "room_idx": room_idx})
        print(f"✅ Extracted {len(new_segments)} wall segments from merged rooms.")
        return new_segments

    def is_polygon_closed(poly):
        return poly.exterior.is_ring if hasattr(poly, "exterior") else False

    def plot_merged_rooms_2d(rooms, title="🧩 Merged Room Layout (2D)"):
        fig, ax = plt.subplots(figsize=(10, 10))
        for idx, room in enumerate(rooms):
            geometries = list(room.geoms) if isinstance(room, MultiPolygon) else [room]
            for poly in geometries:
                if not poly.is_valid or not is_polygon_closed(poly):
                    print(f"⚠️ Room {idx} is not closed or invalid.")
                    continue
                x, y = poly.exterior.xy
                ax.fill(x, y, alpha=0.5, edgecolor="black", linewidth=1.5)
                cx, cy = poly.centroid.x, poly.centroid.y
                ax.text(cx, cy, str(idx), fontsize=10, ha='center', va='center', color='black')
        for i in range(len(rooms)):
            for j in range(i + 1, len(rooms)):
                ri = rooms[i]
                rj = rooms[j]
                for pi in (ri.geoms if isinstance(ri, MultiPolygon) else [ri]):
                    for pj in (rj.geoms if isinstance(rj, MultiPolygon) else [rj]):
                        if pi.intersects(pj):
                            inter = pi.intersection(pj)
                            if inter.area > 0.01:
                                print(f"⚠️ Room {i} overlaps with Room {j} by area {inter.area:.4f}")
        ax.set_title(title, fontsize=14)
        ax.axis("equal")
        ax.grid(True)
        plt.show()

    print("📋 Running DCEL validity check...")
    check_dcel_validity(vertices, half_edges)  # <- insert your actual vertices & half_edges dicts

    G_initial = build_wall_intersections_graph(connection_segments_defined)
    rooms = find_rooms_from_graph(G_initial)
    merged_rooms = merge_small_rooms(rooms, min_area)
    new_wall_segments = extract_wall_segments_from_rooms(merged_rooms)
    plot_merged_rooms_2d(merged_rooms)



In [ ]:
wall_coords_rooms_plot = True

if wall_coords_rooms_plot:

    wall_coords = [list(seg["segment"].coords) for seg in new_wall_segments]

    from collections import defaultdict
    walls_by_room = defaultdict(list)
    for seg in new_wall_segments:
        walls_by_room[seg["room_idx"]].append(seg["segment"])
        
    import matplotlib.pyplot as plt

    def plot_walls(wall_segments):
        fig, ax = plt.subplots(figsize=(10, 10))
        for seg in wall_segments:
            x, y = seg["segment"].xy
            ax.plot(x, y, color="black")
        ax.set_aspect("equal")
        plt.title("🧱 Extracted Wall Segments")
        plt.grid(True)
        plt.show()

    plot_walls(new_wall_segments)

    import matplotlib.pyplot as plt
    import random

    def plot_walls_colored_by_room(wall_segments, title="🧱 Wall Segments by Room"):
        fig, ax = plt.subplots(figsize=(12, 12))
        room_colors = {}

        # Generate a unique color for each room index
        unique_rooms = sorted(set(seg["room_idx"] for seg in wall_segments))
        for room_idx in unique_rooms:
            room_colors[room_idx] = (
                random.random(), random.random(), random.random()
            )

        for seg in wall_segments:
            line = seg["segment"]
            room_idx = seg["room_idx"]
            x, y = line.xy
            ax.plot(x, y, color=room_colors[room_idx], label=f"Room {room_idx}")

        # Optional: Remove duplicate labels in legend
        handles, labels = ax.get_legend_handles_labels()
        unique = dict(zip(labels, handles))
        ax.legend(unique.values(), unique.keys(), title="Room Index", bbox_to_anchor=(1.05, 1), loc='upper left')

        ax.set_title(title, fontsize=14)
        ax.set_aspect("equal")
        ax.grid(True)
        plt.tight_layout()
        plt.show()

    # Call the function
    plot_walls_colored_by_room(new_wall_segments)


ROOMS 3D

In [ ]:
room_3d_watertigth = True

if room_3d_watertigth:

    z_values_computation = True

    if z_values_computation:

        def extract_z_values(data):
            z_values = []

            def extract_from_value(value):
                if isinstance(value, list):
                    if all(isinstance(item, (int, float)) for item in value) and len(value) == 3:
                        z_values.append(value[2])
                    else:
                        for subvalue in value:
                            extract_from_value(subvalue)
                elif isinstance(value, dict):
                    for subkey in value:
                        extract_from_value(value[subkey])

            for key in data:
                extract_from_value(data[key])

            return z_values

        z_values = extract_z_values(walls_data_dict)
        print(z_values)

        z_coords = []

        for wall in walls_data_dict[name]:
            for vertex in wall.get('surface_vertices', []):
                z_coords.append(vertex[2])

        print(z_coords)

    def is_clockwise_xy(pts):

        pts_2d = [(x, y) for x, y, _ in pts]
        area = 0.0
        for i in range(len(pts_2d)):
            x1, y1 = pts_2d[i]
            x2, y2 = pts_2d[(i + 1) % len(pts_2d)]
            area += (x2 - x1) * (y2 + y1)
        return area < 0

    # def generate_3d_segments(base_segments, z_value):
    #     result = []
    #     for seg_data in base_segments:
    #         line = seg_data["segment"]
    #         room_idx = seg_data["room_idx"]
    #         wall_id = seg_data.get("wall_id")
    #         coords_3d = [(x, y, z_value) for (x, y, *_) in line.coords]

    #         new_line_3d = LineString(coords_3d)
    #         result.append({
    #             "segment": new_line_3d,
    #             "room_idx": room_idx,
    #             "wall_id": wall_id,
    #             "z_coord": z_value
    #         })
    #     return result

    def extrude_room_to_3d_walls(rooms, z_min, z_max):
        wall_faces = []
        wall_id = 0
        for room_id, room in enumerate(rooms):
            exterior_coords = list(room.exterior.coords)
            for i in range(len(exterior_coords) - 1):
                x1, y1 = exterior_coords[i]
                x2, y2 = exterior_coords[i + 1]
                quad = [
                    (x1, y1, z_min),
                    (x2, y2, z_min),
                    (x2, y2, z_max),
                    (x1, y1, z_max)
                ]
                if quad[0] != quad[-1]:
                    quad.append(quad[0])
                wall_faces.append({
                    "wall_id": wall_id,
                    "room_id": room_id,
                    "face": quad
                })
                wall_id += 1
        return wall_faces

    # new_wall_segments = extract_wall_segments_from_rooms(merged_rooms)

    # new_segments_base = generate_3d_segments(new_wall_segments, z_min)
    # new_segments_top = generate_3d_segments(new_wall_segments, z_max)
    # horizontals = new_segments_base + new_segments_top

    # wall_faces = extrude_room_to_3d_walls(rooms, z_min, z_max)
    # wall_dict = {wall["wall_id"]: wall for wall in wall_faces}

    # # Optional: print info
    # for wall in wall_faces:
    #     print(f"wall_id={wall['wall_id']} | room_id={wall['room_id']} | face={wall['face']}")

    new_wall_segments = extract_wall_segments_from_rooms(merged_rooms)

    def generate_3d_segments(base_segments, z_value):
        result = []
        for seg_data in base_segments:
            line = seg_data["segment"]
            room_idx = seg_data["room_idx"]
            coords_3d = [(x, y, z_value) for (x, y) in line.coords]

            new_line_3d = LineString(coords_3d)
            result.append({
                "segment": new_line_3d,
                "room_idx": room_idx,
                "wall_id": None,   # optional: assign later
                "z_coord": z_value
            })
        return result

    new_segments_base = generate_3d_segments(new_wall_segments, z_min)
    new_segments_top = generate_3d_segments(new_wall_segments, z_max)
    horizontals = new_segments_base + new_segments_top


In [ ]:
import matplotlib.cm as cm

def create_wall_faces_from_segments(segments_2d, z_min, z_max):
    wall_faces = []
    for wall_id, seg_data in enumerate(segments_2d):
        line = seg_data["segment"]
        room_id = seg_data["room_idx"]
        coords = list(line.coords)

        if len(coords) != 2:
            continue  

        (x1, y1), (x2, y2) = coords

        quad = [
            (x1, y1, z_min),
            (x2, y2, z_min),
            (x2, y2, z_max),
            (x1, y1, z_max),
            (x1, y1, z_min)  
        ]

        wall_faces.append({
            "wall_id": wall_id,
            "room_id": room_id,
            "face": quad
        })

    return wall_faces

wall_faces = create_wall_faces_from_segments(new_wall_segments, z_min, z_max)
wall_dict = {face["wall_id"]: face for face in wall_faces}

def plot_wall_faces_2d(wall_faces, title="3D Wall Faces (2D Projection with Wall IDs)"):
    fig, ax = plt.subplots(figsize=(12, 12))
    
    colors = cm.rainbow(np.linspace(0, 1, len(wall_faces)))
    
    plotted_ids = set()

    for wall, color in zip(wall_faces, colors):
        face = wall["face"]
        wall_id = wall["wall_id"]
        
        x = [pt[0] for pt in face[:2]]
        y = [pt[1] for pt in face[:2]]
        ax.plot(x, y, color=color)

        if wall_id not in plotted_ids:
            mid_x = np.mean(x)
            mid_y = np.mean(y)
            ax.text(mid_x, mid_y, str(wall_id), fontsize=6, color="blue")
            plotted_ids.add(wall_id)

    ax.set_aspect("equal")
    ax.set_title(title)
    ax.grid(True)
    plt.show()

wall_faces = []

for base_seg, top_seg in zip(new_segments_base, new_segments_top):
    base_coords = list(base_seg["segment"].coords)
    top_coords = list(top_seg["segment"].coords)

    quad = [
        base_coords[0],
        base_coords[1],
        top_coords[1],
        top_coords[0]
    ]

    wall_faces.append({
        "wall_id": len(wall_faces),
        "room_id": base_seg["room_idx"],
        "face": quad
    })

plot_wall_faces_2d(wall_faces)


In [ ]:
for wall in wall_dict.values():

    wall.setdefault('openings', [])

wall_id_to_wall = {wall['wall_id']: wall for wall in wall_dict.values()}

for opening in opening_wkts:

    wall_id = opening.get('wall_id')
    
    if wall_id in wall_id_to_wall:
        wall_id_to_wall[wall_id]['openings'].append({
            'opening_id': opening.get('opening_id'),
            'type': opening.get('type'),
            'geometry': opening.get('geometry')
        })


In [ ]:
concrete_match_openings_walls = True

if concrete_match_openings_walls:

    from shapely import wkt

    def extract_opening_centroid_and_normal(geometry_wkt):

        polygon = wkt.loads(geometry_wkt)
        coords = np.array(polygon.exterior.coords)
        centroid = np.mean(coords[:, :2], axis=0)

        if len(coords) >= 2:

            dx, dy = coords[1][:2] - coords[0][:2]
            normal = np.array([-dy, dx])
            norm = np.linalg.norm(normal)
            if norm != 0:
                normal = normal / norm
            else:
                normal = np.array([0.0, 0.0])
        else:
            normal = np.array([0.0, 0.0])

        return centroid, normal

    def get_wall_line(face):
        p1 = np.array(face[0][:2])
        p2 = np.array(face[1][:2])
        return LineString([p1, p2])


    def z_overlap(opening_geom, wall_face, margin=0.2):
        polygon = wkt.loads(opening_geom)
        z_opening = np.array(polygon.exterior.coords)[:, 2]
        z_min_opening = z_opening.min()
        z_max_opening = z_opening.max()

        wall_zs = np.array([pt[2] for pt in wall_face])
        z_min_wall = wall_zs.min()
        z_max_wall = wall_zs.max()

        return not (z_max_opening < z_min_wall - margin or z_min_opening > z_max_wall + margin)


    def opening_to_wall_distance(opening_wkt, wall_line):
        opening_polygon = wkt.loads(opening_wkt)
        return wall_line.distance(opening_polygon)


    def assign_openings_to_walls_precise(walls_dict, threshold=0.5):

        wall_lines = {wid: get_wall_line(w["face"]) for wid, w in walls_dict.items()}
        result = []
        
        opening_to_best_match = {}

        for wall_id, wall in walls_dict.items():
            for opening in wall.get("openings", []):
                opening_id = opening.get("opening_id")
                if opening_id in opening_to_best_match:
                    continue  # Already matched in previous loop (avoid duplicates)

                opening_geom = opening.get("geometry")
                opening_type = opening.get("type")

                centroid, normal = extract_opening_centroid_and_normal(opening_geom)

                candidates = [
                    (wid, line)
                    for wid, line in wall_lines.items()
                    if z_overlap(opening_geom, walls_dict[wid]["face"])
                ]

                best_match = None
                min_dist = float("inf")

                for wid, wall_line in candidates:
                    dist = opening_to_wall_distance(opening_geom, wall_line)
                    if dist < threshold and dist < min_dist:
                        best_match = wid
                        min_dist = dist

                result.append({
                    "Opening ID": opening_id,
                    "Opening Type": opening_type,
                    "Matched Wall ID": best_match,
                    "Distance": round(min_dist, 4) if best_match else None
                })

                if best_match is not None:
                    opening_to_best_match[opening_id] = True  # Mark as matched
                    walls_dict[best_match].setdefault("openings_precise", []).append({
                        "opening_id": opening_id,
                        "type": opening_type,
                        "geometry": opening_geom,
                        "distance": min_dist
                    })

        return pd.DataFrame(result)

    df_precise_matches = assign_openings_to_walls_precise(wall_id_to_wall)
    df_precise_matches.head()

    print(df_precise_matches.head(80))

    for wall_id, wall in wall_id_to_wall.items():
        print(f"\nWall ID {wall_id}")
        for opening in wall.get("openings_precise", []):
            print(f"  - Opening ID {opening['opening_id']} | Type: {opening['type']} | Distance: {opening['distance']:.3f}")

    openings_per_wall = []

    for wall_id, wall in wall_dict.items():
        
        wall.setdefault("openings_precise", [])
        openings = wall["openings_precise"]
        
        if openings:
            print(f"\n🧱 Wall ID {wall_id}")
            wall_opening_summary = {"wall_id": wall_id, "openings": []}
            
            for opening in openings:
                opening_id = opening.get("opening_id")
                opening_type = opening.get("type")
                distance = opening.get("distance", 0.0)
                
                print(f"  🔹 Opening ID {opening_id} | Type: {opening_type} | Distance: {distance:.3f}")
                
                wall_opening_summary["openings"].append({
                    "opening_id": opening_id,
                    "type": opening_type,
                    "distance": distance
                })
            
            openings_per_wall.append(wall_opening_summary)


ADDING 2 WALLS / MORE PRECISE MATCHING

In [ ]:
concrete_match_openings_walls_improved = True

if concrete_match_openings_walls:

    import matplotlib.pyplot as plt
    from shapely import wkt
    from shapely.geometry import Polygon
    import numpy as np

    def get_wall_rotation_matrix(face):
        p0, p1, *_ = face
        x_axis = np.array(p1) - np.array(p0)
        x_axis /= np.linalg.norm(x_axis)
        if len(face) > 3:
            z_axis = np.array(face[3]) - np.array(p0)
        else:
            z_axis = np.array([0, 0, 1])
        z_axis /= np.linalg.norm(z_axis)
        y_axis = np.cross(z_axis, x_axis)
        y_axis /= np.linalg.norm(y_axis)
        return np.stack([x_axis, y_axis, z_axis], axis=1)

    def plot_wall_with_openings(wall, show=True):
        face = np.array(wall["face"])
        wall_id = wall["wall_id"]
        origin = np.mean(face, axis=0)
        R = get_wall_rotation_matrix(face)

        def to_local(pts):
            return (pts - origin) @ R

        face_local = to_local(face)
        face_2d = Polygon(face_local[:, [0, 2]])  # local XZ projection

        fig, ax = plt.subplots(figsize=(8, 3))  # Wide and short to fit elevation

        # Plot wall face
        x, z = face_2d.exterior.xy
        ax.plot(x, z, 'k-', linewidth=2, label=f"Wall {wall_id}")

        seen_labels = set()
        for opening in wall.get("openings_precise", []):
            poly = wkt.loads(opening["geometry"])
            pts = np.array(poly.exterior.coords)
            pts_local = to_local(pts)
            poly_2d = Polygon(pts_local[:, [0, 2]])
            ox, oz = poly_2d.exterior.xy

            label = opening.get("opening_id", "opening")
            if label not in seen_labels:
                ax.fill(ox, oz, color='blue', alpha=0.5, label=label)
                seen_labels.add(label)
            else:
                ax.fill(ox, oz, color='blue', alpha=0.5)

        ax.set_aspect("auto")
        ax.set_title(f"Wall {wall_id} with Openings")
        ax.set_xlabel("Wall Length (X local)")
        ax.set_ylabel("Height (Z)")
        ax.grid(True)
        ax.legend()

        if x and z:  
            x_margin = 0.1 * max(1e-6, max(x) - min(x))
            z_margin = 0.1 * max(1e-6, max(z) - min(z))
            ax.set_xlim(min(x) - x_margin, max(x) + x_margin)
            ax.set_ylim(min(z) - z_margin, max(z) + z_margin)

        plt.tight_layout()
        if show:
            plt.show()
        else:
            plt.close(fig)

    plot_check = False
    if plot_check:
        for wall in wall_dict.values():
            plot_wall_with_openings(wall)


In [ ]:
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon, LineString
from shapely import wkt
import numpy as np

def plot_matches_from_segments(wall_segments, df_matches, wall_dict):
    fig, ax = plt.subplots(figsize=(10, 10))

    for seg in wall_segments:
        line = seg['segment']
        x, y = line.xy
        ax.plot(x, y, color='black', linewidth=1)

    for _, row in df_matches.iterrows():
        wall_id = row["Matched Wall ID"]
        if wall_id is None:
            continue

        wall_face = wall_dict[wall_id]["face"]
        opening_id = row["Opening ID"]
        opening_type = row["Opening Type"]

        opening_wkt = None
        for opening in wall_dict[wall_id].get("openings_precise", []):
            if opening["opening_id"] == opening_id:
                opening_wkt = opening["geometry"]
                break

        if opening_wkt is None:
            continue

        polygon = wkt.loads(opening_wkt)
        coords = polygon.exterior.coords
        centroid = np.mean(np.array(coords)[:, :2], axis=0)
        centroid_pt = Point(centroid)

        wall_line = LineString([wall_face[0][:2], wall_face[1][:2]])
        wall_mid = wall_line.interpolate(0.5, normalized=True)

        # Draw the opening polygon
        poly_2d = Polygon([pt[:2] for pt in coords])
        x, y = poly_2d.exterior.xy
        ax.fill(x, y, alpha=0.4)

        # Arrow from opening to matched wall
        ax.annotate("",
                    xy=wall_mid.coords[0],  # (x, y)
                    xytext=centroid,        # (x, y)
                    arrowprops=dict(arrowstyle="->", color="red", lw=1.5))

        # Mark centroid
        ax.plot(*centroid_pt.xy, 'ro')

        # Optional: label
        ax.text(centroid_pt.x, centroid_pt.y, opening_id, fontsize=8, color='blue')

    ax.set_title("Opening-to-Wall Matches")
    ax.set_aspect("equal")
    ax.grid(True)
    plt.show()

plot_matches_from_segments(new_wall_segments, df_precise_matches, wall_id_to_wall)



In [ ]:
#Access opening_windows
for wall in wall_dict.values():

    for opening in wall.get('openings', []):
        if opening.get('type') == 'window':
            print(f"Wall ID: {wall['wall_id']} | Window ID: {opening['opening_id']} | Geometry window: {opening['geometry']}")

# Only doors
for wall in wall_dict.values():

    for opening in wall.get('openings', []):
        if opening.get('type') == 'door':
            print(f"Wall {wall['wall_id']} | Door ID: {opening['opening_id']} | Geometry door: {opening['geometry']}")

# Wall with windows
walls_with_windows = [
    wall for wall in wall_dict.values()
    if any(opening.get('type') == 'window' for opening in wall.get('openings', []))
]

# Print walls with window openings
for wall in walls_with_windows:
    print(f"Wall {wall['wall_id']} has windows:")
    for opening in wall['openings']:
        if opening['type'] == 'window':
            print(f" - {opening['opening_id']} | {opening['geometry']}")

# # Show exactly how many windows per wall
# for wall_id, wall in wall_dict.items():
#     windows = [o for o in wall.get('openings', []) if o.get('type') == 'window']
#     print(f"Wall {wall_id} has {len(windows)} window(s): {[w['opening_id'] for w in windows]}")



In [ ]:
def get_fine_openings(wall):
    
    from shapely import wkt
    from shapely.geometry import Polygon
    import numpy as np

    face = np.array(wall["face"])
    origin = np.mean(face, axis=0)
    R = get_wall_rotation_matrix(face)

    def to_local(pts):
        return (pts - origin) @ R

    valid_openings = []
    for opening in wall.get("openings_precise", []):
        try:
            poly = wkt.loads(opening["geometry"])
            pts = np.array(poly.exterior.coords)
            pts_local = to_local(pts)
            _ = Polygon(pts_local[:, [0, 2]])  # Just check geometry is valid in 2D
            valid_openings.append(opening)
        except Exception as e:
            print(f"Skipping problematic opening in wall {wall.get('wall_id')}: {e}")
    return valid_openings

def plot_wall_with_openings(wall, fine_openings=None, show=True):
    import matplotlib.pyplot as plt
    from shapely import wkt
    from shapely.geometry import Polygon
    import numpy as np

    face = np.array(wall["face"])
    wall_id = wall["wall_id"]
    origin = np.mean(face, axis=0)
    R = get_wall_rotation_matrix(face)

    def to_local(pts):
        return (pts - origin) @ R

    face_local = to_local(face)
    face_2d = Polygon(face_local[:, [0, 2]])

    fig, ax = plt.subplots(figsize=(8, 3))
    x, z = face_2d.exterior.xy
    ax.plot(x, z, 'k-', linewidth=2, label=f"Wall {wall_id}")

    seen_labels = set()
    if fine_openings is None:
        fine_openings = get_fine_openings(wall)

    for opening in fine_openings:
        poly = wkt.loads(opening["geometry"])
        pts = np.array(poly.exterior.coords)
        pts_local = to_local(pts)
        poly_2d = Polygon(pts_local[:, [0, 2]])
        ox, oz = poly_2d.exterior.xy

        label = opening.get("opening_id", "opening")
        if label not in seen_labels:
            ax.fill(ox, oz, color='blue', alpha=0.5, label=label)
            seen_labels.add(label)
        else:
            ax.fill(ox, oz, color='blue', alpha=0.5)

    ax.set_aspect("auto")
    ax.set_title(f"Wall {wall_id} with Openings")
    ax.set_xlabel("Wall Length (X local)")
    ax.set_ylabel("Height (Z)")
    ax.grid(True)
    ax.legend()

    if x and z:
        x_margin = 0.1 * max(1e-6, max(x) - min(x))
        z_margin = 0.1 * max(1e-6, max(z) - min(z))
        ax.set_xlim(min(x) - x_margin, max(x) + x_margin)
        ax.set_ylim(min(z) - z_margin, max(z) + z_margin)

    plt.tight_layout()
    if show:
        plt.show()
    else:
        plt.close(fig)

openings_by_wall = {}

for wall_id, wall in wall_dict.items():
    fine_openings = get_fine_openings(wall)
    openings_by_wall[wall_id] = fine_openings

# for wall_id, wall in wall_dict.items():
#     plot_wall_with_openings(wall, fine_openings=openings_by_wall[wall_id])


CUT BOTH

In [ ]:
cut_both_shered_walls = True

if cut_both_shered_walls:

    import numpy as np
    from shapely import wkt
    from shapely.geometry import Polygon, LineString
    from shapely.ops import unary_union
    from shapely.validation import make_valid

    def extract_opening_centroid_and_normal(geometry_wkt):
        polygon = wkt.loads(geometry_wkt)
        coords = np.array(polygon.exterior.coords)
        centroid = np.mean(coords[:, :2], axis=0)
        if len(coords) >= 2:
            dx, dy = coords[1][:2] - coords[0][:2]
            normal = np.array([-dy, dx])
            norm = np.linalg.norm(normal)
            return centroid, normal / norm if norm != 0 else np.array([0.0, 0.0])
        return centroid, np.array([0.0, 0.0])

    def get_wall_line(face):
        return LineString([face[0][:2], face[1][:2]])

    def z_overlap(opening_geom, wall_face, margin=0.2):
        polygon = wkt.loads(opening_geom)
        z_opening = np.array(polygon.exterior.coords)[:, 2]
        z_min_opening, z_max_opening = z_opening.min(), z_opening.max()
        wall_zs = np.array([pt[2] for pt in wall_face])
        z_min_wall, z_max_wall = wall_zs.min(), wall_zs.max()
        return not (z_max_opening < z_min_wall - margin or z_min_opening > z_max_wall + margin)

    def opening_to_wall_distance(opening_wkt, wall_line):
        return wall_line.distance(wkt.loads(opening_wkt))

    def get_wall_rotation_matrix(face):
        p0, p1, *_ = face
        x_axis = np.array(p1) - np.array(p0)
        x_axis /= np.linalg.norm(x_axis)
        z_axis = np.array(face[3]) - np.array(p0) if len(face) > 3 else np.array([0, 0, 1])
        z_axis /= np.linalg.norm(z_axis)
        y_axis = np.cross(z_axis, x_axis)
        y_axis /= np.linalg.norm(y_axis)
        return np.stack([x_axis, y_axis, z_axis], axis=1)

    def assign_openings_to_all_matching_walls(walls_dict, opening_wkts, threshold=0.5):
        result = []
        wall_lines = {wid: get_wall_line(w["face"]) for wid, w in walls_dict.items()}
        for wall in walls_dict.values():
            wall.setdefault("openings_precise", [])

        for opening in opening_wkts:
            opening_id = opening.get("opening_id")
            opening_geom = opening.get("geometry")
            opening_type = opening.get("type")
            centroid, normal = extract_opening_centroid_and_normal(opening_geom)

            for wall_id, wall in walls_dict.items():
                wall_line = wall_lines[wall_id]
                if z_overlap(opening_geom, wall["face"]):
                    dist = opening_to_wall_distance(opening_geom, wall_line)
                    if dist < threshold:
                        if not any(op["opening_id"] == opening_id for op in wall["openings_precise"]):
                            wall["openings_precise"].append({
                                "opening_id": opening_id,
                                "type": opening_type,
                                "geometry": opening_geom,
                                "distance": dist
                            })
                            result.append({
                                "Opening ID": opening_id,
                                "Opening Type": opening_type,
                                "Matched Wall ID": wall_id,
                                "Distance": round(dist, 4)
                            })
        return result

    def process_wall_geometry_with_holes(wall_dict, base_buffer=0.075, top_buffer=0.075, tolerance=0.01):
        processed_walls = []

        for _, wall_data in wall_dict.items():
            wall = wall_data.copy()
            wall["original_wall_id"] = wall_data["wall_id"]

            face = np.array(wall['face'])
            origin = np.mean(face, axis=0)
            R = get_wall_rotation_matrix(face)

            def to_local(pts):
                return (pts - origin) @ R

            face_local = to_local(face)
            face_2d = Polygon(face_local[:, [0, 2]])  # Project to XZ
            wall["wall_complete_shape"] = face_2d
            z_wall = face[:, 2]
            z_min_wall, z_max_wall = z_wall.min(), z_wall.max()

            opening_polygons = []

            for opening in wall.get("openings_precise", []):
                poly_global = wkt.loads(opening["geometry"])
                coords = np.array(poly_global.exterior.coords)
                z_coords = coords[:, 2]

                # Adjust bottom
                if np.abs(z_coords.min() - z_min_wall) <= tolerance:
                    coords[:, 2] = np.where(
                        z_coords == z_coords.min(),
                        z_coords + base_buffer,
                        z_coords
                    )

                # Adjust top
                if np.abs(z_coords.max() - z_max_wall) <= tolerance:
                    coords[:, 2] = np.where(
                        z_coords == z_coords.max(),
                        z_coords - top_buffer,
                        z_coords
                    )

                pts_local = to_local(coords)
                poly_2d = Polygon(pts_local[:, [0, 2]])

                # Validate hole
                if not poly_2d.is_valid:
                    poly_2d = make_valid(poly_2d)
                if not poly_2d.is_valid:
                    poly_2d = poly_2d.buffer(0)

                if poly_2d.is_empty or not face_2d.contains(poly_2d):
                    print(f"⚠️ Skipping bad hole (invalid or out of bounds) in wall {wall['wall_id']}")
                    continue

                # Buffer shrink to avoid edge touching
                bbox = poly_2d.bounds
                size = min(bbox[2] - bbox[0], bbox[3] - bbox[1])
                shrink_amount = min(0.01, size / 10.0)
                poly_2d = poly_2d.buffer(-shrink_amount)

                if poly_2d.is_empty or not poly_2d.is_valid:
                    print(f"⚠️ Skipping shrunk hole (invalid) in wall {wall['wall_id']}")
                    continue

                opening_polygons.append({
                    "polygon": poly_2d,
                    "type": opening["type"],
                    "id": opening.get("opening_id")
                })

            if opening_polygons:
                holes = unary_union([op["polygon"] for op in opening_polygons if op["polygon"].is_valid])
                if not holes.is_valid or holes.is_empty:
                    print(f"⚠️ Hole union invalid in wall {wall['wall_id']}")
                    holes = None

                if holes:
                    face_with_openings = face_2d.difference(holes)
                    if face_with_openings.is_empty or not face_with_openings.is_valid or not isinstance(face_with_openings, Polygon):
                        print(f"⚠️ Invalid wall geometry after cutting holes (wall_id = {wall['wall_id']}), skipping cut.")
                        face_with_openings = face_2d
                    else:
                        print(f"✅ Wall {wall['wall_id']} cut successfully.")
                else:
                    face_with_openings = face_2d
            else:
                face_with_openings = face_2d

            wall["wall_shape_2d"] = face_with_openings
            wall["openings_2d"] = opening_polygons
            wall["local_rotation"] = R
            wall["local_origin"] = origin

            processed_walls.append(wall)

        return processed_walls

    # Usage
    assign_openings_to_all_matching_walls(wall_dict, opening_wkts, threshold=0.5)
    processed_walls = process_wall_geometry_with_holes(wall_dict)
    print(f'✅ processed_walls generated: {processed_walls}')
    

In [ ]:
visual_holes_creation_check = False

if visual_holes_creation_check:

    import matplotlib.pyplot as plt
    from shapely.geometry.polygon import orient

    def plot_cut_wall(wall_data):

        shape = wall_data["wall_shape_2d"]
        wall_id = wall_data.get("wall_id", "Unknown")

        shape = orient(shape, sign=1.0)

        fig, ax = plt.subplots(figsize=(6, 8))  # Taller aspect
        plotted = False  # Track if any shape was actually drawn

        if shape.geom_type == 'Polygon':
            xs, ys = shape.exterior.xy
            ax.plot(xs, ys, color='black')
            plotted = True

            if shape.interiors:
                for hole in shape.interiors:
                    if not hole.is_empty:
                        hx, hy = hole.xy
                        ax.fill(hx, hy, color='lightblue')
                    else:
                        print(f"Wall {wall_id} has an empty hole (ignored).")
            else:
                print(f"Wall {wall_id} has no holes.")

        elif shape.geom_type == 'MultiPolygon':
            for i, poly in enumerate(shape):
                xs, ys = poly.exterior.xy
                ax.plot(xs, ys, color='black')
                plotted = True

                if poly.interiors:
                    for hole in poly.interiors:
                        if not hole.is_empty:
                            hx, hy = hole.xy
                            ax.fill(hx, hy, color='lightblue')
                        else:
                            print(f"Wall {wall_id}, Polygon {i} has an empty hole (ignored).")
                else:
                    print(f"Wall {wall_id}, Polygon {i} has no holes.")

        else:
            print(f"Wall {wall_id} has unsupported geometry type: {shape.geom_type}")

        if plotted:
            ax.set_title(f"Wall {wall_id} with Openings Cut")
            ax.set_aspect('equal')
            ax.grid(True)
            plt.show()
        else:
            print(f"Wall {wall_id} was not plotted due to invalid geometry.")

    for wall in processed_walls:
        plot_cut_wall(wall)


TEST MERGE 01

In [ ]:
wall_sign_vertices = -1
wall_openings_sign_vertices = 1
openings_sign_vertices = -1

test_merged_rooms = True

if test_merged_rooms:

    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    from shapely.geometry import Polygon, MultiPolygon
    from shapely.geometry.polygon import orient
    from shapely.ops import unary_union
    from shapely.wkt import dumps, loads
    from collections import defaultdict

    def to_local_coords(pts, origin, R):
        return (pts - origin) @ R

    def get_wall_rotation_matrix(face):
        v1, v2 = face[1] - face[0], face[2] - face[0]
        normal = np.cross(v1, v2)
        normal = normal / np.linalg.norm(normal)
        x_axis = v1 / np.linalg.norm(v1)
        z_axis = np.cross(normal, x_axis)
        z_axis /= np.linalg.norm(z_axis)
        return np.linalg.inv(np.stack([x_axis, normal, z_axis], axis=1))

    def is_significant_opening(poly_2d, min_size=0.1):
        minx, miny, maxx, maxy = poly_2d.bounds
        return (maxx - minx) >= min_size and (maxy - miny) >= min_size

    def convert_openings_to_2d(wall, min_opening_size=0.1):
        face = np.array(wall["face"])
        origin = np.mean(face, axis=0)
        R = get_wall_rotation_matrix(face)
        openings_2d = []

        for opening in wall.get("openings_precise", []):
            poly_global = loads(opening["geometry"])
            pts_global = np.array(poly_global.exterior.coords)
            pts_local = to_local_coords(pts_global, origin, R)
            poly_2d = Polygon(pts_local[:, [0, 2]])  # Use XZ-plane

            if is_significant_opening(poly_2d, min_opening_size):
                openings_2d.append({
                    "polygon": poly_2d,
                    "type": opening["type"],
                    "id": opening.get("opening_id")
                })
        return openings_2d

    def cut_openings_into_wall(wall_polygon, opening_polygons):
        if not opening_polygons:
            return wall_polygon
        openings_union = unary_union(opening_polygons)
        result = wall_polygon.difference(openings_union)

        if isinstance(result, MultiPolygon):
            largest = max(result.geoms, key=lambda p: p.area)
            holes = [poly.exterior.coords for poly in result.geoms if poly != largest]
            return Polygon(largest.exterior.coords, holes)

        return result

    def to_global_coords(pts_local, origin, R):
        return pts_local @ R.T + origin

    def to_multipolygon(geom):
        if geom.geom_type == "Polygon":
            return MultiPolygon([geom])
        elif geom.geom_type == "MultiPolygon":
            return geom
        else:
            raise ValueError(f"Unexpected geometry type: {geom.geom_type}")

    from shapely.geometry import Polygon, MultiPolygon
    from shapely.wkt import dumps

    def to_multipolygon_3d(polygon_list):
        shapely_polys = []
        for coords in polygon_list:
            if len(coords) >= 3:
                shapely_polys.append(Polygon(coords))
        return MultiPolygon(shapely_polys)


    wall_export = []


    for wall in processed_walls:
        wall_id = wall.get("original_wall_id", wall.get("wall_id"))
        room_id = wall.get("room_id", None)

        face = np.array(wall["face"])
        origin = np.mean(face, axis=0)
        R = get_wall_rotation_matrix(face)

        wall_flat = orient(Polygon(wall["wall_shape_2d"].exterior), sign=wall_sign_vertices)

        pts_local = np.array(wall_flat.exterior.coords)
        pts_3d = to_global_coords(np.column_stack((pts_local[:, 0], np.zeros(len(pts_local)), pts_local[:, 1])), origin, R)
        wall_reprojected = Polygon(pts_3d)

        wall["openings_2d"] = convert_openings_to_2d(wall)
        openings = wall["openings_2d"]
        opening_polys = [o["polygon"] for o in openings]
        has_holes = bool(openings)

        # Original wall (no holes)
        wall_export.append({
            "room_id": room_id,
            "wall_id": wall_id,
            "opening_id": None,
            "geometry_type": "wall",
            "variant": "plain",
            "wall_wkt": dumps(to_multipolygon(wall_reprojected), output_dimension=3, rounding_precision=5),
            "has_holes": False
        })

        if has_holes:
            wall_with_holes_2d = cut_openings_into_wall(wall_flat, opening_polys)
            wall_with_holes_2d = orient(wall_with_holes_2d, sign=wall_openings_sign_vertices)

            pts_local = np.array(wall_with_holes_2d.exterior.coords)
            pts_3d = to_global_coords(np.column_stack((pts_local[:, 0], np.zeros(len(pts_local)), pts_local[:, 1])), origin, R)
            wall_with_holes_3d = Polygon(pts_3d)

    
            if has_holes:
                wall_export.append({
                    "room_id": room_id,
                    "wall_id": wall_id,
                    "opening_id": None,
                    "geometry_type": "wall",
                    "variant": "with_holes",
                    "wall_wkt": dumps(to_multipolygon(wall_with_holes_3d), output_dimension=3, rounding_precision=5),
                    "has_holes": True
                })

        for opening in openings:
            opening_poly_2d = orient(opening["polygon"], sign=openings_sign_vertices)
            opening_coords_local = np.array(opening_poly_2d.exterior.coords)
            opening_coords_3d = to_global_coords(
                np.column_stack((opening_coords_local[:, 0], np.zeros(len(opening_coords_local)), opening_coords_local[:, 1])),
                origin, R
            )
            opening_poly_3d = Polygon(opening_coords_3d)

            wall_export.append({
                "room_id": room_id,
                "wall_id": wall_id,
                "opening_id": opening.get("id"),
                "geometry_type": "opening",
                "opening_wkt": dumps(to_multipolygon(opening_poly_3d), output_dimension=3, rounding_precision=5),
                "has_holes": False
            })


    df_export = pd.DataFrame(wall_export)
    df_export = df_export[["room_id", "wall_id", "opening_id", "geometry_type", "wall_wkt", "opening_wkt", "has_holes"]]
    df_export.head(60)

    visual_holes_creation_check = False

    if visual_holes_creation_check:

        def plot_cut_wall(wall_data):
            shape = wall_data["wall_shape_2d"]
            wall_id = wall_data.get("wall_id", "Unknown")
            shape = orient(shape, sign=1.0)

            fig, ax = plt.subplots(figsize=(6, 8))
            plotted = False

            if shape.geom_type == 'Polygon':
                xs, ys = shape.exterior.xy
                ax.plot(xs, ys, color='black')
                plotted = True
                for hole in shape.interiors:
                    if not hole.is_empty:
                        hx, hy = hole.xy
                        ax.fill(hx, hy, color='lightblue')

            elif shape.geom_type == 'MultiPolygon':
                for i, poly in enumerate(shape):
                    xs, ys = poly.exterior.xy
                    ax.plot(xs, ys, color='black')
                    plotted = True
                    for hole in poly.interiors:
                        if not hole.is_empty:
                            hx, hy = hole.xy
                            ax.fill(hx, hy, color='lightblue')

            if plotted:
                ax.set_title(f"Wall {wall_id} with Openings Cut")
                ax.set_aspect('equal')
                ax.grid(True)
                plt.show()

        for wall in processed_walls:
            plot_cut_wall(wall)



In [ ]:
df_export.head(60)

In [ ]:
openings_matching_final = True
already_did_this = True

if openings_matching_final:

    max_distance_cut = 0.025  # 2.5 cm

    from shapely.geometry import Polygon
    from collections import defaultdict
    import pandas as pd

    def wall_centroid_2d(wall):
        return wall["wall_shape_2d"].centroid

    def opening_cut_correctly(wall, opening):
        return not wall["wall_shape_2d"].contains(opening["polygon"])

    filtered_processed_walls = []

    for wall in processed_walls:
        valid_openings = []
        wall_shape = wall["wall_shape_2d"]

        for opening in wall["openings_2d"]:
            distance = wall_shape.distance(opening["polygon"])

            if distance <= max_distance_cut:
                valid_openings.append(opening)
            else:
                print(f"Skipping opening {opening.get('id')} on wall {wall['wall_id']} — too far (distance = {distance:.4f})")

        wall_copy = wall.copy()
        wall_copy["openings_2d"] = valid_openings
        filtered_processed_walls.append(wall_copy)

    processed_walls = filtered_processed_walls

if already_did_this:

    import numpy as np
    import pandas as pd
    from shapely.geometry import Polygon, MultiPolygon
    from shapely.geometry.polygon import orient
    from shapely.ops import unary_union
    from shapely.wkt import dumps, loads
    from collections import defaultdict

    def polygon_2d_to_3d(polygon_2d, R, origin):
        def transform_ring(ring):
            coords_2d = np.array(ring)
            local_3d = np.column_stack([coords_2d[:, 0], np.zeros(len(coords_2d)), coords_2d[:, 1]])
            return (local_3d @ R.T) + origin

        exterior_3d = transform_ring(polygon_2d.exterior.coords)
        interiors_3d = [transform_ring(ring.coords) for ring in polygon_2d.interiors]
        return Polygon(exterior_3d, interiors_3d)

    def to_multipolygon(geom):
        if geom.geom_type == "Polygon":
            return MultiPolygon([geom])
        elif geom.geom_type == "MultiPolygon":
            return geom
        else:
            raise ValueError(f"Unexpected geometry type: {geom.geom_type}")

    wall_export_3d = []

    for wall in processed_walls:
        wall_id = wall.get("original_wall_id", wall.get("wall_id"))
        room_id = wall.get("room_id")
        wall_shape_2d = wall["wall_shape_2d"]
        R = wall["local_rotation"]
        origin = wall["local_origin"]

        wall_local = orient(Polygon(wall_shape_2d.exterior), sign=-1.0)
        openings = wall["openings_2d"]
        opening_polys = [o["polygon"] for o in openings]
        has_holes = bool(openings)

        # Step 1: Export wall (without holes)
        wall_3d = polygon_2d_to_3d(wall_local, R, origin)
        wall_export_3d.append({
            "room_id": room_id,
            "wall_id": wall_id,
            "opening_id": None,
            "geometry_type": "wall",
            "wall_wkt": dumps(to_multipolygon(wall_3d), rounding_precision=3),
            "opening_wkt": None,
            "has_holes": False
        })

        # Step 2: Wall with openings cut
        if has_holes:
            try:
                holes = [o for o in opening_polys if o.is_valid and not o.is_empty]
                wall_with_holes = wall_local
                for hole in holes:
                    wall_with_holes = wall_with_holes.difference(hole)

                wall_with_holes = orient(wall_with_holes, sign=1.0)
                wall_with_holes_3d = polygon_2d_to_3d(wall_with_holes, R, origin)

                wall_export_3d.append({
                    "room_id": room_id,
                    "wall_id": wall_id,
                    "opening_id": None,
                    "geometry_type": "wall_with_holes",
                    "wall_wkt": dumps(to_multipolygon(wall_with_holes_3d), rounding_precision=3),
                    "opening_wkt": None,
                    "has_holes": True
                })
            except Exception as e:
                print(f"Warning: could not cut holes in wall {wall_id}: {e}")

        # Step 3: Export openings
        for opening in openings:
            opening_poly_local = orient(opening["polygon"], sign=-1.0)
            opening_poly_global = polygon_2d_to_3d(opening_poly_local, R, origin)

            wall_export_3d.append({
                "room_id": room_id,
                "wall_id": wall_id,
                "opening_id": opening.get("id"),
                "geometry_type": "opening",
                "wall_wkt": None,
                "opening_wkt": dumps(to_multipolygon(opening_poly_global), rounding_precision=3),
                "has_holes": True
            })

    df_export = pd.DataFrame(wall_export_3d)
    df_export = df_export[[
        "room_id", "wall_id", "opening_id", "geometry_type",
        "wall_wkt", "opening_wkt", "has_holes"
    ]]

    # Display settings
    pd.set_option('display.max_rows', 80)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 1000)

    df_export.head(80)
    

In [ ]:
check_openings_walls_plotting = True

if check_openings_walls_plotting:

    df_export["geometry"] = df_export["wall_wkt"].fillna(df_export["opening_wkt"]).apply(wkt.loads)

    plots = []

    geometry_types = df_export["geometry_type"].unique()

    for gtype in geometry_types:
        subset = df_export[df_export["geometry_type"] == gtype]
        fig, ax = plt.subplots(figsize=(10, 6))
        for geom in subset["geometry"]:
            if geom.geom_type == "MultiPolygon":
                for poly in geom.geoms:
                    x, y = poly.exterior.xy
                    ax.plot(x, y)
            elif geom.geom_type == "Polygon":
                x, y = geom.exterior.xy
                ax.plot(x, y)
        ax.set_title(f"Geometries: {gtype}")
        ax.set_aspect("equal")
        plt.show() 
        

RELATED TO COLUMN

In [ ]:
# # for wall_id, wall_data in wall_id_to_wall.items():
# #     face = wall_data['face']

# #     face_2d = [(x, y) for (x, y, z) in face]
# #     edges = list(zip(face, face[1:] + [face[0]]))

# #     bottom_edge = min(edges, key=lambda e: (e[0][2] + e[1][2]) / 2)
# #     base_segment_2d = [(bottom_edge[0][0], bottom_edge[0][1]), (bottom_edge[1][0], bottom_edge[1][1])]

# #     wall_segment = base_segment_2d
# #     print(f"Wall ID {wall_id}: base segment in 2D = {wall_segment}")

# # from collections import defaultdict

# # # Dictionary to group wall IDs by their normalized 2D segment
# segment_to_wall_ids = defaultdict(list)

# for wall_id, wall_data in wall_id_to_wall.items():

#     face = wall_data['face']

#     edges = list(zip(face, face[1:] + [face[0]]))
#     bottom_edge = min(edges, key=lambda e: (e[0][2] + e[1][2]) / 2)
    
#     # Convert to 2D
#     p1 = (bottom_edge[0][0], bottom_edge[0][1])
#     p2 = (bottom_edge[1][0], bottom_edge[1][1])
    
#     # Normalize: always order points consistently
#     segment = tuple(sorted([p1, p2]))
    
#     # Group wall IDs by this normalized segment
#     segment_to_wall_ids[segment].append(wall_id)

# # Report duplicates
# for segment, wall_ids in segment_to_wall_ids.items():
#     if len(wall_ids) > 1:
#         print(f"Duplicate walls with segment {segment}: Wall IDs {wall_ids}")

# from collections import defaultdict

# segment_to_wall_ids = defaultdict(list)
# external_walls = []

# for wall_id, wall_data in wall_id_to_wall.items():
#     face = wall_data.get('face')

#     if not face:
#         # No face = external wall
#         external_walls.append(wall_id)
#         continue  # Skip this wall

#     # Find base segment (lowest average Z edge)
#     edges = list(zip(face, face[1:] + [face[0]]))
#     bottom_edge = min(edges, key=lambda e: (e[0][2] + e[1][2]) / 2)

#     # Project to 2D (drop Z)
#     p1 = (bottom_edge[0][0], bottom_edge[0][1])
#     p2 = (bottom_edge[1][0], bottom_edge[1][1])

#     # Normalize segment
#     segment = tuple(sorted([p1, p2]))

#     # Group wall IDs by this segment
#     segment_to_wall_ids[segment].append(wall_id)

# # ✅ Report duplicate internal wall segments
# for segment, wall_ids in segment_to_wall_ids.items():
#     if len(wall_ids) > 1:
#         print(f"Duplicate internal walls found with segment {segment}: Wall IDs {wall_ids}")

# # 🚫 List external (unmatched) walls
# if external_walls:
#     print(f"\nExternal walls without face (skipped): {external_walls}")


In [ ]:
# wall_id_to_wall

In [ ]:
# from collections import defaultdict

# # Redefine in case previous session state is missing
# matched_wall_ids = set()
# external_walls = set()
# all_wall_ids = set(wall_id_to_wall.keys())

# # Dictionary to group wall IDs by normalized 2D base segment
# segment_to_wall_ids = defaultdict(list)

# # Go through each wall and compute base segment if it has a face
# for wall_id, wall_data in wall_id_to_wall.items():
#     all_wall_ids.add(wall_id)
#     face = wall_data.get('face')

#     if not face:
#         external_walls.add(wall_id)
#         continue

#     # Get base edge as lowest average Z
#     edges = list(zip(face, face[1:] + [face[0]]))
#     bottom_edge = min(edges, key=lambda e: (e[0][2] + e[1][2]) / 2)

#     # Project to 2D and normalize
#     p1 = (bottom_edge[0][0], bottom_edge[0][1])
#     p2 = (bottom_edge[1][0], bottom_edge[1][1])
#     segment = tuple(sorted([p1, p2]))

#     segment_to_wall_ids[segment].append(wall_id)

# # Track matched wall IDs
# for wall_ids in segment_to_wall_ids.values():
#     if len(wall_ids) > 1:
#         matched_wall_ids.update(wall_ids)

# # Determine unmatched walls (with face but no match)
# walls_with_face = all_wall_ids - external_walls
# unmatched_internal_walls = walls_with_face - matched_wall_ids

# (len(all_wall_ids), len(matched_wall_ids), len(unmatched_internal_walls), list(unmatched_internal_walls)[:10])


In [ ]:
# match_rows = []
# for wall_ids in segment_to_wall_ids.values():
#     if len(wall_ids) > 1:
#         for wall_id in wall_ids:
#             matched = [wid for wid in wall_ids if wid != wall_id]
#             match_rows.append({'wall_id': wall_id, 'matched_walls': matched})

# df_matches_related_to = pd.DataFrame(match_rows).drop_duplicates(subset=['wall_id'])
# df_matches_related_to.head(80)

CEILINGS AND FLOORS

In [ ]:
floors_with_normals = True

if floors_with_normals:
    from shapely.geometry import Polygon, MultiPolygon
    from shapely.geometry.polygon import orient
    from shapely.ops import unary_union
    from shapely.wkt import dumps
    import numpy as np
    import pandas as pd

    # --- PARAMETERS ---
    compute_normals = True  # Set to False to skip printing normals
    polygon_id = 0

    # --- HELPERS ---
    def polygon_3d_from_2d(polygon_2d, z):
        z = round(z, 3)
        def to_3d(poly):
            exterior_3d = [(round(x, 3), round(y, 3), z) for x, y in poly.exterior.coords]
            interiors_3d = [
                [(round(x, 3), round(y, 3), z) for x, y in ring.coords]
                for ring in poly.interiors
            ]
            return Polygon(exterior_3d, interiors_3d)
        if isinstance(polygon_2d, Polygon):
            return MultiPolygon([to_3d(polygon_2d)])
        elif isinstance(polygon_2d, MultiPolygon):
            return MultiPolygon([to_3d(p) for p in polygon_2d.geoms])
        else:
            raise TypeError("Expected Polygon or MultiPolygon")

    def round_polygon_coords(poly, precision=3):
        def round_coords(coords):
            return [(round(x, precision), round(y, precision), round(z, precision)) for x, y, z in coords]
        if isinstance(poly, Polygon):
            exterior = round_coords(poly.exterior.coords)
            interiors = [round_coords(interior.coords) for interior in poly.interiors]
            return Polygon(exterior, interiors)
        elif isinstance(poly, MultiPolygon):
            return MultiPolygon([round_polygon_coords(p, precision) for p in poly.geoms])
        else:
            raise TypeError("Expected Polygon or MultiPolygon")

    def polygon_normal(polygon_3d):
        coords = list(polygon_3d.exterior.coords)
        if len(coords) < 3:
            return None
        p0, p1, p2 = np.array(coords[:3])
        v1 = p1 - p0
        v2 = p2 - p0
        normal = np.cross(v1, v2)
        norm = np.linalg.norm(normal)
        return normal / norm if norm != 0 else None

    def generate_floor_ceiling(room_id, room_geom, z_min, z_max, origin, R, polygon_id_start=0):
        floor_data = []
        ceiling_data = []
        polygons = list(room_geom.geoms) if isinstance(room_geom, MultiPolygon) else [room_geom]
        polygon_id = polygon_id_start

        for poly_2d in polygons:
            if not poly_2d.is_valid or poly_2d.area == 0:
                continue

            for surface_type, sign, z in [("floor", -1.0, z_min), ("ceiling", 1.0, z_max)]:
                poly_oriented = orient(poly_2d, sign=sign)
                poly_3d = polygon_3d_from_2d(poly_oriented, z)
                poly_rounded = round_polygon_coords(poly_3d)

                for subpoly in poly_rounded.geoms:
                    normal = polygon_normal(subpoly) if compute_normals else None

                    surface_data = {
                        "id": polygon_id,
                        "room_id": room_id,
                        "type": surface_type,
                        "wkt": dumps(subpoly, output_dimension=3, rounding_precision=3, trim=True),
                        "local_origin": origin.tolist(),
                        "local_rotation": R.tolist(),
                        "normal": normal.tolist() if normal is not None else None
                    }

                    polygon_id += 1

                    if surface_type == "floor":
                        floor_data.append(surface_data)
                    else:
                        ceiling_data.append(surface_data)

        return floor_data, ceiling_data, polygon_id

    # --- ROOM INPUTS ---
    room_local_origin = {}
    room_local_rotation = {}
    for wall in processed_walls:
        room_id = wall["room_id"]
        if room_id not in room_local_origin:
            room_local_origin[room_id] = wall["local_origin"]
            room_local_rotation[room_id] = wall["local_rotation"]

    all_surfaces = []

    for room_id, room_geom in enumerate(merged_rooms):
        origin = room_local_origin.get(room_id, np.zeros(3))
        R = room_local_rotation.get(room_id, np.eye(3))

        room_shape = unary_union(room_geom)
        if room_shape.is_empty or not room_shape.is_valid or room_shape.area == 0:
            continue
        if room_shape.geom_type == "Polygon":
            room_shape = MultiPolygon([room_shape])

        floor_data, ceiling_data, polygon_id = generate_floor_ceiling(
            room_id, room_shape, z_min, z_max, origin, R, polygon_id
        )
        all_surfaces.extend(floor_data + ceiling_data)

    # --- OUTPUT LOGGING ---
    for surface in all_surfaces:
        print(f"ID: {surface['id']} | Room {surface['room_id']} | {surface['type'].upper()} | Normal: {surface['normal']}")
        print(surface["wkt"])
        print()

    # --- EXPORT TO DATAFRAME ---
    df_surfaces = pd.DataFrame(all_surfaces)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 1000)
    df_surfaces.head(41)
    

In [ ]:
df_export.head(60)

In [ ]:
# df_export["wall_id"] = pd.to_numeric(df_export["wall_id"], errors="coerce")
# df_export = df_export.merge(df_matches_related_to, on="wall_id", how="left")

# df_export.head(80)

ALL ELEMENTS (IN DATAFRAME)

In [ ]:
import pandas as pd
from shapely.wkt import loads, dumps
from shapely.geometry import Polygon, MultiPolygon

def ensure_multipolygon(geometry):
    if isinstance(geometry, Polygon):
        return MultiPolygon([geometry])
    elif isinstance(geometry, MultiPolygon):
        return geometry
    else:
        raise TypeError(f"Unexpected geometry type: {type(geometry)}")

wall_geometry_mapping = {}

def create_room_hierarchy(room_df):
    room_id = room_df['room_id'].iloc[0]
    room_data = {
        'room_id': room_id,
        'walls': [],
        'openings': [],
        'floors': [],
        'ceilings': [],
        'parent_ids': {}  # For storing parent-child relations (wall-openings)
    }
    
    # Separate elements into walls, openings, floors, and ceilings
    for _, row in room_df.iterrows():
        if row['geometry_type'] == 'wall' or row['geometry_type'] == 'wall_with_holes':
            wall_geometry = row['geometry']
            element_id = f"wall_{room_id}_{row['wall_id']}"  # Unique element ID for wall
            
            # Check if this geometry has already been encountered (for walls with and without holes)
            if wall_geometry not in wall_geometry_mapping:
                wall_geometry_mapping[wall_geometry] = element_id
            else:
                # Reuse the existing element_id for the wall with holes if the geometry matches
                element_id = wall_geometry_mapping[wall_geometry]
            
            room_data['walls'].append({
                'wall_id': row['wall_id'],
                'geometry': row['geometry'],
                'has_holes': row['has_holes'],
                'opening_id': row['opening_id'],
                'element_id': element_id
            })
        elif row['geometry_type'] == 'opening':
            element_id = f"opening_{room_id}_{row['opening_id']}"  # Unique element ID for opening
            wall_id = row['wall_id']
            room_data['openings'].append({
                'opening_id': row['opening_id'],
                'geometry': row['geometry'],
                'wall_id': wall_id,
                'element_id': element_id
            })
            # Track wall-openings relation (parent-child association)
            room_data['parent_ids'][element_id] = f"wall_{room_id}_{wall_id}"
        elif row['geometry_type'] == 'floor':
            element_id = f"floor_{room_id}"  # Unique ID for the floor
            room_data['floors'].append({
                'geometry': row['geometry'],
                'element_id': element_id
            })
        elif row['geometry_type'] == 'ceiling':
            element_id = f"ceiling_{room_id}"  # Unique ID for the ceiling
            room_data['ceilings'].append({
                'geometry': row['geometry'],
                'element_id': element_id
            })

    return room_data

df_export_grouped = df_export.groupby('room_id')
room_hierarchy = [create_room_hierarchy(room_df) for _, room_df in df_export_grouped]

building_hierarchy = {
    'building_id': 1,
    'rooms': room_hierarchy
}

flat_data = []

for room in building_hierarchy['rooms']:
    room_id = room['room_id']
    
    for wall in room['walls']:
        flat_data.append({
            'element_id': wall['element_id'],  # Wall ID as element_id
            'thematic_surface': 'wall',  # Type of element
            'geometry': wall['geometry'],  # Wall geometry
            'parent_id': None,  # No parent ID for the wall itself
            'has_holes': wall['has_holes']  # Has holes in the wall (if opening exists)
        })
    
    for opening in room['openings']:
        flat_data.append({
            'element_id': opening['element_id'],  # Opening ID as element_id
            'thematic_surface': 'opening',  # Type of element
            'geometry': opening['geometry'],  # Opening geometry
            'parent_id': room['parent_ids'].get(opening['element_id'], None),  # Parent is the wall ID
            'has_holes': True  # Openings always imply the wall has holes
        })
    
    for floor in room['floors']:
        flat_data.append({
            'element_id': floor['element_id'],  # Unique ID for the floor
            'thematic_surface': 'floor',  # Type of element
            'geometry': floor['geometry'],  # Floor geometry
            'parent_id': room_id,  # Parent for floors is the room itself
            'has_holes': False  # Floors do not have holes
        })
    
    for ceiling in room['ceilings']:
        flat_data.append({
            'element_id': ceiling['element_id'],  # Unique ID for the ceiling
            'thematic_surface': 'ceiling',  # Type of element
            'geometry': ceiling['geometry'],  # Ceiling geometry
            'parent_id': room_id,  # Parent for ceilings is the room itself
            'has_holes': False  # Ceilings do not have holes
        })

df_flat = pd.DataFrame(flat_data)

df_flat = df_flat[['element_id', 'thematic_surface', 'geometry', 'parent_id', 'has_holes']]

df_flat = df_flat.sort_values(by=['element_id']).reset_index(drop=True)
df_flat.head(50)


In [ ]:
print(df_export.columns)
room_data = df_export[df_export['room_id'] == room_id].to_dict(orient='records')
print(room_data[0].keys())


In [ ]:
df_export

In [ ]:
test_merged_rooms_floors = True

if test_merged_rooms_floors:

    from shapely.geometry import Polygon, MultiPolygon
    from shapely.geometry.polygon import orient
    from shapely.ops import unary_union
    from shapely.wkt import dumps
    import numpy as np
    import pandas as pd

    # --- PARAMETERS ---
    compute_normals = True  # Set to False to skip printing normals
    polygon_id = 0

    # --- HELPERS ---
    # def polygon_3d_from_2d(polygon_2d, z):
    #     z = round(z, 3)
    #     def to_3d(poly):
    #         exterior_3d = [(round(x, 3), round(y, 3), z) for x, y in poly.exterior.coords]
    #         interiors_3d = [
    #             [(round(x, 3), round(y, 3), z) for x, y in ring.coords]
    #             for ring in poly.interiors
    #         ]
    #         return Polygon(exterior_3d, interiors_3d)
    #     if isinstance(polygon_2d, Polygon):
    #         return MultiPolygon([to_3d(polygon_2d)])
    #     elif isinstance(polygon_2d, MultiPolygon):
    #         return MultiPolygon([to_3d(p) for p in polygon_2d.geoms])
    #     else:
    #         raise TypeError("Expected Polygon or MultiPolygon")

    def polygon_3d_from_2d(polygon_2d, z):
        z = round(z, 3)

        def to_3d(poly):
            exterior_3d = [(round(x, 3), round(y, 3), z) for x, y in poly.exterior.coords]
            interiors_3d = [
                [(round(x, 3), round(y, 3), z) for x, y in ring.coords]
                for ring in poly.interiors
            ]
            return Polygon(exterior_3d, interiors_3d)

        if isinstance(polygon_2d, Polygon):
            return to_3d(polygon_2d)
        elif isinstance(polygon_2d, MultiPolygon):
            return MultiPolygon([to_3d(p) for p in polygon_2d.geoms])
        else:
            raise TypeError("Expected Polygon or MultiPolygon")

    from shapely.geometry import MultiPolygon, Polygon

    def ensure_multipolygon(geom):
        if isinstance(geom, Polygon):
            return MultiPolygon([geom])
        elif isinstance(geom, MultiPolygon):
            return geom
        else:
            return MultiPolygon()  # fallback

    def round_polygon_coords(poly, precision=3):
        def round_coords(coords):
            return [(round(x, precision), round(y, precision), round(z, precision)) for x, y, z in coords]
        if isinstance(poly, Polygon):
            exterior = round_coords(poly.exterior.coords)
            interiors = [round_coords(interior.coords) for interior in poly.interiors]
            return Polygon(exterior, interiors)
        elif isinstance(poly, MultiPolygon):
            return MultiPolygon([round_polygon_coords(p, precision) for p in poly.geoms])
        else:
            raise TypeError("Expected Polygon or MultiPolygon")

    def polygon_normal(polygon_3d):
        coords = list(polygon_3d.exterior.coords)
        if len(coords) < 3:
            return None
        p0, p1, p2 = np.array(coords[:3])
        v1 = p1 - p0
        v2 = p2 - p0
        normal = np.cross(v1, v2)
        norm = np.linalg.norm(normal)
        return normal / norm if norm != 0 else None

    def generate_floor_ceiling(room_id, room_geom, z_min, z_max, origin, R, polygon_id_start=0):
        floor_data = []
        ceiling_data = []
        polygons = list(room_geom.geoms) if isinstance(room_geom, MultiPolygon) else [room_geom]
        polygon_id = polygon_id_start

        for poly_2d in polygons:
            if not poly_2d.is_valid or poly_2d.area == 0:
                continue

            for surface_type, sign, z in [("floor", -1.0, z_min), ("ceiling", 1.0, z_max)]:
                poly_oriented = orient(poly_2d, sign=sign)
                poly_3d = polygon_3d_from_2d(poly_oriented, z)
                poly_rounded = round_polygon_coords(poly_3d)

                if isinstance(poly_rounded, MultiPolygon):
                    subpolygons = poly_rounded.geoms
                else:
                    subpolygons = [poly_rounded]

                for subpoly in subpolygons:
                    normal = polygon_normal(subpoly) if compute_normals else None

                    surface_data = {
                        "id": polygon_id,
                        "room_id": room_id,
                        "type": surface_type,
                        "wkt": dumps(subpoly, output_dimension=3, rounding_precision=3, trim=True),
                        "local_origin": origin.tolist(),
                        "local_rotation": R.tolist(),
                        "normal": normal.tolist() if normal is not None else None
                    }

                    polygon_id += 1

                    if surface_type == "floor":
                        floor_data.append(surface_data)
                    else:
                        ceiling_data.append(surface_data)


        return floor_data, ceiling_data, polygon_id

    # --- ROOM INPUTS ---
    room_local_origin = {}
    room_local_rotation = {}
    for wall in processed_walls:
        room_id = wall["room_id"]
        if room_id not in room_local_origin:
            room_local_origin[room_id] = wall["local_origin"]
            room_local_rotation[room_id] = wall["local_rotation"]

    all_surfaces = []

    for room_id, room_geom in enumerate(merged_rooms):
        origin = room_local_origin.get(room_id, np.zeros(3))
        R = room_local_rotation.get(room_id, np.eye(3))

        room_shape = unary_union(room_geom)
        if room_shape.is_empty or not room_shape.is_valid or room_shape.area == 0:
            continue
        if room_shape.geom_type == "Polygon":
            room_shape = MultiPolygon([room_shape])

        floor_data, ceiling_data, polygon_id = generate_floor_ceiling(
            room_id, room_shape, z_min, z_max, origin, R, polygon_id
        )
        all_surfaces.extend(floor_data + ceiling_data)

    # --- Add floor and ceiling data to df_export ---
    new_rows = []
    for surface in all_surfaces:
        # Create a new row for floor/ceiling data
        new_rows.append({
            "room_id": surface["room_id"],
            "wall_id": None,
            "opening_id": None,
            "geometry_type": surface["type"],
            "wall_wkt": None,  # No wall WKT for floors and ceilings
            "opening_wkt": None,  # No opening WKT for floors and ceilings
            "has_holes": False,
            "geometry": surface["wkt"]
        })

    # Use pd.concat instead of append
    new_df = pd.DataFrame(new_rows)
    df_export = pd.concat([df_export, new_df], ignore_index=True)

    df_export.head(60)

    from shapely import wkt

    floor_ceiling_merged_rows = []

    for room_id in df_export['room_id'].unique():
        # Floors
        floors = df_export[(df_export['room_id'] == room_id) & (df_export['geometry_type'] == 'floor')]
        if not floors.empty:
            geometries = [wkt.loads(g) for g in floors['geometry']]
            floor_geom = ensure_multipolygon(unary_union(geometries))
            floor_ceiling_merged_rows.append({
                'room_id': room_id,
                'wall_id': None,
                'opening_id': None,
                'geometry_type': 'floor',
                'wall_wkt': None,
                'opening_wkt': None,
                'has_holes': False,
                'geometry': dumps(floor_geom, output_dimension=3, rounding_precision=3, trim=True)
            })

        ceilings = df_export[(df_export['room_id'] == room_id) & (df_export['geometry_type'] == 'ceiling')]
        if not ceilings.empty:
            geometries = [wkt.loads(g) for g in ceilings['geometry']]
            ceiling_geom = ensure_multipolygon(unary_union(geometries))
            floor_ceiling_merged_rows.append({
                'room_id': room_id,
                'wall_id': None,
                'opening_id': None,
                'geometry_type': 'ceiling',
                'wall_wkt': None,
                'opening_wkt': None,
                'has_holes': False,
                'geometry': dumps(ceiling_geom, output_dimension=3, rounding_precision=3, trim=True)
            })

    df_export = df_export[~df_export['geometry_type'].isin(['floor', 'ceiling'])]
    df_export = pd.concat([df_export, pd.DataFrame(floor_ceiling_merged_rows)], ignore_index=True)


In [ ]:
df_export.head(60)

In [ ]:
import pandas as pd

room_ids = df_export['room_id'].unique()

building_hierarchy = {
    'element_id': 0,  # Building is the root element
    'thematic_surface': 'building',
    'geometry': None,  # No geometry for the building itself (optional)
    'parent_id': None,
    'has_holes': False,
    'rooms': []  # List of rooms in the building
}

def generate_element_id(element_type, unique_id):
    return unique_id

element_counter = 1  # Starting ID for the first room/element after the building

flat_data = [] 
wall_ids = {}

def create_room_hierarchy(room_id, room_data):
    global element_counter
    
    room_element_id = generate_element_id('room', element_counter)
    element_counter += 1

    room_hierarchy = {
        'element_id': room_element_id,
        'thematic_surface': 'room',
        'geometry': None,
        'parent_id': building_hierarchy['element_id'],
        'has_holes': False,
        'walls': [],
        'ceilings': [],
        'floors': [],
        'openings': []
    }

    for element in room_data:
        element_id = None
        thematic_surface = None

        if element['geometry_type'] in ['wall', 'wall_with_holes']:
            wall_id = element['wall_id']
            if wall_id not in wall_ids:
                wall_ids[wall_id] = generate_element_id('wall', element_counter)
                element_counter += 1
            element_id = wall_ids[wall_id]
            thematic_surface = 'wall'
            has_holes = element['has_holes']
            matched_walls = element.get('matched_walls')

            room_hierarchy['walls'].append({
                'element_id': element_id,
                'thematic_surface': thematic_surface,
                'geometry': element['geometry'],
                'parent_id': room_element_id,
                'has_holes': has_holes,
                'matched_walls': matched_walls  # ✅ Assign here
            })

        elif element['geometry_type'] == 'floor':
            element_id = generate_element_id('floor', element_counter)
            element_counter += 1
            thematic_surface = 'floor'
            room_hierarchy['floors'].append({
                'element_id': element_id,
                'thematic_surface': thematic_surface,
                'geometry': element['geometry'],
                'parent_id': room_element_id,
                'has_holes': False
            })

        elif element['geometry_type'] == 'ceiling':
            element_id = generate_element_id('ceiling', element_counter)
            element_counter += 1
            thematic_surface = 'ceiling'
            room_hierarchy['ceilings'].append({
                'element_id': element_id,
                'thematic_surface': thematic_surface,
                'geometry': element['geometry'],
                'parent_id': room_element_id,
                'has_holes': False
            })

        elif element['geometry_type'] == 'opening':
            opening_type = element['opening_id'].split('_')[0]
            element_id = generate_element_id('opening', element_counter)
            element_counter += 1
            thematic_surface = f"opening_{opening_type}"
            room_hierarchy['openings'].append({
                'element_id': element_id,
                'thematic_surface': thematic_surface,
                'geometry': element['geometry'],
                'parent_id': wall_ids[element['wall_id']],
                'has_holes': True
            })

        else:
            print(f"Unknown geometry_type for element: {element['geometry_type']}")

    return room_hierarchy



# Loop through each room_id and create room hierarchies
for room_id in room_ids:
    room_data = df_export[df_export['room_id'] == room_id].to_dict(orient='records')

    # Create room hierarchy for each room
    room_hierarchy = create_room_hierarchy(room_id, room_data)

    # Add the room hierarchy to the building
    building_hierarchy['rooms'].append(room_hierarchy)

# Flatten the room hierarchy for easier viewing in a tabular format
for room in building_hierarchy['rooms']:
    for wall in room['walls']:
        flat_data.append({
            'element_id': wall['element_id'],
            'thematic_surface': wall['thematic_surface'],
            'geometry': wall['geometry'],
            'parent_id': wall['parent_id'],
            'has_holes': wall['has_holes'],
            'matched_walls': wall.get('matched_walls')  # ✅ Use .get to avoid KeyError
        })

    for ceiling in room['ceilings']:
        flat_data.append({
            'element_id': ceiling['element_id'],
            'thematic_surface': ceiling['thematic_surface'],
            'geometry': ceiling['geometry'],
            'parent_id': ceiling['parent_id'],
            'has_holes': ceiling['has_holes']
        })
    for floor in room['floors']:
        flat_data.append({
            'element_id': floor['element_id'],
            'thematic_surface': floor['thematic_surface'],
            'geometry': floor['geometry'],
            'parent_id': floor['parent_id'],
            'has_holes': floor['has_holes']
        })
    for opening in room['openings']:
        flat_data.append({
            'element_id': opening['element_id'],
            'thematic_surface': opening['thematic_surface'],
            'geometry': opening['geometry'],
            'parent_id': opening['parent_id'],
            'has_holes': opening['has_holes']
        })


df_elements_hierarchy = pd.DataFrame(flat_data)
df_elements_hierarchy.head(50)


In [ ]:
df_elements_hierarchy.head(150)

In [ ]:
add_floors_ceilings_to_df = True

if add_floors_ceilings_to_df:
        
    import pandas as pd
    import uuid

    element_uuid_map = {}

    def get_stable_uuid(element_id):
        return element_uuid_map.setdefault(element_id, str(uuid.uuid4()))

    transformed_data = []

    building_entry = {
        'element_id': building_hierarchy['element_id'],
        'type': 'building',
        'parent_id': None,
        'geometry': None,
        'id': get_stable_uuid(building_hierarchy['element_id']),
        'has_holes': None
    }
    transformed_data.append(building_entry)

    for room in building_hierarchy['rooms']:

        room_entry = {
            'element_id': room['element_id'],
            'type': 'room',
            'parent_id': building_hierarchy['element_id'],
            'geometry': None,
            'id': get_stable_uuid(room['element_id']),
            'has_holes': None
        }
        transformed_data.append(room_entry)

        for wall in room['walls']:
            wall_entry = {
                'element_id': wall['element_id'],
                'type': 'wall',
                'parent_id': room['element_id'],
                'geometry': wall['geometry'],
                'id': get_stable_uuid(wall['element_id']),
                'has_holes': wall['has_holes'],
                'matched_walls': wall.get('matched_walls')
            }
            transformed_data.append(wall_entry)

        for ceiling in room['ceilings']:
            ceiling_entry = {
                'element_id': ceiling['element_id'],
                'type': 'ceiling',
                'parent_id': room['element_id'],
                'geometry': ceiling['geometry'],
                'id': get_stable_uuid(ceiling['element_id']),
                'has_holes': ceiling['has_holes']
            }
            transformed_data.append(ceiling_entry)

        for floor in room['floors']:
            floor_entry = {
                'element_id': floor['element_id'],
                'type': 'floor',
                'parent_id': room['element_id'],
                'geometry': floor['geometry'],
                'id': get_stable_uuid(floor['element_id']),
                'has_holes': floor['has_holes']
            }
            transformed_data.append(floor_entry)

        for opening in room['openings']:
            opening_entry = {
                'element_id': opening['element_id'],
                'type': opening['thematic_surface'].replace('opening_', ''),
                'parent_id': opening['parent_id'],
                'geometry': opening['geometry'],
                'id': get_stable_uuid(opening['element_id']),
                'has_holes': opening['has_holes']
            }
            transformed_data.append(opening_entry)

    df_transformed = pd.DataFrame(transformed_data)

    if 'matched_walls' not in df_transformed.columns:
        df_transformed['matched_walls'] = None

    # df_transformed = df_transformed[['element_id', 'type', 'parent_id', 'geometry', 'id', 'has_holes', 'matched_walls']]
    df_transformed.head(80)


In [ ]:
df_transformed.head(50)

In [ ]:
import pandas as pd
import uuid

element_uuid_map = {}

def get_stable_uuid(element_id):
    return element_uuid_map.setdefault(element_id, str(uuid.uuid4()))

transformed_data = []

transformed_data.append({
    'element_id': building_hierarchy['element_id'],
    'type': 'building',
    'parent_id': None,
    'geometry': None,
    'id': get_stable_uuid(building_hierarchy['element_id']),
    'has_holes': None
})

for room in building_hierarchy['rooms']:
    transformed_data.append({
        'element_id': room['element_id'],
        'type': 'room',
        'parent_id': building_hierarchy['element_id'],
        'geometry': None,
        'id': get_stable_uuid(room['element_id']),
        'has_holes': None
    })

    for wall in room['walls']:
        transformed_data.append({
            'element_id': wall['element_id'],
            'type': 'wall',
            'parent_id': room['element_id'],
            'geometry': wall['geometry'],
            'id': get_stable_uuid(wall['element_id']),
            'has_holes': wall['has_holes'],
            'matched_walls': wall.get('matched_walls')
        })

    for ceiling in room['ceilings']:
        transformed_data.append({
            'element_id': ceiling['element_id'],
            'type': 'ceiling',
            'parent_id': room['element_id'],
            'geometry': ceiling['geometry'],
            'id': get_stable_uuid(ceiling['element_id']),
            'has_holes': ceiling['has_holes']
        })

    for floor in room['floors']:
        transformed_data.append({
            'element_id': floor['element_id'],
            'type': 'floor',
            'parent_id': room['element_id'],
            'geometry': floor['geometry'],
            'id': get_stable_uuid(floor['element_id']),
            'has_holes': floor['has_holes']
        })

    for opening in room['openings']:
        transformed_data.append({
            'element_id': opening['element_id'],
            'type': opening['thematic_surface'].replace('opening_', ''),
            'parent_id': opening['parent_id'],
            'geometry': opening['geometry'],
            'id': get_stable_uuid(opening['element_id']),
            'has_holes': opening['has_holes']
        })

df_transformed = pd.DataFrame(transformed_data)
if 'matched_walls' not in df_transformed.columns:
    df_transformed['matched_walls'] = None
df_transformed = df_transformed[['element_id', 'type', 'parent_id', 'geometry', 'id', 'has_holes', 'matched_walls']]

df_transformed.head(80)



In [ ]:
df_transformed.head(60)

In [ ]:
cut_both_side_walls = True

if cut_both_side_walls:

    import numpy as np
    from shapely.geometry import Polygon, MultiPolygon, LinearRing
    from shapely.validation import explain_validity
    from shapely.geometry.polygon import orient

    walls_with_openings = df_transformed[(df_transformed["type"] == "wall") & (df_transformed["has_holes"] == True)]
    geometries_with_openings = walls_with_openings["geometry"]

    walls_with_openings = df_transformed[
        (df_transformed["type"] == "wall") & (df_transformed["has_holes"] == True)
    ]
    geometries_with_openings = walls_with_openings["geometry"]

    # --- Geometry Utilities ---
    def coords_to_array(coords):
        return np.array(coords)

    def close_ring(coords, tolerance=1e-17):
        if len(coords) < 2:
            return coords
        if not np.allclose(coords[0], coords[-1], atol=tolerance):
            coords = np.vstack([coords, coords[0]])
        return coords

    def build_local_basis(p0, p1, p2, flip_normal=False):
        u = p1 - p0
        u = u / np.linalg.norm(u)
        n = np.cross(u, p2 - p0)
        n = n / np.linalg.norm(n)
        if flip_normal:
            n = -n
        v = np.cross(n, u)
        return u, v, n

    def project_to_2d_ring(ring_3d, origin, u, v):
        return [(np.dot(pt - origin, u), np.dot(pt - origin, v)) for pt in ring_3d]

    def reproject_to_3d_ring(ring_2d, origin, u, v):
        return [(origin + x * u + y * v).tolist() for x, y in ring_2d]

    def is_valid_ring(ring, min_area=1e-8):
        if len(ring) < 4:
            return False
        try:
            lr = LinearRing(ring)
            return lr.is_simple and lr.is_ring and Polygon(lr).area > min_area
        except:
            return False

    def buffer_ring_inward(ring, amount=1e-4):
        poly = Polygon(ring)
        buffered = poly.buffer(-amount)
        if buffered.is_empty or not buffered.is_valid or not isinstance(buffered, Polygon):
            return ring
        return list(buffered.exterior.coords)

    def fit_best_plane(points):
        """Return (origin, normal) of best-fit plane to a set of 3D points."""
        centroid = np.mean(points, axis=0)
        shifted = points - centroid
        _, _, vh = np.linalg.svd(shifted)
        normal = vh[-1]
        return centroid, normal

    def fit_best_plane(points):
        """Return (origin, normal) of best-fit plane to a set of 3D points."""
        centroid = np.mean(points, axis=0)
        shifted = points - centroid
        _, _, vh = np.linalg.svd(shifted)
        normal = vh[-1]
        return centroid, normal

    def build_basis_from_normal(normal):
        """Return (u, v, n) basis given a normal vector."""
        normal = normal / np.linalg.norm(normal)
        arbitrary = np.array([1, 0, 0]) if abs(normal[0]) < 0.9 else np.array([0, 1, 0])
        u = np.cross(normal, arbitrary)
        u /= np.linalg.norm(u)
        v = np.cross(normal, u)
        return u, v, normal

    cleaned_polygons = []

    for geom_idx, geom in enumerate(geometries_with_openings):
        polygons = [geom] if isinstance(geom, Polygon) else geom.geoms if isinstance(geom, MultiPolygon) else []

        for poly_idx, poly in enumerate(polygons):
            try:
                exterior_3d = close_ring(coords_to_array(poly.exterior.coords))
                if len(exterior_3d) < 4:
                    print(f"❌ Wall {geom_idx}-{poly_idx}: exterior has too few coordinates.")
                    continue

                print(f"🔍 Wall {geom_idx}-{poly_idx} raw exterior points: {len(exterior_3d)}")

                centroid, normal = fit_best_plane(exterior_3d)
                u, v, n = build_basis_from_normal(-normal)  # flip normal

                p0 = centroid
                exterior_2d = project_to_2d_ring(exterior_3d, p0, u, v)

                if not is_valid_ring(exterior_2d):
                    print(f"❌ Wall {geom_idx}-{poly_idx}: invalid exterior (area: {Polygon(exterior_2d).area:.2e})")
                    continue

                holes_2d = []
                for i, ring in enumerate(poly.interiors):
                    hole_3d = close_ring(coords_to_array(ring.coords))
                    hole_2d = project_to_2d_ring(hole_3d, p0, u, v)

                    if len(hole_2d) < 4 or not is_valid_ring(hole_2d):
                        print(f"⚠️ Hole {i} in Wall {geom_idx}-{poly_idx}: invalid or too few points.")
                        continue

                    shell_poly = Polygon(exterior_2d)
                    hole_poly = Polygon(hole_2d)

                    if not shell_poly.contains(hole_poly):
                        buffered = buffer_ring_inward(hole_2d)
                        if not shell_poly.contains(Polygon(buffered)):
                            print(f"⚠️ Hole {i} not strictly inside shell — forcing inclusion due to near match.")
                        else:
                            print(f"✅ Hole {i} accepted via inward buffering.")
                            hole_2d = buffered

                    holes_2d.append(hole_2d)

                # ✅ Enforce reversed winding: CW shell, CCW holes
                exterior_2d_oriented = orient(Polygon(exterior_2d), sign=1.0).exterior.coords[:-1]
                oriented_holes = [list(orient(Polygon(h), sign=1.0).exterior.coords[:-1]) for h in holes_2d]

                poly_2d = Polygon(shell=exterior_2d_oriented, holes=oriented_holes)

                if not poly_2d.is_valid:
                    reason = explain_validity(poly_2d)
                    print(f"⚠️ Wall {geom_idx}-{poly_idx} invalid: {reason}")
                    poly_2d = poly_2d.buffer(0)
                    if not poly_2d.is_valid or poly_2d.is_empty:
                        print(f"❌ Wall {geom_idx}-{poly_idx} not fixable — skipping.")
                        continue
                    else:
                        print(f"✅ Wall {geom_idx}-{poly_idx} fixed with buffer(0).")

                # ✅ Reproject back to 3D
                exterior_3d_final = reproject_to_3d_ring(poly_2d.exterior.coords, p0, u, v)
                holes_3d_final = [reproject_to_3d_ring(hole.coords, p0, u, v) for hole in poly_2d.interiors]
                poly_3d = Polygon(shell=exterior_3d_final, holes=holes_3d_final)

                print(f"✅ Wall {geom_idx}-{poly_idx} cleaned, reoriented, and valid.")
                cleaned_polygons.append((walls_with_openings.index[geom_idx], poly_3d))

            except Exception as e:
                print(f"❌ Wall {geom_idx}-{poly_idx} error: {e}")

    # --- Assign to DataFrame ---
    valid_indices = []
    valid_geometries = []

    for idx, poly in cleaned_polygons:
        if poly and not poly.is_empty:
            wrapped = MultiPolygon([poly]) if isinstance(poly, Polygon) else poly
            valid_indices.append(idx)
            valid_geometries.append(wrapped)
        else:
            print(f"⚠️ Skipping index {idx}: geometry is EMPTY or None.")

    df_transformed.loc[valid_indices, "snapped_geometry"] = valid_geometries
    df_transformed["snapped_geometry"] = df_transformed["snapped_geometry"].fillna(df_transformed["geometry"])

    print(f"\n🎯 Final result: {len(valid_geometries)} valid 3D wall geometries added to 'snapped_geometry'.")


In [ ]:
from shapely.geometry import MultiPolygon

df_transformed.loc[valid_indices, "geometry"] = valid_geometries
df_transformed["geometry"] = df_transformed["geometry"].fillna(df_transformed["snapped_geometry"])
# print(df_transformed[['csv_line_id', 'geometry']].head())

df_transformed.drop(columns='snapped_geometry', inplace=True)

df_transformed = df_transformed.rename(columns={
    'csv_line_id': 'csv_line_id',
    'element_id': 'element_id',
    'type': 'thematic_surface',
    'parent_id': 'parent_id',
    'geometry': 'geometry_wkt',
    'id': 'element_gml_id',
    'has_holes': 'has_holes',
    'matched_walls': 'related_to'
})

df_transformed.head(40)


In [ ]:
## WALLS CHECKING
import pandas as pd
from shapely import wkt
from shapely.geometry import Polygon, MultiPolygon, LineString
import numpy as np
from shapely.geometry import shape

def extract_base_segment(geom):
    
    if isinstance(geom, MultiPolygon):
        polygons = list(geom.geoms)
    elif isinstance(geom, Polygon):
        polygons = [geom]
    else:
        return None

    base_segments = []

    for poly in polygons:
        exterior = np.array(poly.exterior.coords)
        if exterior.shape[1] < 3:
            continue  # Not 3D
        min_z = np.min(exterior[:, 2])
        base_ring = np.array([pt[:2] for pt in exterior if np.isclose(pt[2], min_z, atol=1e-6)])

        if len(base_ring) >= 2:
            base_segments.append(LineString(base_ring))

    if len(base_segments) == 1:
        return base_segments[0]
    elif base_segments:
        return base_segments  # return list of LineStrings if more than one
    else:
        return None

def safe_load_wkt(val):
    if isinstance(val, str):
        return wkt.loads(val)
    return val  # already a geometry

walls_df = df_transformed[df_transformed["thematic_surface"] == "wall"].copy()
walls_df["geometry"] = walls_df["geometry_wkt"].apply(safe_load_wkt)
walls_df["base_2d"] = walls_df["geometry"].apply(extract_base_segment)

print(f'n. of wall_2d {len(walls_df["base_2d"])}\n{walls_df["base_2d"]}' )


ADD VALIDATION WALLS

In [ ]:
dcel_structure_networx = True

if dcel_structure_networx:

    import networkx as nx
    from shapely.geometry import LineString, Polygon, MultiPolygon, box
    from shapely.ops import polygonize_full, unary_union
    from collections import defaultdict
    import plotly.graph_objects as go
    import math
    import random

    # --- CONFIG ---
    l1 = 0.9
    l2 = 0.9
    min_area = l1 * l2
    snap_precision = 9

    # --- HELPERS ---
    def snap_coords(pt, precision=snap_precision):
        return (round(pt[0], precision), round(pt[1], precision))

    def compute_angle(p0, p1):
        dx, dy = p1[0] - p0[0], p1[1] - p0[1]
        angle = math.atan2(dy, dx)
        return angle if angle >= 0 else angle + 2 * math.pi

    # --- ROOM POLYGONIZATION ---
    def build_wall_intersections_graph(segments):
        G = nx.Graph()
        for seg_data in segments:
            seg = seg_data.get("segment")
            if seg and isinstance(seg, LineString):
                coords = list(seg.coords)
                if len(coords) == 2:
                    p1, p2 = snap_coords(coords[0]), snap_coords(coords[1])
                    G.add_edge(p1, p2, weight=seg.length)
        return G

    def find_rooms_from_graph(G):
        edge_lines = [LineString([u, v]) for u, v in G.edges if u != v]
        polygons_gc, *_ = polygonize_full(edge_lines)
        return [p for p in polygons_gc.geoms if p.is_valid and p.area > 0.01]

    def merge_small_rooms(rooms, min_area):
        big, small = [], []
        for r in rooms:
            (big if r.area >= min_area else small).append(r)
        for s in small:
            nearest = min(big, key=lambda b: s.centroid.distance(b.centroid))
            merged = unary_union([s, nearest])
            if merged.is_valid and not merged.is_empty:
                big.remove(nearest)
                big.append(merged)
        return big

    def extract_wall_segments_from_rooms(rooms):
        walls = []
        for room_idx, room in enumerate(rooms):
            polys = list(room.geoms) if isinstance(room, MultiPolygon) else [room]
            for poly in polys:
                coords = list(poly.exterior.coords)
                for i in range(len(coords) - 1):
                    p1, p2 = coords[i], coords[i + 1]
                    line = LineString([p1, p2])
                    walls.append({"segment": line, "room_idx": room_idx})
        return walls


    def build_dcel_from_segments(segments, closure_segments_list):
        vertices, point_to_vid, half_edges = {}, {}, {}
        wall_info = {}
        vertex_id, edge_id, wall_id = 0, 0, 0
        closure_set = set()

        # Step 1: Add closure segments to the input segment list
        for closure in closure_segments_list:
            seg = closure.get("segment")
            if isinstance(seg, LineString):
                segments.append({
                    "segment": seg,
                    "is_closure": True,
                    "room_idx": None,
                    "from_box": False
                })
                # Also add to closure_set (for edge comparison)
                coords = list(seg.coords)
                for i in range(len(coords) - 1):
                    start, end = snap_coords(coords[i]), snap_coords(coords[i + 1])
                    closure_set.add(frozenset([start, end]))

        # Step 2: Build DCEL from all segments
        for conn in segments:
            coords = list(conn["segment"].coords)
            is_bbox = conn.get("from_box", False)
            room_id = conn.get("room_idx")
            for i in range(len(coords) - 1):
                start, end = snap_coords(coords[i]), snap_coords(coords[i + 1])
                for pt in (start, end):
                    if pt not in point_to_vid:
                        point_to_vid[pt] = vertex_id
                        vertices[vertex_id] = {"coordinates": pt, "incident_edge": None}
                        vertex_id += 1

                sid, eid_ = point_to_vid[start], point_to_vid[end]
                he1, he2 = edge_id, edge_id + 1

                is_closure = frozenset([start, end]) in closure_set or conn.get("is_closure", False)

                half_edges[he1] = {
                    "origin": sid, "twin": he2, "next": None, "prev": None,
                    "incident_face": None, "is_closure": is_closure, "from_box": is_bbox
                }
                half_edges[he2] = {
                    "origin": eid_, "twin": he1, "next": None, "prev": None,
                    "incident_face": None, "is_closure": is_closure, "from_box": is_bbox
                }

                if vertices[sid]["incident_edge"] is None:
                    vertices[sid]["incident_edge"] = he1
                if vertices[eid_]["incident_edge"] is None:
                    vertices[eid_]["incident_edge"] = he2

                origin_coords = vertices[sid]["coordinates"]
                dest_coords = vertices[eid_]["coordinates"]
                line_geom = LineString([origin_coords, dest_coords])

                wall_info[wall_id] = {
                    "wall_id": wall_id,
                    "eid_1": he1,
                    "eid_2": he2,
                    "room_id": room_id,
                    "wall_type": None,
                    "segment": line_geom  # ✅ Store geometry for lookup
                }

                half_edges[he1]["wall_id"] = wall_id
                half_edges[he2]["wall_id"] = wall_id

                wall_id += 1
                edge_id += 2

        return vertices, half_edges, wall_info

    # --- FACE ASSIGNMENT AND WALL CLASSIFICATION ---
    def assign_faces_and_classify_walls(rooms, vertices, half_edges, wall_info):
        edge_lines = {
            (vertices[e['origin']]['coordinates'], vertices[half_edges[e['twin']]['origin']]['coordinates']): eid
            for eid, e in half_edges.items()
        }
        faces, used = {}, set()
        face_id = 0
        room_face_map = {}

        for room_idx, room in enumerate(rooms):
            polys = list(room.geoms) if isinstance(room, MultiPolygon) else [room]
            for poly in polys:
                coords = list(poly.exterior.coords)
                loop = []
                for i in range(len(coords) - 1):
                    p1, p2 = tuple(coords[i]), tuple(coords[i + 1])
                    key = (p1, p2)
                    rev_key = (p2, p1)
                    eid = edge_lines.get(key) or edge_lines.get(rev_key)
                    if eid is not None:
                        loop.append(eid)
                        used.add(eid)
                if len(loop) >= 3:
                    for eid in loop:
                        half_edges[eid]["incident_face"] = face_id
                    faces[face_id] = {"outer_component": loop[0]}
                    room_face_map[face_id] = room_idx
                    face_id += 1

        # Classification loop
        for eid, edge in half_edges.items():
            twin = half_edges[edge["twin"]]
            f1 = edge.get("incident_face")
            f2 = twin.get("incident_face")
            related_faces = set()
            # wall_type = "ambiguous"

            if f1 is not None:
                related_faces.add(f1)
            if f2 is not None:
                related_faces.add(f2)

            # if edge.get("is_closure"):
            #     wall_type = "closure"
            # elif len(related_faces) == 2:
            #     wall_type = "internal"
            # elif len(related_faces) == 1:
            #     wall_type = "external"
            if edge.get("is_closure"):
                wall_type = "closure"
            else:
                if len(related_faces) == 2:
                    fids = list(related_faces)
                    r1 = room_face_map.get(fids[0])
                    r2 = room_face_map.get(fids[1])
                    wall_type = "internal" if r1 == r2 else "party_wall"
                elif len(related_faces) == 1:
                    wall_type = "external"
                else:
                    wall_type = "ambiguous"

            fids = list(related_faces)
            if len(fids) == 2:
                r1 = room_face_map.get(fids[0])
                r2 = room_face_map.get(fids[1])
                if r1 != r2:
                    wall_type = "party_wall"

            edge["wall_type"] = wall_type
            twin["wall_type"] = wall_type

            wall_info[edge["wall_id"]]["wall_type"] = wall_type
            wall_info[edge["wall_id"]]["related_faces"] = list(related_faces)

        # ✅ FINAL SWEEP to ensure all half-edges reflect updated wall_info
        for eid, edge in half_edges.items():
            wall_id = edge.get("wall_id")
            if wall_id is not None and wall_id in wall_info:
                edge["wall_type"] = wall_info[wall_id]["wall_type"]

        wall_dictionary = {
            wid: info for wid, info in wall_info.items()
            if info["wall_type"] != "ambiguous"
        }

        return faces, wall_dictionary, wall_info
    
    def normalize_segment_key(seg, precision=6):
        coords = [tuple(round(c, precision) for c in pt) for pt in seg.coords]
        return tuple(sorted(coords))  # Order-independent


    # # --- MAIN EXECUTION ---
    # def full_dcel_pipeline(connection_segments_defined, closure_segments_list):

    #     G = build_wall_intersections_graph(connection_segments_defined)
    #     rooms = find_rooms_from_graph(G)
    #     merged_rooms = merge_small_rooms(rooms, min_area)
    #     new_segments = extract_wall_segments_from_rooms(merged_rooms)
    #     vertices, half_edges, wall_info = build_dcel_from_segments(new_segments, closure_segments_list)
    #     faces, _, wall_info = assign_faces_and_classify_walls(merged_rooms, vertices, half_edges, wall_info)
    #     filtered_wall_info = {wid: info for wid, info in wall_info.items() if info["wall_type"] != "ambiguous"}
        
    #     return vertices, half_edges, faces, merged_rooms, filtered_wall_info

    def full_dcel_pipeline(connection_segments_defined, closure_segments_list):

        G = build_wall_intersections_graph(connection_segments_defined)
        rooms = find_rooms_from_graph(G)
        merged_rooms = merge_small_rooms(rooms, min_area)
        new_segments = extract_wall_segments_from_rooms(merged_rooms)
        vertices, half_edges, wall_info = build_dcel_from_segments(new_segments, closure_segments_list)
        faces, _, wall_info = assign_faces_and_classify_walls(merged_rooms, vertices, half_edges, wall_info)

        # Remove ambiguous walls
        # filtered_wall_info = {wid: info for wid, info in wall_info.items() if info["wall_type"] != "ambiguous"}
        filtered_wall_info = {
            wid: info for wid, info in wall_info.items()
            if info["wall_type"] != "ambiguous" or info["wall_type"] == "closure"
        }

        # --- Correlate connection_segments with wall_info ---
        def normalize_segment_key(seg, precision = 9):
            coords = [tuple(round(c, precision) for c in pt) for pt in seg.coords]
            return tuple(sorted(coords))

        wall_segment_lookup = {}
        
        for wid, info in filtered_wall_info.items():
            he1 = info["eid_1"]
            he2 = info["eid_2"]
            origin = vertices[half_edges[he1]["origin"]]["coordinates"]
            dest = vertices[half_edges[he2]["origin"]]["coordinates"]
            seg = LineString([origin, dest])
            key = normalize_segment_key(seg)
            wall_segment_lookup[key] = wid

        for conn in connection_segments_defined:
            seg = conn["segment"]
            key = normalize_segment_key(seg)
            conn["matched_wall_id"] = wall_segment_lookup.get(key)

        return vertices, half_edges, faces, merged_rooms, filtered_wall_info
    
    # --- DCEL VISUALIZATION ---
    def plot_dcel_faces(vertices, half_edges, faces, segments):
        fig = go.Figure()
        def get_face_coords(start_eid):
            coords, visited = [], set()
            current = start_eid
            while current not in visited and current is not None:
                visited.add(current)
                coords.append(vertices[half_edges[current]['origin']]['coordinates'])
                current = half_edges[current]['next']
            return coords

        for conn in segments:
            x, y = zip(*conn['segment'].coords)
            fig.add_trace(go.Scatter3d(x=list(x), y=list(y), z=[0]*len(x), mode="lines", line=dict(color='lightgray'), showlegend=False))

        for fid, fdata in faces.items():
            coords = get_face_coords(fdata['outer_component'])
            if len(coords) < 3: continue
            x, y = zip(*coords)
            z = [0.01] * len(x)
            fig.add_trace(go.Mesh3d(x=x, y=y, z=z, i=[0]*(len(x)-2), j=list(range(1,len(x)-1)), k=list(range(2,len(x))), opacity=0.6))

        wall_colors = {
            "internal": "green",
            "external": "red",
            "closure": "blue",
            "party_wall": "orange",  # or any other color you like
            "ambiguous": "black"
        }

        for eid, edge in half_edges.items():
            origin = vertices[edge["origin"]]["coordinates"]
            dest = vertices[half_edges[edge["twin"]]["origin"]]["coordinates"]
            color = wall_colors.get(edge.get("wall_type", "ambiguous"), "gray")
            fig.add_trace(go.Scatter3d(x=[origin[0], dest[0]], y=[origin[1], dest[1]], z=[0.02, 0.02], mode="lines", line=dict(width=4, color=color), showlegend=False))

        fig.update_layout(scene=dict(aspectmode="data"))
        fig.show()
  

In [ ]:
from shapely.geometry import LineString

all_wall_segments = []

for idx, base in walls_df["base_2d"].dropna().items():
    if isinstance(base, list):
        segments = base
    elif isinstance(base, LineString):
        coords = list(base.coords)
        segments = [LineString([coords[i], coords[i + 1]]) for i in range(len(coords) - 1)]
    else:
        continue  # skip unknown types

    for seg in segments:
        all_wall_segments.append({"segment": seg, "wall_idx": idx})

def normalize_segment_key(seg, precision=9):
    coords = [tuple(round(c, precision) for c in pt) for pt in seg.coords]
    return tuple(sorted(coords))

closure_keys = set(
    normalize_segment_key(conn["segment"]) for conn in closure_segments_list
)

for seg_data in all_wall_segments:

    seg = seg_data["segment"]
    key = normalize_segment_key(seg)
    if key in closure_keys:
        seg_data["is_closure"] = True

G = build_wall_intersections_graph(all_wall_segments)
rooms = find_rooms_from_graph(G)
merged_rooms = merge_small_rooms(rooms, min_area)

new_wall_segments = extract_wall_segments_from_rooms(merged_rooms)

l1 = 0.9
l2 = 0.9
min_area = l1 * l2
snap_precision = 9

for idx, base in walls_df["base_2d"].dropna().items():

    if isinstance(base, list):
        segments = base
    elif isinstance(base, LineString):
        coords = list(base.coords)
        segments = [LineString([coords[i], coords[i + 1]]) for i in range(len(coords) - 1)]
    else:
        continue

    for seg in segments:
        snapped_coords = [snap_coords(pt) for pt in seg.coords]
        snapped_seg = LineString(snapped_coords)
        all_wall_segments.append({"segment": snapped_seg, "wall_idx": idx})

vertices, half_edges, faces, merged_rooms, wall_dictionary  = full_dcel_pipeline(all_wall_segments, closure_segments_list)
check_dcel_validity(vertices, half_edges)
plot_dcel_faces(vertices, half_edges, faces, connection_segments_defined)
print(f"🏠 Rooms detected: {len(rooms)}")

for fid, face in faces.items():
    print(f"Face {fid} -> outer_edge: {face['outer_component']}")

for wid, info in wall_dictionary.items():  # ✅ Correct
    print(f"Wall {wid}: faces = {info.get('related_faces')}, type = {info['wall_type']}")

for wid, info in wall_dictionary.items():
    if len(info.get("related_faces", [])) > 1:
        print(f"✅ Shared Wall {wid}: faces = {info['related_faces']} -> type = {info['wall_type']}")
        

TYPES OF WALLS

In [ ]:
from shapely import wkt
from shapely.geometry import MultiPolygon, Polygon, LineString

def normalize_segment_key(seg, precision=9):
    coords = [tuple(round(c, precision) for c in pt[:2]) for pt in seg.coords]
    return tuple(sorted(coords))

def extract_edges_from_geometry(geom, precision=9):
    keys = []
    if isinstance(geom, (MultiPolygon, Polygon)):
        polys = geom.geoms if isinstance(geom, MultiPolygon) else [geom]
        for poly in polys:
            coords = list(poly.exterior.coords)
            for i in range(len(coords) - 1):
                seg = LineString([coords[i], coords[i + 1]])
                keys.append(normalize_segment_key(seg, precision))
    return keys

wall_lookup = {
    normalize_segment_key(info["segment"]): info["wall_type"]
    for info in wall_dictionary.values()
}

for idx, row in df_transformed[df_transformed["thematic_surface"] == "wall"].iterrows():
    try:
        geom_wkt = row["geometry_wkt"]
        geom = wkt.loads(geom_wkt) if isinstance(geom_wkt, str) else geom_wkt
        keys = extract_edges_from_geometry(geom)

        wall_type = None
        for key in keys:
            if key in wall_lookup:
                wall_type = wall_lookup[key]
                break

        if wall_type:
            if row.get("has_holes", False) and wall_type in ["external", "partly_wall"]:
                wall_type += "_with_hole"
            df_transformed.at[idx, "thematic_surface"] = wall_type

    except Exception as e:
        print(f"⚠️ Error at index {idx}: {e}")


In [ ]:
closure_surfaces_and_thematic = True    

if closure_surfaces_and_thematic:

    import pandas as pd
    import numpy as np
    import shapely.wkt
    from shapely.geometry import Polygon

    # ✅ Step 1: Build room DataFrame from merged_rooms
    rooms_df = pd.DataFrame({
        "element_id": [f"room_{i}" for i in range(len(merged_rooms))],
        "geometry": merged_rooms
    })
    rooms_df["centroid"] = rooms_df["geometry"].apply(lambda g: g.centroid)

    # ✅ Step 2: Build closure DataFrame from closure_segments_list
    closure_rows = []
    # next_id = df_transformed["element_id"].max() + 1
    numeric_ids = pd.to_numeric(df_transformed["element_id"], errors="coerce")
    next_id = int(numeric_ids.dropna().max()) + 1

    if 'csv_line_id' not in df_transformed.columns:
        df_transformed.insert(0, 'csv_line_id', range(1, len(df_transformed) + 1))

    next_csv_id = df_transformed["csv_line_id"].max() + 1

    for i, closure in enumerate(closure_segments_list):
        seg = closure["segment"]
        (x1, y1), (x2, y2) = seg.coords

        # Perpendicular offset for wall thickness
        dx, dy = y2 - y1, x1 - x2
        length = (dx**2 + dy**2)**0.5 or 1e-6
        dx, dy = dx / length * 0.01, dy / length * 0.01

        p1 = (x1 - dx, y1 - dy, z_min)
        p2 = (x2 - dx, y2 - dy, z_min)
        p3 = (x2 - dx, y2 - dy, z_max)
        p4 = (x1 - dx, y1 - dy, z_max)
        polygon = Polygon([p1, p2, p3, p4, p1])
        centroid = polygon.centroid

        closure_rows.append({
            "csv_line_id": next_csv_id + i,
            "element_id": next_id + i,
            "thematic_surface": "closure",
            "parent_id": None,  # will be set next
            "geometry_wkt": polygon.wkt,
            "element_gml_id": f"closure-wall-{i}",
            "has_holes": False,
            "geometry": polygon,
            "centroid": centroid
        })

    closure_df = pd.DataFrame(closure_rows)

    # ✅ Step 3: Assign closest room as parent_id
    room_centroids = np.array([[c.x, c.y] for c in rooms_df["centroid"]])
    room_ids = rooms_df["element_id"].values

    def find_closest_room_id(centroid):
        xy = np.array([centroid.x, centroid.y])
        dists = np.linalg.norm(room_centroids - xy, axis=1)
        return room_ids[np.argmin(dists)]

    closure_df["parent_id"] = closure_df["centroid"].apply(find_closest_room_id)

    # ✅ Step 4: Clean up temp columns
    closure_df = closure_df.drop(columns=["geometry", "centroid"])

    # ✅ Step 5: Append closures to df_transformed
    df_transformed = pd.concat([df_transformed, closure_df], ignore_index=True)

    print("✅ Closure surfaces added and linked to nearest rooms.")
    print(closure_df[["element_id", "parent_id", "element_gml_id"]])


In [ ]:
df_transformed.head(80)

In [ ]:
walls_to_plot = df_transformed[df_transformed["thematic_surface"].isin(["external", "party_wall", "closure"])]
print(f"🧱 Walls to plot: {len(walls_to_plot)}")

import shapely.wkt
from shapely.geometry import MultiPolygon, Polygon
import plotly.graph_objects as go

def plot_walls_3d(df_subset):
    fig = go.Figure()

    color_map = {
        "external": "red",
        "party_wall": "orange",
        "closure": "blue",
        'wall': 'green'
    }

    for idx, row in df_subset.iterrows():
        geom_raw = row["geometry_wkt"]
        surface_type = row["thematic_surface"]
        color = color_map.get(surface_type, "gray")

        # Parse geometry only if needed
        if isinstance(geom_raw, str):
            geom = shapely.wkt.loads(geom_raw)
        else:
            geom = geom_raw

        # Handle both Polygon and MultiPolygon
        polys = list(geom.geoms) if isinstance(geom, MultiPolygon) else [geom]

        for poly in polys:
            coords = list(poly.exterior.coords)
            if len(coords[0]) < 3:
                continue  # skip if no Z
            x, y, z = zip(*coords)

            fig.add_trace(go.Mesh3d(
                x=x, y=y, z=z,
                i=[0] * (len(x)-2),
                j=list(range(1, len(x)-1)),
                k=list(range(2, len(x))),
                opacity=0.6,
                color=color,
                name=surface_type,
                showscale=False
            ))

    fig.update_layout(
        scene=dict(aspectmode="data"),
        title="🧱 3D Wall Types: External (Red), Party Wall (Orange), Closure (Blue)",
        showlegend=True
    )
    fig.show()

plot_walls_3d(walls_to_plot)



In [ ]:
df_transformed.head(90)

In [ ]:
def adjust_wall_labels_with_holes(df):
    def update_label(row):
        if row["has_holes"] == True:
            if row["thematic_surface"] == "external":
                return "external_with_hole"
            elif row["thematic_surface"] == "party_wall":
                return "party_wall_with_hole"
        return row["thematic_surface"]
    
    df["thematic_surface"] = df.apply(update_label, axis=1)
    return df

df_transformed = adjust_wall_labels_with_holes(df_transformed)
df_transformed.head(60)


CHECK EXTERNAL_WALLS_WITH_OPENINGS NORMALS

In [ ]:
import shapely.wkt
from shapely.geometry import MultiPolygon, Polygon
import numpy as np
import pandas as pd
import plotly.graph_objects as go

def compute_normal(polygon):
    coords = list(polygon.exterior.coords)
    if len(coords) < 3:
        return None
    p1, p2, p3 = np.array(coords[0]), np.array(coords[1]), np.array(coords[2])
    if len(p1) == 2: p1 = np.append(p1, 0.0)
    if len(p2) == 2: p2 = np.append(p2, 0.0)
    if len(p3) == 2: p3 = np.append(p3, 0.0)
    v1 = p2 - p1
    v2 = p3 - p1
    normal = np.cross(v1, v2)
    norm = np.linalg.norm(normal)
    return None if norm == 0 else normal / norm

def get_building_center(df):
    centroids = []
    for idx, row in df.iterrows():
        geom_raw = row["geometry_wkt"]
        try:
            geom = shapely.wkt.loads(geom_raw) if isinstance(geom_raw, str) else geom_raw
            if isinstance(geom, (Polygon, MultiPolygon)):
                polys = geom.geoms if isinstance(geom, MultiPolygon) else [geom]
                for poly in polys:
                    c = poly.centroid.coords[0]
                    centroid = np.array([*c, 0.0]) if len(c) == 2 else np.array(c)
                    centroids.append(centroid)
        except Exception:
            continue
    return np.mean(centroids, axis=0) if centroids else np.array([0.0, 0.0, 0.0])

def is_outward(normal, centroid, ref_point):
    direction = np.array(centroid) - np.array(ref_point)
    return np.dot(normal, direction) > 0

def plot_normals_with_planes(df, title="Wall Normals", surface_filter="external_with_hole"):
    import shapely.wkt
    from shapely.geometry import MultiPolygon, Polygon
    import numpy as np
    import plotly.graph_objects as go

    fig = go.Figure()
    building_center = get_building_center(df)

    for idx, row in df[df["thematic_surface"] == surface_filter].iterrows():
        geom_raw = row["geometry_wkt"]
        try:
            geom = shapely.wkt.loads(geom_raw) if isinstance(geom_raw, str) else geom_raw
        except Exception:
            continue
        polys = geom.geoms if isinstance(geom, MultiPolygon) else [geom]

        for poly in polys:
            coords = list(poly.exterior.coords)
            if len(coords) < 3:
                continue

            normal = compute_normal(poly)
            if normal is None:
                continue

            centroid_coords = poly.centroid.coords[0]
            centroid = np.array([*centroid_coords, 0.0]) if len(centroid_coords) == 2 else np.array(centroid_coords)

            outward = is_outward(normal, centroid, building_center)
            color = 'green' if outward else 'red'

            # Normal line
            normal_end = centroid + normal * 0.5
            fig.add_trace(go.Scatter3d(
                x=[centroid[0], normal_end[0]],
                y=[centroid[1], normal_end[1]],
                z=[centroid[2], normal_end[2]],
                mode='lines',
                line=dict(color=color, width=5),
                name="Normal OK" if outward else "Flipped"
            ))

            # Centroid point
            fig.add_trace(go.Scatter3d(
                x=[centroid[0]], y=[centroid[1]], z=[centroid[2]],
                mode='markers',
                marker=dict(size=4, color=color),
                showlegend=False
            ))

            # Add plane (as translucent triangle mesh)
            dx, dy = np.cross(normal, [0, 0, 1])[:2]  # orthogonal in XY
            offset = 0.5
            corners = [
                centroid + normal * 0.01 + np.array([ dx,  dy, 0]) * offset,
                centroid + normal * 0.01 + np.array([-dx,  dy, 0]) * offset,
                centroid + normal * 0.01 + np.array([-dx, -dy, 0]) * offset,
                centroid + normal * 0.01 + np.array([ dx, -dy, 0]) * offset,
            ]
            x, y, z = zip(*corners)
            fig.add_trace(go.Mesh3d(
                x=x, y=y, z=z,
                i=[0], j=[1], k=[2],
                opacity=0.2,
                color='red' if outward else 'blue',
                name="Plane Out" if outward else "Plane In",
                showlegend=False
            ))

    fig.update_layout(
        title=title,
        scene=dict(aspectmode="data"),
        legend=dict(itemsizing='constant')
    )
    fig.show()

def correct_normals_in_df(df):
    df_ext = df[df["thematic_surface"] == "external_with_hole"].copy()
    building_center = get_building_center(df_ext)

    for idx, row in df_ext.iterrows():
        geom_raw = row["geometry_wkt"]
        try:
            geom = shapely.wkt.loads(geom_raw) if isinstance(geom_raw, str) else geom_raw
        except Exception:
            continue

        new_polys = []
        polys = geom.geoms if isinstance(geom, MultiPolygon) else [geom]

        for poly in polys:
            coords = list(poly.exterior.coords)
            if len(coords) < 3:
                new_polys.append(poly)
                continue

            normal = compute_normal(poly)
            if normal is None:
                new_polys.append(poly)
                continue

            c = poly.centroid.coords[0]
            centroid = np.array([*c, 0.0]) if len(c) == 2 else np.array(c)
            outward = is_outward(normal, centroid, building_center)

            if not outward:
                # Flip normal
                poly = Polygon(list(poly.exterior.coords)[::-1], [list(ring.coords)[::-1] for ring in poly.interiors])

            new_polys.append(poly)

        new_geom = MultiPolygon(new_polys) if isinstance(geom, MultiPolygon) else new_polys[0]
        df.at[idx, "geometry_wkt"] = new_geom.wkt

    return df

# 🔍 Plot BEFORE fix
plot_normals_with_planes(df_transformed, title="Before Fix: Wall Normals")

df_transformed = df_transformed.drop(columns = 'related_to')
df_transformed = correct_normals_in_df(df_transformed)

# 🔍 Plot AFTER fix
plot_normals_with_planes(df_transformed, title="After Fix: Wall Normals")




In [ ]:
# df_transformed.head(120)

# from IPython.display import display, HTML
# display(HTML(df_transformed.head(150).to_html()))

In [ ]:
# from shapely.geometry import Polygon, MultiPolygon

# def extract_wall_face_coords(geom):
#     try:
#         if geom is None:
#             return None
#         faces = []
#         polys = geom.geoms if isinstance(geom, MultiPolygon) else [geom]
#         for poly in polys:
#             coords = list(poly.exterior.coords)
#             if len(coords) < 4:
#                 continue
#             # Remove closure if present (first == last)
#             if coords[0] == coords[-1]:
#                 coords = coords[:-1]
#             # Keep face if it's 4 distinct 3D points (typical vertical quad wall)
#             if len(coords) == 4:
#                 faces.append(coords)
#         return faces if faces else None
#     except Exception as e:
#         print(f"❌ Failed on geometry: {e}")
#         return None

# # Now apply to your parsed column (geometry_obj):
# walls_df['wall_faces_3d'] = walls_df['geometry_obj'].apply(extract_wall_face_coords)

# for _, row in walls_df[['element_id', 'parent_id', 'wall_faces_3d']].head(10).iterrows():
#     print(f"🧱 Wall {row['element_id']}, Parent Room {row['parent_id']}")
#     if row['wall_faces_3d']:
#         for face in row['wall_faces_3d']:
#             print("   🔹 Face:")
#             for pt in face:
#                 print(f"     {pt}")



In [ ]:
import numpy as np
import pandas as pd
from shapely import wkt
from shapely.errors import ShapelyError
from numpy.linalg import norm
from collections import defaultdict
from itertools import combinations

# ----------------------------------
# STEP 1: Safely load geometries
# ----------------------------------
def safe_wkt_load(val):
    try:
        if isinstance(val, str):
            return wkt.loads(val)
        return val
    except ShapelyError:
        return None

df_transformed['geometry_obj'] = df_transformed['geometry_wkt'].apply(safe_wkt_load)

# ----------------------------------
# STEP 2: Extract wall face coordinates
# ----------------------------------
def extract_wall_face_coords(geom):
    if geom is None:
        return None
    try:
        polys = geom.geoms if hasattr(geom, 'geoms') else [geom]
        faces = []
        for poly in polys:
            coords = list(poly.exterior.coords)
            if coords[0] == coords[-1]:
                coords = coords[:-1]  # remove closure
            if len(coords) == 4:
                face = [tuple(round(c, 5) for c in pt) for pt in coords]
                faces.append(face)
        return faces if faces else None
    except Exception as e:
        print("❌ Failed extracting face:", e)
        return None

walls_df = df_transformed[['element_id', 'parent_id', 'geometry_obj']].copy()
walls_df['wall_faces_3d'] = walls_df['geometry_obj'].apply(extract_wall_face_coords)

# ----------------------------------
# STEP 3: Canonicalize face (order & rotation invariant)
# ----------------------------------
def canonical_face(face, precision=3):
    if not isinstance(face, (list, tuple)) or len(face) != 4:
        return None
    rounded = [tuple(np.round(pt, precision)) for pt in face]
    variants = [tuple(rounded[i:] + rounded[:i]) for i in range(4)]
    variants += [tuple(reversed(v)) for v in variants]
    return min(variants)

# ----------------------------------
# STEP 4: Explode and clean
# ----------------------------------
walls_df = walls_df.explode('wall_faces_3d').dropna(subset=['wall_faces_3d']).copy()
walls_df['face_signature'] = walls_df['wall_faces_3d'].apply(canonical_face)
walls_df = walls_df.dropna(subset=['face_signature']).copy()

# ----------------------------------
# STEP 5: Find shared wall faces across rooms (exact match)
# ----------------------------------
face_map = defaultdict(list)
for _, row in walls_df.iterrows():
    face_map[row['face_signature']].append((row['element_id'], row['parent_id']))

shared_faces = []
for sig, group in face_map.items():
    parent_ids = {pid for _, pid in group}
    if len(parent_ids) > 1:
        shared_faces.append({'face_signature': sig, 'instances': group})

shared_df = pd.DataFrame(shared_faces)
print(f"✅ Shared wall faces across rooms (exact): {len(shared_df)}")
display(shared_df.head())

# ----------------------------------
# STEP 6: Fuzzy matching (geometry similarity)
# ----------------------------------
def face_distance(f1, f2, precision=3):
    try:
        f1 = [tuple(np.round(pt, precision)) for pt in f1]
        f2 = [tuple(np.round(pt, precision)) for pt in f2]
        return sum(min(norm(np.array(p1) - np.array(p2)) for p2 in f2) for p1 in f1)
    except Exception:
        return np.inf

matches = []
rows = list(walls_df[['element_id', 'parent_id', 'wall_faces_3d']].itertuples(index=False))
for (e1, p1, f1), (e2, p2, f2) in combinations(rows, 2):
    if p1 != p2 and face_distance(f1, f2) < 0.05:
        matches.append(((e1, p1), (e2, p2)))

print(f"🔍 Approximate matches found: {len(matches)}")




In [ ]:
# # Step 1: Initialize mapping
# related_map = defaultdict(set)

# # Step 2: Collect matches from exact shared walls
# for group in shared_faces:
#     ids = [eid for eid, _ in group['instances']]
#     for eid in ids:
#         related_map[eid].update(i for i in ids if i != eid)

# # Step 3: Add fuzzy matches as well
# for (e1, _), (e2, _) in matches:
#     related_map[e1].add(e2)
#     related_map[e2].add(e1)

# # Step 4: Convert to DataFrame
# related_df = pd.DataFrame({'element_id': list(related_map.keys())})
# related_df['related_to'] = related_df['element_id'].apply(lambda eid: sorted(related_map[eid]))

# # Step 5: Join with wall metadata if needed
# walls_with_related = walls_df[['element_id']].drop_duplicates().merge(related_df, on='element_id', how='left')
# walls_with_related['related_to'] = walls_with_related['related_to'].apply(lambda x: x if isinstance(x, list) else [])

# # Done
# print("✅ Related walls collected.")
# display(walls_with_related.head())

# Step 1: Build related wall mapping from shared faces and fuzzy matches
from collections import defaultdict

related_map = defaultdict(set)

# 🔁 Exact shared wall face matches
for group in shared_faces:
    ids = [eid for eid, _ in group['instances']]
    for eid in ids:
        related_map[eid].update(i for i in ids if i != eid)

# 🔁 Fuzzy matches
for (e1, _), (e2, _) in matches:
    related_map[e1].add(e2)
    related_map[e2].add(e1)

# Step 2: Create DataFrame
related_df = pd.DataFrame({'element_id': list(related_map.keys())})
related_df['related_to'] = related_df['element_id'].apply(lambda eid: sorted(related_map[eid]))

# Step 3: Add to df_transformed (ensure element_id is int for safe merge)
df_transformed['element_id'] = df_transformed['element_id'].astype(int)
related_df['element_id'] = related_df['element_id'].astype(int)

# Step 4: Merge into df_transformed
df_transformed = df_transformed.merge(related_df, on='element_id', how='left')
df_transformed['related_to'] = df_transformed['related_to'].apply(lambda x: x if isinstance(x, list) else [])

df_transformed = df_transformed.drop(columns = 'geometry_obj')

print("✅ Column 'related_to' added to df_transformed.")
display(df_transformed[['element_id', 'related_to']].head())



In [ ]:
df_transformed.head(220)


In [ ]:
# # Build quick lookup for thematic_surface by wall_id
# surface_map = df_transformed.set_index('element_id')['thematic_surface'].to_dict()

# # Check consistency
# inconsistent_closure_pairs = []

# for idx, row in df_transformed.iterrows():
#     wall_id = row['element_id']
#     this_label = row['thematic_surface']
#     related_ids = row.get('related_to', [])

#     for rid in related_ids:
#         related_label = surface_map.get(rid)
#         if (this_label == 'closure') != (related_label == 'closure'):
#             inconsistent_closure_pairs.append((wall_id, rid))

# # Report
# if inconsistent_closure_pairs:
#     print(f"❌ Found {len(inconsistent_closure_pairs)} inconsistent 'Closure' thematic_surface pairs:")
#     for pair in inconsistent_closure_pairs[:10]:  # show a few
#         print("  Wall", pair[0], "<→> Wall", pair[1])
# else:
#     print("✅ All related walls have consistent 'Closure' labels.")
    

# ########
# # Build quick lookups
# # Build lookups
# surface_map = df_transformed.set_index('element_id')['thematic_surface'].to_dict()
# parent_map = df_transformed.set_index('element_id')['parent_id'].to_dict()

# # Store mismatches with detailed info
# inconsistent_closure_pairs = []

# # Process rows
# for idx, row in df_transformed.iterrows():
#     wall_id = row['element_id']
#     this_label = row['thematic_surface']
#     related_ids = row.get('related_to', [])

#     for rid in related_ids:
#         related_label = surface_map.get(rid)

#         if (this_label == 'closure') != (related_label == 'closure'):
#             # Get parent IDs
#             parent_first = parent_map.get(wall_id)
#             parent_second_before = parent_map.get(rid)

#             # Store for reporting
#             inconsistent_closure_pairs.append((wall_id, rid, parent_first, parent_second_before))

#             # Fix: assign parent of first wall to second
#             df_transformed.loc[df_transformed['element_id'] == rid, 'parent_id'] = parent_first

# # Report
# if inconsistent_closure_pairs:
#     print(f"❌ Found {len(inconsistent_closure_pairs)} inconsistent 'closure' thematic_surface pairs:")
#     for wall_id, rid, parent_first, parent_second_before in inconsistent_closure_pairs[:10]:  # show a few
#         print(f"  Wall {wall_id} (parent_id={parent_first}) → Wall {rid} (was {parent_second_before}, now {parent_first})")
# else:
#     print("✅ All related walls have consistent 'closure' labels.")


# # Re-index for fast lookup
# df_indexed = df_transformed.set_index('element_id')

# # Track rows to drop (closures), and external walls to update
# closures_to_drop = set()
# externals_to_promote = []

# # Track processed groups
# processed_ids = set()

# for idx, row in df_transformed.iterrows():
#     wall_id = row['element_id']
#     if wall_id in processed_ids:
#         continue

#     related_ids = row.get('related_to', [])
#     group_ids = [wall_id] + related_ids
#     processed_ids.update(group_ids)

#     # Get actual rows in the group
#     group_rows = df_indexed.loc[df_indexed.index.intersection(group_ids)]

#     closure_rows = group_rows[group_rows['thematic_surface'] == 'closure']
#     external_rows = group_rows[group_rows['thematic_surface'] == 'external']

#     if not closure_rows.empty and not external_rows.empty:
#         # Case: both closure and external found → promote external
#         external_id = external_rows.index[0]
#         externals_to_promote.append(external_id)

#         # Drop all closure walls
#         closures_to_drop.update(closure_rows.index)

# # Apply changes
# # Drop closure rows
# df_transformed = df_transformed[~df_transformed['element_id'].isin(closures_to_drop)].reset_index(drop=True)

# # Update thematic_surface of promoted externals
# df_transformed.loc[df_transformed['element_id'].isin(externals_to_promote), 'thematic_surface'] = 'closure'

# print(f"✅ Replaced {len(externals_to_promote)} 'external' walls with 'closure' and removed {len(closures_to_drop)} redundant 'closure' walls.")


# --- Step 1: Build lookups ---
surface_map = df_transformed.set_index('element_id')['thematic_surface'].to_dict()
parent_map = df_transformed.set_index('element_id')['parent_id'].to_dict()

# Track inconsistencies and fix parent_ids
inconsistent_closure_pairs = []

for idx, row in df_transformed.iterrows():
    wall_id = row['element_id']
    this_label = row['thematic_surface']
    related_ids = row.get('related_to', [])

    for rid in related_ids:
        related_label = surface_map.get(rid)
        if related_label is None:
            continue

        # Inconsistency if one is 'closure' and the other is not
        if (this_label == 'closure') != (related_label == 'closure'):
            parent_first = parent_map.get(wall_id)
            parent_second_before = parent_map.get(rid)

            inconsistent_closure_pairs.append((wall_id, rid, parent_first, parent_second_before))

            # Fix: unify parent_id
            df_transformed.loc[df_transformed['element_id'] == rid, 'parent_id'] = parent_first

# --- Report inconsistencies fixed ---
if inconsistent_closure_pairs:
    print(f"❌ Fixed {len(inconsistent_closure_pairs)} inconsistent 'closure' label pairs:")
    for wall_id, rid, parent_first, parent_second_before in inconsistent_closure_pairs[:10]:
        print(f"  Wall {wall_id} (parent_id={parent_first}) → Wall {rid} (was {parent_second_before}, now {parent_first})")
else:
    print("✅ All related walls have consistent 'closure' labels.")

# --- Step 2: Prefer 'closure' over 'external' in related groups ---
df_indexed = df_transformed.set_index('element_id')

externals_to_drop = set()
closures_to_keep = set()
processed_ids = set()

for idx, row in df_transformed.iterrows():
    wall_id = row['element_id']
    if wall_id in processed_ids:
        continue

    related_ids = row.get('related_to', [])
    group_ids = [wall_id] + related_ids
    processed_ids.update(group_ids)

    group_rows = df_indexed.loc[df_indexed.index.intersection(group_ids)]

    closure_rows = group_rows[group_rows['thematic_surface'] == 'closure']
    external_rows = group_rows[group_rows['thematic_surface'] == 'external']

    if not closure_rows.empty and not external_rows.empty:
        # Prefer closure walls: keep them, drop external ones
        closures_to_keep.update(closure_rows.index)
        externals_to_drop.update(external_rows.index)

# --- Step 3: Apply changes ---

# Drop external walls that are redundant
df_transformed = df_transformed[~df_transformed['element_id'].isin(externals_to_drop)].reset_index(drop=True)

print(f"✅ Kept {len(closures_to_keep)} 'closure' walls and removed {len(externals_to_drop)} redundant 'external' walls.")




In [ ]:
from shapely import wkt
from shapely.geometry import MultiPolygon, Polygon
from shapely.errors import ShapelyError
import uuid

def safe_wkt_load(val):
    try:
        if isinstance(val, str):
            return wkt.loads(val)
        return val
    except ShapelyError:
        return None

df_transformed['geometry_obj'] = df_transformed['geometry_wkt'].apply(safe_wkt_load)

def to_multipolygon(geom):
    if geom is None:
        return None
    if isinstance(geom, Polygon):
        return MultiPolygon([geom])
    return geom

df_transformed['geometry_obj'] = df_transformed['geometry_obj'].apply(to_multipolygon)

df_transformed['element_gml_id'] = df_transformed.apply(
    lambda row: str(uuid.uuid4()) if row['thematic_surface'] == 'closure' and row['geometry_obj'] else row['element_gml_id'],
    axis=1
)

multipolygon_count = df_transformed['geometry_obj'].apply(lambda g: isinstance(g, MultiPolygon)).sum()
total = len(df_transformed)
print(f"✅ {multipolygon_count}/{total} geometries are now MultiPolygon.")

df_transformed = df_transformed.drop(columns='geometry_wkt')
df_transformed['geometry_wkt'] = df_transformed['geometry_obj'].apply(lambda g: g.wkt if g else '')
df_transformed = df_transformed.drop(columns='geometry_obj')



In [ ]:
print(df_transformed['element_id'].dtype)

df_transformed = df_transformed.dropna(subset=['parent_id'])
df_transformed['parent_id'] = pd.to_numeric(df_transformed['parent_id'], errors='coerce')

# Step 2: Drop rows with NaN in parent_id
df_transformed = df_transformed.dropna(subset=['parent_id'])

# Step 3: Convert to integer safely
df_transformed['parent_id'] = df_transformed['parent_id'].astype(int)


# df_transformed['parent_id'] = pd.to_numeric(df_transformed['parent_id'], errors='coerce').astype('Int64')
print(df_transformed['parent_id'].dtype)


In [ ]:
import numpy as np
import pandas as pd

thematic = df_transformed['thematic_surface'].copy()
parent_ids = df_transformed['parent_id'].copy()

for i in df_transformed.index:
    if i == 0:
        continue  # ⛔️ Skip first row

    if thematic[i] == 'wall':
        print(f"🧱 Wall at index {i} needs adjustment.")
        prev_theme = thematic[i - 1]
        next_theme = thematic[i + 1] if i < len(thematic) - 1 else np.nan

        assigned_theme = None
        for neighbor in [prev_theme, next_theme]:
            if pd.notna(neighbor) and neighbor != 'wall':
                assigned_theme = neighbor
                thematic[i] = assigned_theme
                break

        print(f"[thematic_surface] Row {i}: was 'wall'. Prev: {prev_theme}, Next: {next_theme}, Assigned: {assigned_theme}")

    if pd.isna(parent_ids[i]):
        prev_pid = parent_ids[i - 1]
        next_pid = parent_ids[i + 1] if i < len(parent_ids) - 1 else pd.NA

        assigned_pid = None
        for neighbor in [prev_pid, next_pid]:
            if pd.notna(neighbor):
                assigned_pid = neighbor
                parent_ids[i] = assigned_pid
                break

        print(f"[parent_id]       Row {i}: was <NA>. Prev: {prev_pid}, Next: {next_pid}, Assigned: {assigned_pid}")

df_transformed['thematic_surface'] = thematic
df_transformed['parent_id'] = parent_ids



In [ ]:
df_transformed.head(120)

from IPython.display import display, HTML
display(HTML(df_transformed.head(280).to_html()))


SAVE THE FILE

In [ ]:
wkt_csv = r"C:\Users\oscar\OneDrive - Fondazione Bruno Kessler\KUL_GeometricModel\script\outputs\jsons\first_floor_v13.csv"
wkt_txt = r"C:\Users\oscar\OneDrive - Fondazione Bruno Kessler\KUL_GeometricModel\script\outputs\jsons\first_floor_v13.txt"

# Export the DataFrame to CSV and TXT
df_transformed.to_csv(wkt_csv, index=False, encoding="utf-8")
df_transformed.to_csv(wkt_txt, sep="\t", index=False, encoding="utf-8")

# Print confirmation of export completion
print("✅ Export complete:")
print(f"📄 CSV written to: {wkt_csv}")
print(f"📄 TXT written to: {wkt_txt}")


In [ ]:
ssssss

PLOT RESULTS

In [ ]:
import pandas as pd
import numpy as np
from shapely import wkt
from shapely.geometry import Polygon, MultiPolygon
from shapely.geometry.base import BaseGeometry
import plotly.graph_objects as go
import plotly.express as px

# ✅ Load and filter data
df = df_transformed.copy()
df = df[df["geometry_wkt"].notnull()].copy()

def safe_load_wkt(val):
    if isinstance(val, BaseGeometry):
        return val
    if isinstance(val, str) and val.strip():
        try:
            return wkt.loads(val)
        except Exception as e:
            print(f"Invalid WKT: {val} | Error: {e}")
            return None
    return None

df["geometry"] = df["geometry_wkt"].apply(safe_load_wkt)
df["geometry"] = df["geometry"].apply(lambda g: MultiPolygon([g]) if isinstance(g, Polygon) else g)

# ✅ Classify thematic surfaces
def classify_surface(row):
    if row["thematic_surface"] == "closure":
        if isinstance(row["parent_id"], str) and row["parent_id"].startswith("room_"):
            return "room_closure"
    return row["thematic_surface"]

df["thematic_surface"] = df.apply(classify_surface, axis=1)

# ✅ Refine wall types based on existing type + hole info
def refine_wall_labels(row):
    if row["thematic_surface"] == "party_wall" and row.get("has_holes") is True:
        return "external_wall"
    elif row["thematic_surface"] == "party_wall":
        return "internal_wall"
    return row["thematic_surface"]

df["thematic_surface"] = df.apply(refine_wall_labels, axis=1)

# ✅ Assign colors to thematic surfaces
surface_types = df["thematic_surface"].dropna().unique()
colors = px.colors.qualitative.Set2 + px.colors.qualitative.Set3
color_map = {s: colors[i % len(colors)] for i, s in enumerate(surface_types)}

# ✅ Filter out rows with invalid geometries before computing bounding box
valid_geoms = df["geometry"].dropna()

all_xyz = np.concatenate([
    np.array(poly.exterior.coords)
    for geom in valid_geoms
    for poly in geom.geoms
])

x_range = [all_xyz[:, 0].min(), all_xyz[:, 0].max()]
y_range = [all_xyz[:, 1].min(), all_xyz[:, 1].max()]
z_range = [all_xyz[:, 2].min(), all_xyz[:, 2].max()]

# ✅ Create 3D plot
fig = go.Figure()

# ✅ Filter out invalid geometries before plotting
df = df[df["geometry"].notnull()].copy()

for _, row in df.iterrows():
    color = color_map.get(row["thematic_surface"], "gray")
    for poly in row["geometry"].geoms:
        coords = np.array(poly.exterior.coords)
        hover_text = f"""
        Type: {row['thematic_surface']}<br>
        Element ID: {row['element_id']}<br>
        Parent: {row['parent_id']}<br>
        Has Holes: {row['has_holes']}
        """
        fig.add_trace(go.Scatter3d(
            x=coords[:, 0], y=coords[:, 1], z=coords[:, 2],
            mode='lines',
            line=dict(color=color, width=4.5),
            name=row["thematic_surface"],
            showlegend=False,
            hoverinfo="text",
            text=hover_text
        ))


# ✅ Add manual legend
for thematic, color in color_map.items():
    fig.add_trace(go.Scatter3d(
        x=[None], y=[None], z=[None],
        mode='markers',
        marker=dict(size=6, color=color),
        name=thematic
    ))

aspect = {
    "x": max(x_range[1] - x_range[0], 1e-6),
    "y": max(y_range[1] - y_range[0], 1e-6),
    "z": max(z_range[1] - z_range[0], 1e-6)
}
max_range = max(aspect.values())
aspect_ratio = {axis: round(aspect[axis] / max_range, 4) for axis in aspect}

# fig.update_layout(
#     scene=dict(
#         xaxis=dict(title='X', range=x_range),
#         yaxis=dict(title='Y', range=y_range),
#         zaxis=dict(title='Z', range=z_range),
#         aspectmode='manual',
#         aspectratio=aspect_ratio
#     ),
#     title="3D Building Model — Colored by Thematic Surface",
#     legend=dict(title="Thematic Surface", itemsizing="constant"),
#     margin=dict(l=10, r=10, b=10, t=40),
#     width=1200,   # ✅ Wider
#     height=900    # ✅ Taller
# )

fig.update_layout(
    scene=dict(
        xaxis=dict(title='X', range=x_range, backgroundcolor='white', gridcolor='lightgray', zerolinecolor='gray'),
        yaxis=dict(title='Y', range=y_range, backgroundcolor='white', gridcolor='lightgray', zerolinecolor='gray'),
        zaxis=dict(title='Z', range=z_range, backgroundcolor='white', gridcolor='lightgray', zerolinecolor='gray'),
        aspectmode='manual',
        aspectratio=aspect_ratio,
        bgcolor='white'  # ✅ Set scene background to white
    ),
    paper_bgcolor='white',  # ✅ Remove outer blue background
    plot_bgcolor='white',   # ✅ Remove plot background
    title="3D Building Model — Colored by Thematic Surface",
    legend=dict(title="Thematic Surface", itemsizing="constant"),
    margin=dict(l=10, r=10, b=10, t=40),
    width=1800,
    height=900
)

fig.show()




GRAPH

In [ ]:
# Your provided light positions (replace the previous 'lights' list)
light_positions = [
    [-37.26045413208008, 28.02169111251831, 6.63],
    [-36.92945413208008, 24.524691112518312, 6.656],
    [-33.838454132080074, 24.51269111251831, 6.601],
    [-34.15545413208008, 28.213691112518312, 6.563],
    [-43.96745413208008, 24.33569111251831, 6.565],
    [-40.93245413208008, 24.46569111251831, 6.604],
    [-44.41745413208008, 28.05269111251831, 6.594],
    [-40.94445413208008, 28.10569111251831, 6.594],
    [-51.834145132080076, 24.289691112518312, 6.565],
    [-48, 24.289691112518312, 6.565],
    [-51.834145132080076, 28.289691112518312, 6.565],
    [-48, 28.289691112518312, 6.565],

    [-37.26045413208008, 7.978308887481688, 6.63],
    [-36.92945413208008, 11.475308887481688, 6.656],
    [-33.838454132080074, 11.487308887481688, 6.601],
    [-34.15545413208008, 7.786308887481688, 6.563],
    [-43.96745413208008, 11.664308887481688, 6.565],
    [-40.93245413208008, 11.534308887481688, 6.604],
    [-44.41745413208008, 7.947308887481688, 6.594],
    [-40.94445413208008, 7.894308887481688, 6.594],
    [-51.834145132080076, 11.710308887481688, 6.565],
    [-48, 11.710308887481688, 6.565],
    [-51.834145132080076, 7.710308887481688, 6.565],
    [-48, 7.710308887481688, 6.565],
]

lights = [
    {"id": f"light_{i}", "position": pos}
    for i, pos in enumerate(light_positions)
]

rad_x = [-34, -37, -41, -45, -47, -51]
rad_y = [6.8, 29.3] * len(rad_x)
radiators = [{"id": f"rad_{x}_{y}", "position": [x, y, 4]} for x, y in zip(rad_x, rad_y)]

temp_x = [-37, -42.50, -50]
temp_y = [22.50, 14]
hum_x = [-37, -42.50, -50]
hum_y = [22.50, 14]

temp_positions = [(x, y) for x in temp_x for y in temp_y]
hum_positions = [(x, y) for x in hum_x for y in hum_y]

sensors = (
    [{"id": f"temp_{x}_{y}", "type": "temperature", "position": [x, y, 4.70]} for x, y in temp_positions] +
    [{"id": f"hum_{x}_{y}", "type": "humidity", "position": [x, y, 5]} for x, y in hum_positions]
)

devices = {
    "light": lights,
    "radiator": radiators,
    "sensor": sensors
}

import pandas as pd
from shapely import wkt
from shapely.geometry import Polygon, MultiPolygon

# TopologicPy imports
from topologicpy.Vertex import Vertex
from topologicpy.Face import Face
from topologicpy.Shell import Shell
from topologicpy.Cell import Cell
from topologicpy.CellComplex import CellComplex
from topologicpy.Cluster import Cluster
from topologicpy.Dictionary import Dictionary
from topologicpy.Topology import Topology
from topologicpy.Plotly import Plotly
import plotly.graph_objects as go

# --- 1. Convert WKT to Topologic Face ---
def wkt_to_face(geom):
    try:
        if isinstance(geom, str):
            geom = wkt.loads(geom)
        if isinstance(geom, Polygon):
            coords = list(geom.exterior.coords)
        elif isinstance(geom, MultiPolygon):
            coords = list(list(geom.geoms)[0].exterior.coords)
        else:
            return None
        vertices = [Vertex.ByCoordinates(x, y, z) for x, y, z in coords]
        return Face.ByVertices(vertices)
    except Exception as e:
        print(f"Error converting geometry: {e}")
        return None
    
# --- 2. Build Room Cells ---
df = df_transformed  # Your building dataset
rooms = df[df["thematic_surface"] == "room"]
cells = []

for _, room in rooms.iterrows():
    room_id = room["element_id"]
    room_faces_df = df[df["parent_id"] == float(room_id)]
    faces = []
    for _, row in room_faces_df[room_faces_df["geometry_wkt"].notna()].iterrows():
        face = wkt_to_face(row["geometry_wkt"])
        if face:
            faces.append(face)
    print(f"Room {room_id} has {len(faces)} faces")
    if len(faces) >= 3:
        shell = Shell.ByFaces(faces)
        if shell:
            cell = Cell.ByShell(shell)
            if cell:
                d = Dictionary.ByKeysValues(["type"], ["room"])
                Topology.SetDictionary(cell, d)
                cells.append(cell)
                print(f"Added cell for room {room_id}")
            else:
                print(f"Failed to create Cell for room {room_id}")
        else:
            print(f"Failed to create Shell for room {room_id} -- skipping")
    else:
        print(f"Not enough faces to create shell for room {room_id}")

if not cells:
    raise RuntimeError("❌ No valid cells created.")

cell_complex = CellComplex.ByCells(cells)

from topologicpy.Topology import Topology

room_vertices = []
for i, cell in enumerate(cells):
    centroid = Topology.Centroid(cell)
    d = Dictionary.ByKeysValues(["type", "room", "index"], ["room", str(i), str(i)])
    centroid = Topology.SetDictionary(centroid, d)
    room_vertices.append(centroid)


# --- 3. Thematic Faces (walls, floors, windows, etc.) ---
element_types = ["external", "external_with_hole", "parly_wall", "party_wall_with_hole", "floor", "ceiling", "window", "door"]
surfaces_df = df[df["thematic_surface"].isin(element_types) & df["geometry_wkt"].notna()]
faces = []

for _, row in surfaces_df.iterrows():
    face = wkt_to_face(row["geometry_wkt"])
    if face:
        element_type = row["thematic_surface"]
        element_id = str(row["element_id"])
        d = Dictionary.ByKeysValues(["id", "type"], [element_id, element_type])
        face = Topology.SetDictionary(face, d)
        faces.append(face)

def add_thematic_vertex(obj_list, obj_type):
    vertices = []
    for obj in obj_list:
        v = Vertex.ByCoordinates(*obj["position"])
        keys = ["id", "type"]
        values = [obj["id"], obj_type]
        if "type" in obj and obj_type == "sensor":
            keys.append("subtype")
            values.append(obj["type"])
        d = Dictionary.ByKeysValues(keys, values)
        v = Topology.SetDictionary(v, d)
        vertices.append(v)
    return vertices

light_vertices = add_thematic_vertex(devices["light"], "light")
radiator_vertices = add_thematic_vertex(devices["radiator"], "radiator")
sensor_vertices = add_thematic_vertex(devices["sensor"], "sensor")

from topologicpy.Edge import Edge
from topologicpy.Graph import Graph
import math

def distance(v1, v2):
    x1, y1, z1 = Vertex.Coordinates(v1)
    x2, y2, z2 = Vertex.Coordinates(v2)
    return math.sqrt((x1 - x2)**2 + (y1 - y2)**2 + (z1 - z2)**2)

edges = []

# Connect each device to the closest room centroid
all_devices = light_vertices + radiator_vertices + sensor_vertices
for device in all_devices:
    closest_room = min(room_vertices, key=lambda rv: distance(rv, device))
    edge = Edge.ByVertices(device, closest_room)
    edges.append(edge)


# --- 4. Room Shell Plot ---
data_cells = Plotly.DataByTopology(cell_complex, faceOpacity=0.25)

# --- 5. Device Colors ---
type_colors = {
    "light": "rgb(255, 191, 0)",        # amber
    "radiator": "rgb(255, 87, 34)",     # orange/red
    "temperature": "rgb(0, 188, 212)",  # cyan
    "humidity": "rgb(103, 58, 183)"     # purple
}

graph_vertices = all_devices + room_vertices
graph = Graph.ByVerticesEdges(graph_vertices, edges)


# --- 6. Device Markers by Category (for legend) ---
def device_scatter(vertices, label, color):
    return go.Scatter3d(
        x=[Vertex.Coordinates(v)[0] for v in vertices],
        y=[Vertex.Coordinates(v)[1] for v in vertices],
        z=[Vertex.Coordinates(v)[2] for v in vertices],
        mode='markers',
        name=label,
        marker=dict(size=6, color=color),
        showlegend=True
    )

device_traces = [
    device_scatter(light_vertices, "Light", type_colors["light"]),
    device_scatter(radiator_vertices, "Radiator", type_colors["radiator"]),
    device_scatter(
        [v for v in sensor_vertices if Dictionary.ValueAtKey(Topology.Dictionary(v), "subtype") == "temperature"],
        "Sensor (Temp)", type_colors["temperature"]
    ),
    device_scatter(
        [v for v in sensor_vertices if Dictionary.ValueAtKey(Topology.Dictionary(v), "subtype") == "humidity"],
        "Sensor (Humidity)", type_colors["humidity"]
    )
]

print(f"Number of cells created: {len(cells)}")

print(f"Number of valid cells: {len(cells)}")

cell_complex = None
if len(cells) > 0:
    cell_complex = CellComplex.ByCells(cells)
    if cell_complex is None:
        print("CellComplex.ByCells returned None")

if cell_complex:
    data_cells = Plotly.DataByTopology(cell_complex, faceOpacity=0.25)
else:
    print("Falling back to cluster plotting...")
    cluster = Cluster.ByTopologies(cells)
    data_cells = Plotly.DataByTopology(cluster, faceOpacity=0.25)

if data_cells:
    fig = Plotly.FigureByData(data_cells)
else:
    print("Warning: No data to plot, creating empty figure.")
    fig = go.Figure()

for trace in device_traces:
    if trace:
        fig.add_trace(trace)

print("Number of vertices:", len(graph_vertices))
print("Number of edges:", len(edges))
print("Graph is None:", graph is None)

graph_data = Plotly.DataByTopology(graph)

# Clean inputs before creating graph
edges = [e for e in edges if e is not None]
graph_vertices = [v for v in graph_vertices if v is not None]

import itertools

room_room_edges = []

# Adjust this number to control how many neighbors to connect
K = 12

for i, v1 in enumerate(room_vertices):
    # Compute distances to all other rooms
    distances = [(v2, distance(v1, v2)) for j, v2 in enumerate(room_vertices) if j != i]
    distances.sort(key=lambda x: x[1])
    nearest = distances[:K]
    
    for v2, _ in nearest:
        edge = Edge.ByVertices(v1, v2)
        if edge:
            room_room_edges.append(edge)

all_edges = edges + room_room_edges


room_graph_trace = go.Scatter3d(
    x=[],
    y=[],
    z=[],
    mode='lines',
    line=dict(color='gray', width=1.5),
    name='Room-Room Links',
    showlegend=True
)

for edge in all_edges:
    if edge:
        verts = Topology.Vertices(edge)
        if verts and len(verts) == 2:
            x1, y1, z1 = Vertex.Coordinates(verts[0])
            x2, y2, z2 = Vertex.Coordinates(verts[1])
            room_graph_trace.x += (x1, x2, None)
            room_graph_trace.y += (y1, y2, None)
            room_graph_trace.z += (z1, z2, None)

fig.add_trace(room_graph_trace)


fig.update_layout(
    title="Topologic 3D Building Model (Enhanced View)",
    title_font_size=20,
    width=1400,   # ✅ Increase width
    height=900,   # ✅ Increase height
    scene=dict(
        bgcolor="white",
        xaxis=dict(visible=False),
        yaxis=dict(visible=False),
        zaxis=dict(visible=False),
    ),
    margin=dict(l=0, r=0, t=40, b=0),
    legend=dict(
        bgcolor="rgba(255,255,255,0.95)",
        bordercolor="lightgray",
        borderwidth=1,
        x=0.01,
        y=0.99,
        font=dict(size=12),
        orientation="v"
    )
)


fig.show()

import random
import plotly.graph_objects as go

# --- Final Visualization: Unique Color Per Room ---
colored_room_traces = []

for i, cell in enumerate(cells):
    cell_data = Plotly.DataByTopology(cell, faceOpacity=0.8)

    if not cell_data:
        continue

    # Generate a distinct RGB color
    r = random.randint(50, 230)
    g = random.randint(50, 230)
    b = random.randint(50, 230)
    color = f'rgb({r},{g},{b})'

    for trace in cell_data:
        if isinstance(trace, go.Mesh3d):
            trace.color = color
            trace.opacity = 0.8
        colored_room_traces.append(trace)

# --- Plot with Real Aspect Ratio ---
fig = go.Figure(data=colored_room_traces)

fig.update_layout(
    title="Topologic 3D Rooms (Each with Unique Color, Real-World Proportions)",
    title_font_size=20,
    scene=dict(
        bgcolor="white",
        xaxis=dict(visible=False),
        yaxis=dict(visible=False),
        zaxis=dict(visible=False),
        aspectmode='data'  # Maintain real-world proportions!
    ),
    margin=dict(l=0, r=0, t=40, b=0)
)

fig.show()


In [ ]:
import networkx as nx
import plotly.graph_objects as go
from topologicpy.Vertex import Vertex
from topologicpy.Topology import Topology
from topologicpy.Dictionary import Dictionary

# --- Helper: Get unique and consistent ID for a Vertex ---
def get_vertex_id(v):
    d = Topology.Dictionary(v)
    id_val = Dictionary.ValueAtKey(d, "id")
    if id_val:
        return str(id_val)
    else:
        x, y, z = Vertex.Coordinates(v)
        return f"{x:.2f}_{y:.2f}_{z:.2f}"

# --- Create Graph ---
G = nx.Graph()
vertex_id_map = {}

# Assume: graph_vertices and all_edges are already defined (from Topologic context)

# --- Add nodes ---
for v in graph_vertices:
    x, y, z = Vertex.Coordinates(v)
    node_id = get_vertex_id(v)
    G.add_node(node_id, x=x, y=y, z=z)
    vertex_id_map[node_id] = (x, y, z)

# --- Add edges ---
for e in all_edges:
    verts = Topology.Vertices(e)
    if verts and len(verts) == 2:
        id1 = get_vertex_id(verts[0])
        id2 = get_vertex_id(verts[1])
        G.add_edge(id1, id2)

# --- Prepare Plotly Data ---
node_x, node_y, node_z, node_labels = [], [], [], []
for node, data in G.nodes(data=True):
    node_x.append(data["x"])
    node_y.append(data["y"])
    node_z.append(data["z"])
    node_labels.append(node)

edge_x, edge_y, edge_z = [], [], []
for u, v in G.edges():
    x0, y0, z0 = G.nodes[u]["x"], G.nodes[u]["y"], G.nodes[u]["z"]
    x1, y1, z1 = G.nodes[v]["x"], G.nodes[v]["y"], G.nodes[v]["z"]
    edge_x += [x0, x1, None]
    edge_y += [y0, y1, None]
    edge_z += [z0, z1, None]

# --- Plotly 3D Figure ---
fig = go.Figure()

fig.add_trace(go.Scatter3d(
    x=node_x, y=node_y, z=node_z,
    mode='markers+text',
    text=node_labels,
    textposition="top center",
    marker=dict(size=5, color='blue'),
    name="Nodes"
))

fig.add_trace(go.Scatter3d(
    x=edge_x, y=edge_y, z=edge_z,
    mode='lines',
    line=dict(color='gray', width=1.5),
    name="Edges"
))

fig.update_layout(
    title="Topologic Graph (NetworkX + Plotly 3D)",
    scene=dict(
        bgcolor='white',
        xaxis=dict(visible=False),
        yaxis=dict(visible=False),
        zaxis=dict(visible=False)
    ),
    margin=dict(l=0, r=0, t=40, b=0),
    legend=dict(
        bgcolor='rgba(255,255,255,0.95)',
        bordercolor='lightgray',
        borderwidth=1,
        x=0.01,
        y=0.99
    )
)

# --- Save and Show HTML ---
fig.write_html("topologic_graph.html", include_plotlyjs='cdn')
import webbrowser
webbrowser.open("topologic_graph.html")



In [ ]:
import networkx as nx
import plotly.graph_objects as go
import random

# Force layout with 2D positioning
pos = nx.spring_layout(G, dim=2, seed=42, k=0.4)  # k tunes spacing

# Define color scheme
color_map = {
    "light": "#fdd835",       # yellow (lights)
    "radiator": "#ef5350",    # red
    "sensor_temperature": "#4dd0e1",  # cyan
    "sensor_humidity": "#9575cd",     # purple
    "room": "#66bb6a",        # green
    None: "#90a4ae"           # gray default
}

# Group nodes
node_groups = {}
for node, data in G.nodes(data=True):
    vtype = data.get("type")
    subtype = data.get("subtype")
    key = f"sensor_{subtype}" if vtype == "sensor" and subtype else vtype or "other"
    if key not in node_groups:
        node_groups[key] = []
    node_groups[key].append(node)

# Build edge trace
edge_x, edge_y = [], []
for u, v in G.edges():
    x0, y0 = pos[u]
    x1, y1 = pos[v]
    edge_x += [x0, x1, None]
    edge_y += [y0, y1, None]

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.6, color='rgba(200,150,200,0.4)'),  # soft pink-purple
    hoverinfo='none',
    mode='lines',
    name='Links'
)

# Create node traces
node_traces = []
for key, nodes in node_groups.items():
    x, y, text = [], [], []
    color = color_map.get(key, "#BDBDBD")  # fallback gray
    for n in nodes:
        x.append(pos[n][0])
        y.append(pos[n][1])
        text.append(n)
    trace = go.Scatter(
        x=x, y=y,
        mode='markers',
        marker=dict(size=10, color=color, line=dict(width=0.5, color='black')),
        text=text,
        hoverinfo='text',
        name=key.replace("_", " ").title()
    )
    node_traces.append(trace)

# Create figure
fig = go.Figure()
fig.add_trace(edge_trace)
for trace in node_traces:
    fig.add_trace(trace)

fig.update_layout(
    title="🎯 Stylized Network Graph (Flat Layout)",
    title_font_size=22,
    showlegend=True,
    hovermode='closest',
    margin=dict(l=10, r=10, t=50, b=10),
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    plot_bgcolor='white',
    legend=dict(
        bgcolor='rgba(255,255,255,0.95)',
        bordercolor='lightgray',
        borderwidth=1,
        font=dict(size=11),
        x=0.01,
        y=0.99
    )
)

# Save and open
fig.write_html("cool_flat_graph.html", include_plotlyjs='cdn')
import webbrowser
webbrowser.open("cool_flat_graph.html")



In [ ]:
import networkx as nx
import plotly.graph_objects as go
from topologicpy.Vertex import Vertex
from topologicpy.Topology import Topology
from topologicpy.Dictionary import Dictionary

# --- 1. Helper to assign consistent vertex IDs ---
def get_vertex_id(v):
    d = Topology.Dictionary(v)
    id_val = Dictionary.ValueAtKey(d, "id")
    if id_val:
        return str(id_val)
    else:
        x, y, z = Vertex.Coordinates(v)
        return f"{x:.2f}_{y:.2f}_{z:.2f}"

# --- 2. Build Graph ---
G = nx.Graph()
for v in graph_vertices:
    x, y, z = Vertex.Coordinates(v)
    d = Topology.Dictionary(v)
    node_id = get_vertex_id(v)
    G.add_node(node_id,
               x=x, y=y, z=z,
               type=Dictionary.ValueAtKey(d, "type"),
               subtype=Dictionary.ValueAtKey(d, "subtype"))

for e in all_edges:
    verts = Topology.Vertices(e)
    if verts and len(verts) == 2:
        id1 = get_vertex_id(verts[0])
        id2 = get_vertex_id(verts[1])
        if id1 != id2:
            G.add_edge(id1, id2)

# --- 3. Force-directed layout (tight & technical) ---
pos = nx.spring_layout(G, seed=42, k=0.15, iterations=200)

# --- 4. Summer-fancy color palette ---
color_map = {
    "light": "#ffe066",              # Banana yellow
    "radiator": "#ff6b6b",           # Watermelon red
    "sensor_temperature": "#4ecdc4", # Turquoise
    "sensor_humidity": "#5f27cd",    # Purple sky
    "room": "#10ac84",               # Palm green
    None: "#8395a7"                  # Neutral gray
}

# --- 5. Node properties with fancy colors ---
node_x, node_y, node_color, node_size = [], [], [], []
for node, data in G.nodes(data=True):
    x, y = pos[node]
    vtype = data.get("type")
    subtype = data.get("subtype")
    key = f"sensor_{subtype}" if vtype == "sensor" and subtype else vtype
    color = color_map.get(key, "#8395a7")

    node_x.append(x)
    node_y.append(y)
    node_color.append(color)

    # Size by degree, bold and technical
    size = 14 + nx.degree(G, node) * 0.6
    node_size.append(size)

# --- 7. Plotly node trace ---
node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    marker=dict(
        color=node_color,
        size=node_size,
        line=dict(width=1, color='white'),  # white outline for pop
        opacity=0.95
    ),
    hoverinfo='skip',
    name='Devices'
)

# --- 8. Plot background, tech layout with a vibrant feel ---
fig.update_layout(
    title="🌴 Summer-Styled Technical Network Graph",
    title_font_size=22,
    showlegend=False,
    hovermode='closest',
    plot_bgcolor='white',
    margin=dict(l=20, r=20, t=40, b=20),
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)
)

# --- 9. Save & open ---
fig.write_html("technical_graph.html", include_plotlyjs='cdn')
import webbrowser
webbrowser.open("technical_graph.html")



In [ ]:
graph_ok_0 = True

if graph_ok_0:

    !pip install --upgrade pyvis jinja2

    from pyvis.network import Network
    import networkx as nx
    from topologicpy.Vertex import Vertex
    from topologicpy.Topology import Topology
    from topologicpy.Dictionary import Dictionary

    # --- Step 1: Create Graph ---
    G = nx.Graph()

    def get_vertex_id(v):
        d = Topology.Dictionary(v)
        id_val = Dictionary.ValueAtKey(d, "id")
        if id_val:
            return str(id_val)
        else:
            x, y, z = Vertex.Coordinates(v)
            return f"{x:.2f}_{y:.2f}_{z:.2f}"

    # --- Step 2: Add Nodes ---
    for v in graph_vertices:
        x, y, z = Vertex.Coordinates(v)
        d = Topology.Dictionary(v)
        node_id = get_vertex_id(v)
        G.add_node(node_id,
                type=Dictionary.ValueAtKey(d, "type"),
                subtype=Dictionary.ValueAtKey(d, "subtype"))

    # --- Step 3: Add Edges ---
    for e in all_edges:
        verts = Topology.Vertices(e)
        if verts and len(verts) == 2:
            id1 = get_vertex_id(verts[0])
            id2 = get_vertex_id(verts[1])
            if id1 != id2:
                G.add_edge(id1, id2)

    # --- Step 4: Summer Color Palette (ROOMS = RED) ---
    color_map = {
        "light": "#ffe066",               # Banana yellow
        "radiator": "#ff6b6b",            # Watermelon
        "sensor_temperature": "#4ecdc4",  # Tropical turquoise
        "sensor_humidity": "#5f27cd",     # Summer night purple
        "room": "#e74c3c",                # 🔴 Bright red
        None: "#8395a7"                   # Default gray
    }

    # --- Step 5: Build Pyvis Graph ---
    net = Network(height='850px', width='100%', bgcolor='white', font_color='black')
    net.barnes_hut(gravity=-30000, spring_length=120, damping=0.9)  # tighter clustering

    for node, data in G.nodes(data=True):
        ntype = data.get("type")
        subtype = data.get("subtype")
        key = f"sensor_{subtype}" if ntype == "sensor" and subtype else ntype
        color = color_map.get(key, "#8395a7")
        size = 100 if key == "room" else 80 + G.degree(node) * 1.5

        label = key if key else "unknown"
        net.add_node(node, label=label, color=color, size=size)

    for u, v in G.edges():
        net.add_edge(u, v, color="#747373", width=4)

    # --- Legend: Fake nodes at top-left ---
    legend_items = {
        "Light": "#ffe066",
        "Radiator": "#ff6b6b",
        "Sensor (Temperature)": "#4ecdc4",
        "Sensor (Humidity)": "#5f27cd",
        "Room": "#e74c3c"
    }


    # --- Step 6: Show / Save ---
    net.show_buttons(filter_=['physics'])  # optional: enable tweaking layout live
    net.write_html("summer_pyvis_graph.html")
    import webbrowser; webbrowser.open("summer_pyvis_graph.html")



In [ ]:
graph_ok_1 = True

if graph_ok_1:

    !pip install --upgrade pyvis jinja2

    import networkx as nx
    from pyvis.network import Network
    from topologicpy.Vertex import Vertex
    from topologicpy.Topology import Topology
    from topologicpy.Dictionary import Dictionary
    from topologicpy.Face import Face

    # === HELPERS ===

    def get_vertex_id(v):
        d = Topology.Dictionary(v)
        id_val = Dictionary.ValueAtKey(d, "id")
        if id_val:
            return str(id_val)
        else:
            x, y, z = Vertex.Coordinates(v)
            return f"{x:.2f}_{y:.2f}_{z:.2f}"

    # Summer color theme
    color_map = {
        "light": "#ffe066",
        "radiator": "#ff6b6b",
        "sensor_temperature": "#4ecdc4",
        "sensor_humidity": "#5f27cd",
        "room": "#e74c3c",
        "face": "#d3d3d3",
        None: "#8395a7"
    }

    # === SETUP ===

    G = nx.Graph()
    net = Network(height='850px', width='100%', bgcolor='white', font_color='black')
    net.barnes_hut(gravity=-30000, spring_length=120, damping=0.9)

    # === DEVICES ===

    for v in graph_vertices:
        x, y, z = Vertex.Coordinates(v)
        d = Topology.Dictionary(v)
        node_id = get_vertex_id(v)
        vtype = Dictionary.ValueAtKey(d, "type")
        subtype = Dictionary.ValueAtKey(d, "subtype")
        key = f"sensor_{subtype}" if vtype == "sensor" and subtype else vtype
        color = color_map.get(key, "#8395a7")
        size = 80

        G.add_node(node_id, type=key)
        net.add_node(node_id, label=key, color=color, size=size)

    # === ROOMS ===

    room_node_ids = set()

    for i, rv in enumerate(room_vertices):
        x, y, z = Vertex.Coordinates(rv)
        room_id = f"room_{i}"
        room_node_ids.add(room_id)

        G.add_node(room_id, type="room")
        net.add_node(
            room_id,
            label=f"Room {i}",
            x=x * 100,
            y=y * 100,
            color=color_map["room"],
            size=80,
            shape="dot",
            font={"size": 18},
            physics=True
        )

    # === FACES ===

    for i, cell in enumerate(cells):
        room_id = f"room_{i}"
        if room_id not in room_node_ids:
            continue

        faces = Topology.Faces(cell)
        for j, face in enumerate(faces):
            centroid = Topology.Centroid(face)
            if not centroid:
                continue
            x, y, z = Vertex.Coordinates(centroid)
            face_id = f"{room_id}_face_{j}"

            G.add_node(face_id, type="face")
            G.add_edge(room_id, face_id)
            net.add_node(
                face_id,
                label="face",
                color=color_map["face"],
                size=80,
                shape="dot",
                font={"size": 10},
                x=x * 100,
                y=y * 100,
                physics=True
            )
            net.add_edge(room_id, face_id, color="#7a7a7a", width=4)

    # === EDGES ===

    for e in all_edges:
        verts = Topology.Vertices(e)
        if verts and len(verts) == 2:
            id1 = get_vertex_id(verts[0])
            id2 = get_vertex_id(verts[1])
            if id1 != id2:
                G.add_edge(id1, id2)
                net.add_edge(id1, id2, color="#7a7a7a", width=4)

    # === LEGEND ===

    legend_items = {
        "Light": "#ffe066",
        "Radiator": "#ff6b6b",
        "Sensor (Temperature)": "#4ecdc4",
        "Sensor (Humidity)": "#5f27cd",
        "Room": "#e74c3c",
        "Face": "#d3d3d3"
    }

    legend_y = -1000
    legend_x = 0
    x_spacing = 200

    for i, (label, color) in enumerate(legend_items.items()):
        net.add_node(
            f"legend_{i}",
            label=label,
            x=legend_x + i * x_spacing,
            y=legend_y,
            physics=False,
            color=color,
            size=80,
            shape="dot",
            font={"size": 20}
        )

    # === EXPORT ===

    net.write_html("summer_pyvis_graph_with_faces.html")

    import webbrowser
    webbrowser.open("summer_pyvis_graph_with_faces.html")


In [ ]:
graph_ok_2 = True

if graph_ok_2:

    !pip install --upgrade pyvis jinja2

    from pyvis.network import Network
    from topologicpy.Vertex import Vertex
    from topologicpy.Topology import Topology
    from topologicpy.Dictionary import Dictionary
    from topologicpy.Face import Face
    from topologicpy.Cell import Cell

    import math
    import webbrowser

    # === HELPERS ===
    def get_vertex_id(v):
        d = Topology.Dictionary(v)
        id_val = Dictionary.ValueAtKey(d, "id")
        if id_val:
            return str(id_val)
        x, y, z = Vertex.Coordinates(v)
        return f"{x:.2f}_{y:.2f}_{z:.2f}"

    def vertex_distance(v1, v2):
        x1, y1, z1 = Vertex.Coordinates(v1)
        x2, y2, z2 = Vertex.Coordinates(v2)
        return math.sqrt((x1 - x2)**2 + (y1 - y2)**2 + (z1 - z2)**2)

    # === COLOR MAP ===
    color_map = {
        "light": "#ffe066",
        "radiator": "#ff6b6b",
        "sensor_temperature": "#4ecdc4",
        "sensor_humidity": "#5f27cd",
        "room": "#e74c3c",
        "face": "#d3d3d3"
    }

    # === INITIALIZE NETWORK ===
    net = Network(height='850px', width='100%', bgcolor='white', font_color='black')
    net.barnes_hut(gravity=-30000, spring_length=120, damping=0.9)

    # === ROOMS ===
    room_node_ids = []
    for i, rv in enumerate(room_vertices):
        x, y, z = Vertex.Coordinates(rv)
        room_id = f"room_{i}"
        room_node_ids.append(room_id)

        net.add_node(
            room_id,
            label=f"Room {i}",
            x=x * 100,
            y=y * 100,
            color=color_map["room"],
            size=120,
            shape="dot",
            font={"size": 18},
            physics=True
        )

    # === CONNECT ROOMS TO CLOSEST NEIGHBORS ===
    for i, rv1 in enumerate(room_vertices):
        distances = []
        for j, rv2 in enumerate(room_vertices):
            if i != j:
                dist = vertex_distance(rv1, rv2)
                distances.append((j, dist))
        distances.sort(key=lambda x: x[1])
        # Connect to 2 closest rooms
        for j, _ in distances[:2]:
            net.add_edge(f"room_{i}", f"room_{j}", color="#cccccc", width=2)

    # === FACES PER ROOM ===
    for i, cell in enumerate(cells):
        room_id = f"room_{i}"
        faces = Topology.Faces(cell)
        for j, face in enumerate(faces):
            centroid = Topology.Centroid(face)
            if not centroid:
                continue
            x, y, z = Vertex.Coordinates(centroid)
            face_id = f"{room_id}_face_{j}"

            net.add_node(
                face_id,
                label="face",
                color=color_map["face"],
                size=100,
                shape="dot",
                font={"size": 10},
                x=x * 100,
                y=y * 100,
                physics=True
            )
            # This is already linked to the room correctly:
            net.add_edge(room_id, face_id, color="#aaaaaa", width=2)


    # === DEVICES ===
    all_devices = light_vertices + radiator_vertices + sensor_vertices

    for v in all_devices:
        x, y, z = Vertex.Coordinates(v)
        d = Topology.Dictionary(v)
        node_id = get_vertex_id(v)

        vtype = Dictionary.ValueAtKey(d, "type")
        subtype = Dictionary.ValueAtKey(d, "subtype")
        key = f"sensor_{subtype}" if vtype == "sensor" and subtype else vtype
        color = color_map.get(key, "#8395a7")

        net.add_node(node_id, label=key, color=color, size=80)

        # === Use actual containment instead of proximity ===
        connected = False
        for i, cell in enumerate(cells):
            if Topology.IsInside(v, cell):  # topologicpy-style
                net.add_edge(node_id, f"room_{i}", color="#7a7a7a", width=4)
                connected = True
                break

        # fallback: connect to closest if no containment match
        if not connected:
            closest_room_index = min(
                range(len(room_vertices)),
                key=lambda i: vertex_distance(v, room_vertices[i])
            )
            net.add_edge(node_id, f"room_{closest_room_index}", color="#aaaaaa", width=2, title="approx")


    # === LEGEND ===
    legend_items = {
        "Light": "#ffe066",
        "Radiator": "#ff6b6b",
        "Sensor (Temperature)": "#4ecdc4",
        "Sensor (Humidity)": "#5f27cd",
        "Room": "#e74c3c",
        "Face": "#d3d3d3"
    }
    for i, (label, color) in enumerate(legend_items.items()):
        net.add_node(
            f"legend_{i}",
            label=label,
            x=i * 250,
            y=-1000,
            physics=False,
            color=color,
            size=100,
            shape="dot",
            font={"size": 20}
        )

    # === EXPORT AND OPEN ===
    output_file = "final_network_rooms_devices_faces.html"
    net.write_html(output_file)
    webbrowser.open(output_file)



    # Save the HTML output
    net.write_html("simplified_rooms_devices_network.html")

    # Open it in the default browser
    import webbrowser
    webbrowser.open("simplified_rooms_devices_network.html")


In [ ]:
from pyvis.network import Network
from topologicpy.Vertex import Vertex
from topologicpy.Topology import Topology
from topologicpy.Dictionary import Dictionary
from topologicpy.Face import Face
from topologicpy.Cell import Cell

import math
import webbrowser

# === HELPERS ===
def get_vertex_id(v):
    d = Topology.Dictionary(v)
    id_val = Dictionary.ValueAtKey(d, "id")
    if id_val:
        return str(id_val)
    x, y, z = Vertex.Coordinates(v)
    return f"{x:.2f}_{y:.2f}_{z:.2f}"

def vertex_distance(v1, v2):
    x1, y1, z1 = Vertex.Coordinates(v1)
    x2, y2, z2 = Vertex.Coordinates(v2)
    return math.sqrt((x1 - x2)**2 + (y1 - y2)**2 + (z1 - z2)**2)

def is_vertex_near_cell(vertex, cell, tolerance=1.0):
    try:
        verts = Topology.Vertices(cell)
        return any(vertex_distance(vertex, v) < tolerance for v in verts)
    except:
        return False

# === COLOR MAP ===
color_map = {
    "light": "#ffe066",
    "radiator": "#ff6b6b",
    "sensor_temperature": "#4ecdc4",
    "sensor_humidity": "#5f27cd",
    "room": "#e74c3c",
    "face": "#d3d3d3"
}

# === INITIALIZE NETWORK ===
net = Network(height='850px', width='100%', bgcolor='white', font_color='black')
net.barnes_hut(gravity=-2000, spring_length=150, damping=0.15)

# === ROOMS ===
room_node_ids = []
for i, rv in enumerate(room_vertices):
    x, y, z = Vertex.Coordinates(rv)
    room_id = f"room_{i}"
    room_node_ids.append(room_id)

    net.add_node(
        room_id,
        label=f"Room {i}",
        x=x * 100,
        y=y * 100,
        color=color_map["room"],
        size=50,  # 🔹 Smaller
        shape="dot",
        font={"size": 14},
        physics=True
    )

# === CONNECT ROOMS TO CLOSEST NEIGHBORS (2-nearest, no duplicates) ===
added_edges = set()
for i, rv1 in enumerate(room_vertices):
    distances = []
    for j, rv2 in enumerate(room_vertices):
        if i != j:
            dist = vertex_distance(rv1, rv2)
            distances.append((j, dist))
    distances.sort(key=lambda x: x[1])
    for j, _ in distances[:2]:
        edge_key = tuple(sorted([i, j]))
        if edge_key not in added_edges:
            net.add_edge(f"room_{i}", f"room_{j}", color="#cccccc", width=1.5)
            added_edges.add(edge_key)

# === FACES PER ROOM ===
for i, cell in enumerate(cells):
    room_id = f"room_{i}"
    faces = Topology.Faces(cell)
    for j, face in enumerate(faces):
        centroid = Topology.Centroid(face)
        if not centroid:
            continue
        x, y, z = Vertex.Coordinates(centroid)
        face_id = f"{room_id}_face_{j}"

        net.add_node(
            face_id,
            label="face",
            color=color_map["face"],
            size=25,  # 🔹 Smaller
            shape="dot",
            font={"size": 10},
            x=x * 100,
            y=y * 100,
            physics=True
        )
        net.add_edge(room_id, face_id, color="#aaaaaa", width=1.5)

# === DEVICES ===
all_devices = light_vertices + radiator_vertices + sensor_vertices

for v in all_devices:
    x, y, z = Vertex.Coordinates(v)
    d = Topology.Dictionary(v)
    node_id = get_vertex_id(v)

    vtype = Dictionary.ValueAtKey(d, "type")
    subtype = Dictionary.ValueAtKey(d, "subtype")
    key = f"sensor_{subtype}" if vtype == "sensor" and subtype else vtype
    color = color_map.get(key, "#8395a7")

    net.add_node(node_id, label=key, title=f"{key}<br>ID: {node_id}", color=color, size=30)  # 🔹 Smaller

    connected = False
    for i, cell in enumerate(cells):
        if is_vertex_near_cell(v, cell, tolerance=1.0):
            net.add_edge(node_id, f"room_{i}", color="#7a7a7a", width=2)
            connected = True
            break

    if not connected:
        closest_room_index = min(
            range(len(room_vertices)),
            key=lambda i: vertex_distance(v, room_vertices[i])
        )
        net.add_edge(node_id, f"room_{closest_room_index}", color="#aaaaaa", width=1.5, title="approx")

# === LEGEND ===
legend_items = {
    "Light": "#ffe066",
    "Radiator": "#ff6b6b",
    "Sensor (Temperature)": "#4ecdc4",
    "Sensor (Humidity)": "#5f27cd",
    "Room": "#e74c3c",
    "Face": "#d3d3d3"
}
for i, (label, color) in enumerate(legend_items.items()):
    net.add_node(
        f"legend_{i}",
        label=label,
        x=i * 250,
        y=-1200,
        physics=False,
        color=color,
        size=40,  # 🔹 Smaller
        shape="dot",
        font={"size": 14}
    )

# === EXPORT AND OPEN ===
output_file = "final_network_rooms_devices_faces.html"
net.write_html(output_file)
webbrowser.open(output_file)

# Optional: simplified version
net.write_html("simplified_rooms_devices_network.html")
webbrowser.open("simplified_rooms_devices_network.html")


In [ ]:
from pyvis.network import Network
from topologicpy.Vertex import Vertex
from topologicpy.Topology import Topology
from topologicpy.Dictionary import Dictionary
from topologicpy.Face import Face
from topologicpy.Cell import Cell

import math
import webbrowser

# === HELPERS ===
def get_vertex_id(v):
    d = Topology.Dictionary(v)
    id_val = Dictionary.ValueAtKey(d, "id")
    if id_val:
        return str(id_val)
    x, y, z = Vertex.Coordinates(v)
    return f"{x:.2f}_{y:.2f}_{z:.2f}"

def vertex_distance(v1, v2):
    x1, y1, z1 = Vertex.Coordinates(v1)
    x2, y2, z2 = Vertex.Coordinates(v2)
    return math.sqrt((x1 - x2)**2 + (y1 - y2)**2 + (z1 - z2)**2)

# === COLOR MAP ===
color_map = {
    "light": "#ffe066",
    "radiator": "#ff6b6b",
    "sensor_temperature": "#4ecdc4",
    "sensor_humidity": "#5f27cd",
    "room": "#e74c3c",
    "wall": "#d3d3d3",
    "floor": "#7D7D7D",     # light blue
    "ceiling": "#7D7D7D"    # plum
}

# === INITIALIZE NETWORK ===
net = Network(height='850px', width='100%', bgcolor='white', font_color='black')
net.barnes_hut(gravity=-2000, spring_length=150, damping=0.15)

# === ROOMS ===
room_node_ids = []
for i, rv in enumerate(room_vertices):
    x, y, z = Vertex.Coordinates(rv)
    room_id = f"room_{i}"
    room_node_ids.append(room_id)

    net.add_node(
        room_id,
        label=f"Room {i}",
        x=x * 100,
        y=y * 100,
        color=color_map["room"],
        size=50,
        shape="dot",
        font={"size": 14},
        physics=True
    )

# === CONNECT ROOMS TO CLOSEST NEIGHBORS (2-nearest) ===
added_edges = set()
for i, rv1 in enumerate(room_vertices):
    distances = []
    for j, rv2 in enumerate(room_vertices):
        if i != j:
            dist = vertex_distance(rv1, rv2)
            distances.append((j, dist))
    distances.sort(key=lambda x: x[1])
    for j, _ in distances[:2]:
        edge_key = tuple(sorted([i, j]))
        if edge_key not in added_edges:
            net.add_edge(f"room_{i}", f"room_{j}", color="#BB1B1B", width=1.5)
            added_edges.add(edge_key)

# === FACES PER ROOM ===
from topologicpy.Face import Face as TopoFace  # Ensure this is at the top

# === FACES PER ROOM ===
for i, cell in enumerate(cells):
    room_id = f"room_{i}"
    faces = Topology.Faces(cell)
    for j, face in enumerate(faces):
        centroid = Topology.Centroid(face)
        if not centroid:
            continue
        x, y, z = Vertex.Coordinates(centroid)

        normal = TopoFace.Normal(face)
        nx, ny, nz = normal if normal else (0, 0, 0)

        # Determine face type
        if abs(nz) > 0.9:
            face_type = "ceiling" if nz > 0 else "floor"
        else:
            face_type = "wall"

        face_id = f"{room_id}_face_{j}"

        net.add_node(
            face_id,
            label=face_type,
            title=f"{face_type} face",
            color=color_map[face_type],  # 🟡 Updated
            size=25,
            shape="dot",
            font={"size": 10},
            x=x * 100,
            y=y * 100,
            physics=True
        )
        net.add_edge(room_id, face_id, color="#777777", width=1.5)

# === DEVICES ===
all_devices = light_vertices + radiator_vertices + sensor_vertices

for v in all_devices:
    x, y, z = Vertex.Coordinates(v)
    d = Topology.Dictionary(v)
    node_id = get_vertex_id(v)

    vtype = Dictionary.ValueAtKey(d, "type")
    subtype = Dictionary.ValueAtKey(d, "subtype")
    key = f"sensor_{subtype}" if vtype == "sensor" and subtype else vtype
    color = color_map.get(key, "#8395a7")

    net.add_node(node_id, label=key, title=f"{key}<br>ID: {node_id}", color=color, size=30)

    connected = False
    for i, cell in enumerate(cells):
        try:
            if Topology.IsInside(cell, v):  # Best method
                net.add_edge(node_id, f"room_{i}", color="#7a7a7a", width=2)
                connected = True
                break
        except:
            # Fallback: close to centroid
            centroid = Topology.Centroid(cell)
            if centroid and vertex_distance(v, centroid) < 3.0:
                net.add_edge(node_id, f"room_{i}", color="#7a7a7a", width=2)
                connected = True
                break

    if not connected:
        # Final fallback: closest room vertex
        closest_room_index = min(
            range(len(room_vertices)),
            key=lambda i: vertex_distance(v, room_vertices[i])
        )
        net.add_edge(node_id, f"room_{closest_room_index}", color="#aaaaaa", width=1.5, title="approx")

legend_items = {
    "Light": color_map["light"],
    "Radiator": color_map["radiator"],
    "Sensor (Temperature)": color_map["sensor_temperature"],
    "Sensor (Humidity)": color_map["sensor_humidity"],
    "Room": color_map["room"],
    "Wall": color_map["wall"],
    "Floor": color_map["floor"],
    "Ceiling": color_map["ceiling"]
}


legend_x = -500
legend_y = -1000
spacing_y = 80

# Legend Title
net.add_node(
    "legend_title",
    label="Legend",
    x=legend_x,
    y=legend_y,
    physics=False,
    shape="box",
    color="#f1f2f6",
    font={"size": 18, "color": "black"}
)

# Legend Items
for i, (label, color) in enumerate(legend_items.items()):
    net.add_node(
        f"legend_{i}",
        label=label,
        x=legend_x,
        y=legend_y - ((i + 1) * spacing_y),
        physics=False,
        color=color,
        size=35,
        shape="dot",
        font={"size": 14}
    )

# === EXPORT AND OPEN ===
output_file = "final_network_rooms_devices_faces.html"
net.write_html(output_file)
webbrowser.open(output_file)

# Optional: simplified version
net.write_html("simplified_rooms_devices_network.html")
webbrowser.open("simplified_rooms_devices_network.html")



BIM TRASFORMATION